In [4]:
import os
from pathlib import Path

# =================================================================
# 1. 設定: 変更したい文字列と対象ディレクトリ
# =================================================================

# 変更したい（削除したい）文字列のプレフィックス
PREFIX_TO_REMOVE = "parse_error_"

# (おそらく前のセルで定義されていますが、念のため)
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"
OUTPUT_DIR = os.path.join(BASE_DIR, "structure/MD_Al")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
# TENSILE_DIR = os.path.join(OUTPUT_DIR, "tensile01")
# COMPRESSION_DIR = os.path.join(OUTPUT_DIR, "compression")

# ★ プレフィックスを変更したいファイルが存在するディレクトリのリスト
# (HEAT_DIR, TENSILE_DIR など、スクリプトで定義済みの変数を使えます)
directories_to_clean = [
    HEAT_DIR,
    # COMPRESSION_DIR, # 必要に応じて追加
    # OUTPUT_DIR,        # 必要に応じて追加
]

# =================================================================
# 2. リネーム実行処理
# =================================================================

print("ファイルのリネーム処理を開始します...")

total_renamed_count = 0

for directory_path in directories_to_clean:
    
    target_dir = Path(directory_path) # Pathオブジェクトに変換
    
    if not target_dir.is_dir():
        print(f"\n⚠️ スキップ: ディレクトリが見つかりません: {target_dir}")
        continue

    print(f"\n--- ディレクトリをチェック中: {target_dir} ---")
    
    renamed_in_this_dir = 0
    
    # .iterdir() でディレクトリ内の全ファイル/フォルダを取得
    for file_path in target_dir.iterdir():
        
        # ファイルのみを対象にし、指定したプレフィックスで始まるかチェック
        if file_path.is_file() and file_path.name.startswith(PREFIX_TO_REMOVE):
            
            # --- 新しいファイル名を生成 ---
            # "parse_error_" の部分を削除
            # (Python 3.9+ で利用可能な removeprefix を使用)
            try:
                new_filename = file_path.name.removeprefix(PREFIX_TO_REMOVE)
            except AttributeError:
                # Python 3.8 以前の場合のフォールバック
                new_filename = file_path.name[len(PREFIX_TO_REMOVE):]

            
            # --- 新しいファイルパスを生成 ---
            new_file_path = file_path.with_name(new_filename)
            
            # --- リネーム実行 ---
            try:
                file_path.rename(new_file_path)
                print(f"  ✅ Renamed: '{file_path.name}'\n"
                      f"        ->  '{new_file_path.name}'")
                renamed_in_this_dir += 1
            
            except OSError as e:
                print(f"  ❌ エラー: '{file_path.name}' のリネームに失敗しました: {e}")
    
    if renamed_in_this_dir == 0:
        print("  (対象ファイルはありませんでした)")
    
    total_renamed_count += renamed_in_this_dir

print("\n" + "="*50)
print(f"処理が完了しました。合計 {total_renamed_count} 個のファイルをリネームしました。")
print("="*50)

ファイルのリネーム処理を開始します...

--- ディレクトリをチェック中: /home/jovyan/Kaori/MD/LiB_2/structure/MD_Al/heat ---
  ✅ Renamed: 'parse_error_Interface_Al_on_NMC_Co_increase_104_P0.000_T300K_equil.traj'
        ->  'Interface_Al_on_NMC_Co_increase_104_P0.000_T300K_equil.traj'
  ✅ Renamed: 'parse_error_Interface_Al_on_NMC_Ni_decrease_012_P0.000_T300K_HT500K.traj'
        ->  'Interface_Al_on_NMC_Ni_decrease_012_P0.000_T300K_HT500K.traj'
  ✅ Renamed: 'parse_error_Interface_Al_on_NMC_NMC622_approx_104_P0.000_T300K_HT500K.traj'
        ->  'Interface_Al_on_NMC_NMC622_approx_104_P0.000_T300K_HT500K.traj'
  ✅ Renamed: 'parse_error_Interface_Al_on_NMC_NMC811_approx_010_P0.000_T300K_equil.traj'
        ->  'Interface_Al_on_NMC_NMC811_approx_010_P0.000_T300K_equil.traj'
  ✅ Renamed: 'parse_error_Interface_Al_on_NMC_NMC811_approx_010_P0.000_T300K_HT500K.traj'
        ->  'Interface_Al_on_NMC_NMC811_approx_010_P0.000_T300K_HT500K.traj'
  ✅ Renamed: 'parse_error_Interface_Al_on_NMC_NMC622_approx_001_P0.000_T300K_equil.t

In [15]:
#!/usr/bin/env python3
"""
メモリ効率改善版 統合NMC-Al界面構造生成・最適化システム
リチウムイオン電池リサイクル研究用

主な改善点:
- メモリ使用量を最小化
- 一つずつ処理して即座に保存
- 大量のデータを同時にメモリに保持しない
- エラー耐性の向上
- 必要最小限の構造のみ生成
"""

import numpy as np
import os, sys, time
from pathlib import Path
import random
import math
import gc  # ガベージコレクション用
# ASE
import ase
from ase import Atoms, units
from ase.io import read, write
from ase.build import surface, fcc100,bulk
from ase.constraints import FixAtoms
from ase.optimize import BFGS

# PFP (Matlantis) - エラー耐性版
try:
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    from pfp_api_client.pfp.estimator import Estimator
    from matlantis_features.atoms import MatlantisAtoms
    from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
    from matlantis_features.utils.calculators import pfp_estimator_fn
    from pfp_api_client.pfp.estimator import EstimatorCalcMode
    
    estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
    calculator = ASECalculator(estimator)
    MATLANTIS_AVAILABLE = True
    print("✅ PFP/Matlantis計算器を正常に初期化")
    
except ImportError as e:
    print(f"⚠️ PFP/Matlantisが利用できません: {e}")
    print("  基本ASE最適化のみを使用")
    calculator = None
    MATLANTIS_AVAILABLE = False

class MemoryEfficientNMCSystem:
    """メモリ効率重視のNMC-Al界面構造システム"""
    
    def __init__(self, base_dir="/home/jovyan/Kaori/MD/LiB_2"):
        self.base_dir = Path(base_dir)
        self.output_dir = self.base_dir / "structure" 
        self.cif_dir = self.output_dir / "opt"
        
        # ディレクトリ作成
        for dir_path in [self.output_dir, self.cif_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)
        
        # 計算器設定
        self.calculator = calculator
        
        # Miller指数リスト（重要度順）
        self.miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 0, 4),  # NMC(104) - 中活性、多面体形状
            (0, 1, 2),  # NMC(012) - 高活性、切頭八面体形状
            (0, 1, 0),  # NMC(010) - Li拡散活性面
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        self.al_miller_indices = [
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        self.al2o3_miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
        ]
        self.alf3_miller_indices = [
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
        ]
        
        # シミュレーション計画の定義
        self.simulation_plan = [
            # ニッケル(Ni)増減の影響
            {"name": "Ni_increase", "composition": {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}},
            {"name": "Ni_decrease", "composition": {"Ni": 0.2, "Mn": 0.4, "Co": 0.4}},
            
            # マンガン(Mn)増減の影響
            {"name": "Mn_increase", "composition": {"Ni": 0.2, "Mn": 0.6, "Co": 0.2}},
            {"name": "Mn_decrease", "composition": {"Ni": 0.4, "Mn": 0.2, "Co": 0.4}},
            
            # コバルト(Co)増減の影響
            {"name": "Co_increase", "composition": {"Ni": 0.2, "Mn": 0.2, "Co": 0.6}},
            {"name": "Co_decrease", "composition": {"Ni": 0.4, "Mn": 0.4, "Co": 0.2}},
            
            # ハイニッケル化トレンド
            {"name": "NMC523_approx", "composition": {"Ni": 0.50, "Mn": 0.25, "Co": 0.25}},
            {"name": "NMC622_approx", "composition": {"Ni": 0.60, "Mn": 0.20, "Co": 0.20}},
            {"name": "NMC811_approx", "composition": {"Ni": 0.80, "Mn": 0.10, "Co": 0.10}},
        ]
        
        print(f"✅ メモリ効率システム初期化完了")
        print(f"  出力ディレクトリ: {self.output_dir}")
        print(f"  CIF保存先: {self.cif_dir}")
    
    def load_or_create_nmc_base(self):
        """NMC基本構造の読み込みまたは作成（元の機能を復元）"""
        input_dir = Path("/home/jovyan/Kaori/MD/input")
        cif_path = input_dir / "Li3MnCoNiO6.cif"
        
        if cif_path.exists():
            print(f"CIFファイルから読み込み: {cif_path}")
            try:
                base_atoms = read(str(cif_path))
                print(f"✅ CIF読み込み成功: {len(base_atoms)} atoms")
                return base_atoms
            except Exception as e:
                print(f"❌ CIF読み込み失敗: {e}")
        
        print("CIFファイルがないため、基本構造を生成します")
        return self.create_nmc_base_structure()
    
    # def create_nmc_base_structure(self):
    #     """NMC基本構造の作成（簡素版）"""
    #     print("NMC基本構造を作成中...")
        
    #     # 簡素化したNMC構造
    #     a = 2.87  # 格子定数
    #     c = 14.2
        
    #     positions = [
    #         [0.0, 0.0, 0.0],      # Li
    #         [0.5, 0.5, 0.0],      # Li
    #         [0.0, 0.5, 0.25],     # Ni
    #         [0.5, 0.0, 0.25],     # Mn
    #         [0.0, 0.0, 0.125],    # O
    #         [0.5, 0.5, 0.125],    # O
    #         [0.0, 0.5, 0.375],    # O
    #         [0.5, 0.0, 0.375],    # O
    #         [0.0, 0.0, 0.5],      # Co
    #         [0.5, 0.5, 0.5],      # Ni
    #     ]
        
    #     symbols = ['Li', 'Li', 'Ni', 'Mn', 'O', 'O', 'O', 'O', 'Co', 'Ni']
    #     cell = [[a, 0, 0], [0, a, 0], [0, 0, c]]
        
    #     atoms = Atoms(symbols=symbols, 
    #                  positions=np.array(positions) * [a, a, c],
    #                  cell=cell, 
    #                  pbc=True)
        
    #     print(f"✅ NMC基本構造作成完了: {len(atoms)} atoms")
    #     return atoms
    
    def create_al_structure(self):
        """Al基本構造の作成"""
        print("Al基本構造を作成中...")
        al_atoms = bulk("Al", "fcc", a=4.05, cubic=True)
        # al2o3_atoms = read('/home/jovyan/Kaori/MD/input/Al2O3.cif') 
        # alf3_atoms = read('/home/jovyan/Kaori/MD/input/AlF3_cubic.cif') 
        print(f"✅ Al構造作成完了: {len(al_atoms)} atoms")
        return al_atoms
    
    def modify_composition(self, atoms, target_composition, name="variant"):
        """組成変更（即座に保存）"""
        print(f"組成変更中: {name}")
        # if cif_path.exists():
        #     print(f"  既存ファイルをスキップ: {cif_path.name}")
        #     return str(cif_path)
        try:
            variant_atoms = atoms.copy()
            tm_indices = [i for i, atom in enumerate(variant_atoms) 
                         if atom.symbol in ["Ni", "Mn", "Co"]]
            
            if len(tm_indices) == 0:
                print(f"  警告: 遷移金属が見つかりません")
                return None
            
            total_tm_sites = len(tm_indices)
            target_counts = {symbol: round(fraction * total_tm_sites) 
                           for symbol, fraction in target_composition.items()}
            
            # 四捨五入調整
            current_total = sum(target_counts.values())
            diff = total_tm_sites - current_total
            if diff != 0:
                most_abundant = max(target_composition, key=target_composition.get)
                target_counts[most_abundant] += diff
            
            # 新しい元素リストを作成してシャッフル
            new_symbols = []
            for symbol, count in target_counts.items():
                new_symbols.extend([symbol] * count)
            
            random.seed(42)
            random.shuffle(tm_indices)
            
            for i, atom_index in enumerate(tm_indices):
                variant_atoms[atom_index].symbol = new_symbols[i]
            
            # 即座にCIF保存
            cif_path = self.cif_dir / f"NMC_{name}_bulk.cif"
            write(str(cif_path), variant_atoms)
            print(f"  ✅ 保存完了: {cif_path.name}")
            
            return variant_atoms
            
        except Exception as e:
            print(f"  ❌ 組成変更エラー: {e}")
            return None
        finally:
            # メモリクリーンアップ
            gc.collect()
    
    def optimize_structure_systematic(self, atoms, name="structure"):
        """
        構造最適化（BFGS + Matlantis 2段階版）
        1. ASEのBFGSで格子と原子位置を予備的に最適化
        2. Matlantis Featuresで最終的な精密最適化
        """
        print(f"=== {name}の構造最適化開始 (BFGS → Matlantis) ===")

        if self.calculator is None or not MATLANTIS_AVAILABLE:
            print("  -> ⚠️ 計算器が利用できないため、最適化をスキップします")
            return atoms

        try:
            # --- Step 1: ASEのBFGSによる予備最適化 ---
            print("  -> Step 1: BFGSによる予備最適化 (fmax=0.05)")
            write(f"{name}.xyz",atoms)
            # 計算機をatomsオブジェクトにセット
            temp_estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
            atoms.calc = ASECalculator(temp_estimator)

            # UnitCellFilterを使って格子と原子位置を同時に最適化
            bfgs_optimizer = BFGS(atoms)
            bfgs_optimizer.run(fmax=0.05, steps=100) # やや緩めの条件で実行
            
            print(f"    -> BFGS最適化完了。中間エネルギー: {atoms.get_potential_energy():.3f} eV")

            # --- Step 2: Matlantis Featuresによる精密最適化 ---
            print("  -> Step 2: Matlantis Featuresによる精密最適化 (fmax=0.01)")
            
            # BFGSで最適化された構造をMatlantisオプティマイザに入力
            estimator_function = pfp_estimator_fn(
                model_version='v7.0.0',
                calc_mode=EstimatorCalcMode.CRYSTAL_U0_PLUS_D3
            )

            matlantis_optimizer = FireLBFGSASEOptFeature(
                estimator_fn=estimator_function,
                # filter=True,  # 格子も最終調整する
                # trajectory=str(self.output_dir / f"{name}_matlantis_final.traj"),
                n_run=3000,
                fmax=0.01,  # より厳密な条件
                show_progress_bar=True
            )
            
            result = matlantis_optimizer(MatlantisAtoms(atoms))
            optimized_atoms = result.atoms.ase_atoms
            final_energy = result.output.energy_log[-1]
            
            print(f"    -> ✔️ Matlantis最適化完了。最終エネルギー: {final_energy:.3f} eV")

            return optimized_atoms

        except Exception as e:
            print(f"  -> ❌ 最適化中にエラーが発生しました: {e}")
            # エラーが発生した場合は、元の構造をそのまま返す
            return atoms
        finally:
            # メモリクリーンアップ
            gc.collect()
    
    def create_surface_and_save(self, atoms, miller, base_name):
        """表面作成と即座保存"""
        miller_str = f"{miller[0]}{miller[1]}{miller[2]}"
        surface_name = f"{base_name}_{miller_str}"
        cif_path = self.cif_dir / f"{surface_name}_surface.cif"
        
        # 既存ファイルのチェック
        if cif_path.exists():
            print(f"  既存ファイルをスキップ: {cif_path.name}")
            return str(cif_path)
        
        try:
            print(f"  表面作成中: {surface_name}")
            
            # 元の設定値に戻す
            slab_unit = surface(atoms, miller, layers=4, vacuum=0.0)
            initial_dims=slab_unit.cell.lengths()
            def repeat_slab_to_thickness(slab, target_thickness):
                unit_thickness = slab.cell.lengths()[2]
                if unit_thickness > 0:
                    repeat_z = math.ceil(target_thickness / unit_thickness)
                else:
                    repeat_z = 1
                return slab.repeat((1, 1, repeat_z))
            
            # 呼び出し側
            slab = repeat_slab_to_thickness(slab_unit, 25.0)
            # target_dims = np.array([12, 12, 25])
            # repean_num=np.ceil(target_dims / initial_dims).astype(int)
            # slab = slab_unit.repeat(repeat_num)  # 元のリピート数に戻す
            
            # 位置調整
            positions = slab.get_positions()
            z_min = positions[:, 2].min()
            slab.set_positions(positions - [0, 0, z_min])
            
            # 下部原子の固定（元の固定比率）
            z_coords = slab.positions[:, 2]
            z_threshold = z_coords.min() + (z_coords.max() - z_coords.min()) * 0.3
            fixed_indices = [i for i, z in enumerate(z_coords) if z <= z_threshold]
            
            if fixed_indices:
                constraint = FixAtoms(indices=fixed_indices)
                slab.set_constraint(constraint)
            
            # 軽い最適化
            slab = self.optimize_structure_systematic(slab, surface_name)
            
            # 即座に保存
            write(str(cif_path), slab)
            print(f"  ✅ 表面保存完了: {cif_path.name}")
            
            return str(cif_path)
            
        except Exception as e:
            print(f"  ❌ 表面作成失敗: {surface_name} - {e}")
            return None
        finally:
            # メモリクリーンアップ
            # del slab if 'slab' in locals() else None
            gc.collect()
    
    def create_interface_and_save(self, surface1_path, surface2_path):
        """界面作成と即座保存（元の複雑なトリミング処理を復元）"""
        # try:
            # ファイル名から界面名を生成
        name1 = Path(surface1_path).stem.replace('_surface', '')
        name2 = Path(surface2_path).stem.replace('_surface', '')
        interface_name = f"Interface_{name2}_on_{name1}"
        cif_path = self.cif_dir / f"{interface_name}.cif"
        # 既存ファイルのチェック
        if cif_path.exists():
            print(f"  既存界面をスキップ: {cif_path.name}")
            return str(cif_path)
        
        print(f"  界面作成中: {interface_name}")
        
        # 表面構造の読み込み
        slab1 = read(surface1_path)
        slab2 = read(surface2_path)
        separation = 2.0
        # XY方向のトリミング
        unit_thickness1 = slab1.cell.lengths()[2]
        unit_thickness_xy1=[slab1.cell.lengths()[0],slab1.cell.lengths()[1]]
        # 目標とする厚みを設定
        target_thickness = 19.5
        target_xy=[slab2.cell.lengths()[0],slab2.cell.lengths()[1]]
        
        # 必要なZ方向の繰り返し回数を計算 (小数点以下を切り上げ)
        if unit_thickness1 > 0:
            repeat_z1 = math.ceil(target_thickness / unit_thickness1)
        else:
            repeat_z = 1 # 厚みが0の場合は1回とする
        # 計算した回数でslab1をZ方向に繰り返す
        
        if unit_thickness_xy1[0]> 0:
            repeat_x1 = math.ceil(target_xy[0]/ unit_thickness_xy[0])
        else:
            repeat_x1=1#厚みが0の場合は1回とする
        
        if unit_thickness_xy1[1]> 0:
            repeat_y1 = math.ceil(target_xy[1]/ unit_thickness_xy[1])
        else:
            repeat_y1=1 # 厚みが0の場合は1回とする
        
        final_slab1 = slab1.repeat((repeat_x1,repeat_y1, repeat_z1))
        unit_thickness2 = slab2.cell.lengths()[2]
        # 必要なZ方向の繰り返し回数を計算 (小数点以下を切り上げ)
        if unit_thickness2 > 0:
            repeat_z2 = math.ceil(target_thickness / unit_thickness2)
        else:
            repeat_z = 1 # 厚みが0の場合は1回とする
        # 計算した回数でslab1をZ方向に繰り返す
        final_slab2 = slab2.repeat((1, 1, repeat_z2))
        
        pos1 = final_slab1.get_positions()
        pos2 = final_slab2.get_positions()
        x_max=pos2[:,0].max()
        y_max=pos2[:,1].max()
        target_xy=(x_max,y_max)
        indices_to_delete1 = np.where((pos1[:, 1] < 0)|(pos1[:, 0] < 0)| (pos1[:, 0] >= target_xy[0]) | 
                                     (pos1[:, 1] >= target_xy[1]) | (pos1[:, 2] > 25))[0]
        indices_to_delete2 = np.where((pos2[:, 1] < 0)|(pos2[:, 0] < 0)|(pos2[:, 0] >= target_xy[0]) | 
                                     (pos2[:, 1] >= target_xy[1]) | (pos2[:, 2] > 25))[0]
        
        cut_slab1, cut_slab2 = final_slab1.copy(), final_slab2.copy()
        if len(indices_to_delete1) > 0: 
            del cut_slab1[indices_to_delete1]
        if len(indices_to_delete2) > 0: 
            del cut_slab2[indices_to_delete2]
        
        # Z方向の位置調整
        z1_max = cut_slab1.positions[:, 2].max()
        z2_min = cut_slab2.positions[:, 2].min()
        cut_slab2.positions[:, 2] += z1_max - z2_min + separation
        
        # 界面の結合
        interface = cut_slab1 + cut_slab2
        interface.cell = [x_max,y_max , 53]
        interface.pbc = [True, True, False]
        interface.center(vacuum=1.0, axis=2)
        
        # STEP 2: 界面のトリミング（元の複雑な処理）
        # interface = self.trim_interface(interface, thickness_per_slab=25.0)
        # if interface is None:
        #     print("   -> ❌ トリミングに失敗")
        #     return None
        
        # STEP 3: 構造最適化
        indices_to_fix = [atom.index for atom in interface if atom.position[2] <= 3.0]

        if indices_to_fix:
            print(f"    下部 {len(indices_to_fix)} 個の原子を固定します。")
            constraint = FixAtoms(indices=indices_to_fix)
            interface.set_constraint(constraint)
        else:
            print("    警告: 固定する原子が見つかりませんでした。")

        interface = self.optimize_structure_systematic(interface, interface_name)
        interface.set_constraint() 
        # 即座に保存
        write(str(cif_path), interface)
        print(f"  ✅ 界面保存完了: {cif_path.name}")
        
        return str(cif_path)
            
        # except Exception as e:
        #     print(f"  ❌ 界面作成失敗: {e}")
        #     return None
        # finally:
        #     # メモリクリーンアップ
        #     for var in ['slab1', 'slab2', 'cut_slab1', 'cut_slab2', 'interface']:
        #         if var in locals():
        #             del locals()[var]
        #     gc.collect()
    
    def trim_interface(self, atoms, thickness_per_slab=25.0):
        """界面のトリミング（元の複雑な処理を復元）"""
        print(f"    2. 界面をトリミング中 (各スラブ厚み: {thickness_per_slab} Å)...")
        
        # 界面のZ座標を推定
        al_indices = [a.index for a in atoms if a.symbol == 'Al']
        nmc_indices = [a.index for a in atoms if a.symbol not in ['Al']]
        
        if not al_indices or not nmc_indices: 
            return None
            
        al_pos = atoms.get_positions()[al_indices]
        nmc_pos = atoms.get_positions()[nmc_indices]
        
        if np.mean(al_pos[:, 2]) < np.mean(nmc_pos[:, 2]):
            interface_z = (al_pos[:, 2].max() + nmc_pos[:, 2].min()) / 2
            is_al_bottom = True
        else:
            interface_z = (nmc_pos[:, 2].max() + al_pos[:, 2].min()) / 2
            is_al_bottom = False

        # 削除する原子を特定
        indices_to_delete = []
        for atom in atoms:
            if is_al_bottom:
                if atom.symbol == 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol != 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)
            else:
                if atom.symbol != 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol == 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)

        print(f"       -> {len(indices_to_delete)} 個の原子を削除")
        if len(indices_to_delete) > 0:
            del atoms[indices_to_delete]
        
        return atoms
    
    def process_single_nmc_variant(self, base_atoms, composition_info):
        """単一のNMC変異体の処理（メモリ効率版）"""
        name = composition_info["name"]
        composition = composition_info["composition"]
        
        print(f"\n--- {name}の処理開始 ---")
        
        # 1. 組成変更と保存
        variant_atoms = self.modify_composition(base_atoms, composition, name)
        if variant_atoms is None:
            return []
        
        # 2. 表面作成と保存（全Miller指数対応）
        surface_files = []
        for miller in self.miller_indices[:4]:  # NMC面のみ
            surface_file = self.create_surface_and_save(variant_atoms, miller, f"NMC_{name}")
            if surface_file:
                surface_files.append(surface_file)
        
        # メモリクリーンアップ
        del variant_atoms
        gc.collect()
        
        return surface_files
    
    def run_memory_efficient_workflow(self):
        """メモリ効率重視のワークフロー実行"""
        print("🚀 メモリ効率NMC-Al界面システム 開始")
        print("=" * 60)
        
        start_time = time.time()
        all_surface_files = []
        
        try:
            # 基本構造作成
            print("\n📐 基本構造作成")
            base_nmc = self.load_or_create_nmc_base()  # 元の関数名に修正
            base_al = self.create_al_structure()
            
            # NMC組成変異体の処理（一つずつ）
            print("\n🔬 NMC組成変異体処理")
            nmc_surfaces = []
            
            # pristine版も追加
            pristine_surfaces = self.process_single_nmc_variant(
            base_nmc, {"name": "pristine", "composition": {"Ni": 1/3, "Mn": 1/3, "Co": 1/3}})
            nmc_surfaces.extend(pristine_surfaces)
            
            # 全組成変異体の処理
            for comp_info in self.simulation_plan:
                surfaces = self.process_single_nmc_variant(base_nmc, comp_info)
                nmc_surfaces.extend(surfaces)
                # 各処理後にメモリクリーンアップ
                gc.collect()
            
            # Al表面の処理（全Miller指数）
            print("\n⚗️ Al表面処理")
            al_surfaces = []
            for miller in self.al_miller_indices[3:]:  # Al面のみ
                surface_file = self.create_surface_and_save(base_al, miller, "Al")
                if surface_file:
                    al_surfaces.append(surface_file)
            al2o3_atoms=read("/home/jovyan/Kaori/MD/LiB_2/structure/output/Al2O3_pos_opt_final.xyz")
            al2o3_path = self.cif_dir / "Al2O3_surface.cif" 
            write(al2o3_path, al2o3_atoms)
            # Atomsオブジェクトではなく、ファイルパスをリストに追加
            al_surfaces.append(str(al2o3_path))
            alf3_atoms=read("/home/jovyan/Kaori/MD/LiB_2/structure/output/AlF3_pos_opt_final.xyz")
            alf3_path = self.cif_dir / "AlF3_surface.cif" 
            write(alf3_path, alf3_atoms)
            al_surfaces.append(str(alf3_path))
            # Atomsオブジェクトではなく、ファイルパスをリストに追加
            # al_surfaces.append(al2o3)
            # al_surfaces.append(alf3)
            # メモリクリーンアップ
            del base_nmc, base_al
            gc.collect()
            
            # 界面作成（一つずつ処理）
            print("\n🔗 界面構造作成")
            interface_count = 0
            total_combinations = len(nmc_surfaces) * len(al_surfaces)
            current = 0
            
            for nmc_surface in nmc_surfaces:
                for al_surface in al_surfaces:
                    current += 1
                    print(f"  進行状況: {current}/{total_combinations}")
                    interface_file = self.create_interface_and_save(nmc_surface, al_surface)
                    if interface_file:
                        interface_count += 1
                    # 各界面作成後にメモリクリーンアップ
                    gc.collect()
            
            # 最終結果
            execution_time = time.time() - start_time
            
            print(f"\n✅ 処理完了!")
            print(f"  実行時間: {execution_time:.1f}秒")
            print(f"  NMC表面: {len(nmc_surfaces)}個")
            print(f"  Al表面: {len(al_surfaces)}個") 
            print(f"  界面構造: {interface_count}個")
            
            # 保存されたファイルの確認
            cif_files = list(self.cif_dir.glob("*.cif"))
            print(f"  保存されたCIFファイル総数: {len(cif_files)}個")
            
            # サマリー保存
            self.save_summary(len(nmc_surfaces), len(al_surfaces), interface_count, execution_time)
            
            return True
            
        except Exception as e:
            print(f"\n❌ ワークフロー実行エラー: {e}")
            return False
        finally:
            # 最終メモリクリーンアップ
            gc.collect()
    
    def save_summary(self, nmc_count, al_count, interface_count, exec_time):
        """実行サマリーの保存"""
        summary = f"""メモリ効率NMC-Al界面構造システム 実行結果
========================================
実行日時: {time.strftime('%Y-%m-%d %H:%M:%S')}
実行時間: {exec_time:.1f}秒

生成された構造:
- NMC表面構造: {nmc_count}個
- Al表面構造: {al_count}個  
- 界面構造: {interface_count}個

保存先: {self.cif_dir}

処理した組成:
- NMC pristine (Ni:Mn:Co = 1:1:1)
- Ni増減バリアント (高活性/低活性)
- Mn増減バリアント (構造安定性) 
- Co増減バリアント (電気化学特性)
- ハイニッケル化トレンド (NMC523/622/811)

処理したMiller指数:
- (001) NMC最安定面、プレート形状
- (104) NMC中活性、多面体形状
- (012) NMC高活性、切頭八面体形状
- (010) NMC Li拡散活性面
- (111) Al最安定面
- (100) 基本面
- (110) 副次面
"""
        
        summary_path = self.output_dir / "execution_summary.txt"
        with open(summary_path, "w", encoding="utf-8") as f:
            f.write(summary)
        
        print(f"📄 サマリー保存: {summary_path}")

def main():
    """メイン実行関数"""
    print("=" * 70)
    print("🧪 メモリ効率NMC-Al界面構造システム")
    print("   リチウムイオン電池リサイクル研究用")
    print("=" * 70)
    
    try:
        # システム初期化
        system = MemoryEfficientNMCSystem()
        
        # ワークフロー実行
        success = system.run_memory_efficient_workflow()
        
        if success:
            print("\n🎉 全処理が正常完了!")
            print(f"📂 結果: {system.cif_dir}")
        else:
            print("\n⚠️ 一部処理でエラーが発生しましたが、")
            print("   完了した結果は保存されています")
            
    except Exception as e:
        print(f"\n💥 システムエラー: {e}")
        
        # 最小限のフォールバック
        try:
            print("\n🔄 フォールバック実行...")
            fallback_system = MemoryEfficientNMCSystem()
            base_nmc = fallback_system.load_or_create_nmc_base()  # 元の関数名に修正
            
            fallback_path = fallback_system.cif_dir / "NMC_fallback.cif"
            write(str(fallback_path), base_nmc)
            print(f"✅ フォールバック完了: {fallback_path}")
            
        except Exception as fb_error:
            print(f"❌ フォールバックも失敗: {fb_error}")

def quick_test():
    """動作確認用クイックテスト"""
    print("🧪 クイックテスト開始")
    
    try:
        system = MemoryEfficientNMCSystem()
        
        # 基本構造テスト
        nmc = system.create_nmc_base_structure()
        al = system.create_al_structure()
        
        print(f"✅ NMC: {len(nmc)} atoms")
        print(f"✅ Al: {len(al)} atoms")
        
        # テスト組成変更
        test_comp = {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}
        variant = system.modify_composition(nmc, test_comp, "test")
        
        if variant:
            print("✅ 組成変更テスト成功")
        
        return True
        
    except Exception as e:
        print(f"❌ テスト失敗: {e}")
        return False

if __name__ == "__main__":
    # 通常実行
    main()
    
    # テスト実行の場合は以下をコメントアウト
    # quick_test()

✅ PFP/Matlantis計算器を正常に初期化
🧪 メモリ効率NMC-Al界面構造システム
   リチウムイオン電池リサイクル研究用
✅ メモリ効率システム初期化完了
  出力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure
  CIF保存先: /home/jovyan/Kaori/MD/LiB_2/structure/opt
🚀 メモリ効率NMC-Al界面システム 開始

📐 基本構造作成
CIFファイルから読み込み: /home/jovyan/Kaori/MD/input/Li3MnCoNiO6.cif
✅ CIF読み込み成功: 24 atoms
Al基本構造を作成中...
✅ Al構造作成完了: 4 atoms

🔬 NMC組成変異体処理

--- pristineの処理開始 ---
組成変更中: pristine
  ✅ 保存完了: NMC_pristine_bulk.cif
  表面作成中: NMC_pristine_001
=== NMC_pristine_001の構造最適化開始 (BFGS → Matlantis) ===
  -> Step 1: BFGSによる予備最適化 (fmax=0.05)
      Step     Time          Energy          fmax
BFGS:    0 08:05:04    12176.419807    14286.020037
BFGS:    1 08:05:04     3035.586852     3013.007735
BFGS:    2 08:05:04     3818.473071    13630.501635
BFGS:    3 08:05:04     4594.534475    12286.209095
BFGS:    4 08:05:05     1762.101813     3720.801293
BFGS:    5 08:05:05     1543.652615     4211.743684
BFGS:    6 08:05:05     1639.599251     6875.037237
BFGS:    7 08:05:05      893.988643     3999.0346

  0%|          | 0/3001 [00:00<?, ?it/s]

    -> ✔️ Matlantis最適化完了。最終エネルギー: -896.045 eV
  ✅ 表面保存完了: NMC_pristine_001_surface.cif
  表面作成中: NMC_pristine_104
=== NMC_pristine_104の構造最適化開始 (BFGS → Matlantis) ===
  -> Step 1: BFGSによる予備最適化 (fmax=0.05)
      Step     Time          Energy          fmax
BFGS:    0 08:06:00     -216.076167      306.612500
BFGS:    1 08:06:01    -1359.874794       56.877841
BFGS:    2 08:06:02    -1537.089896      208.545869
BFGS:    3 08:06:03    -1673.716726      132.288332
BFGS:    4 08:06:04    -1822.965526       62.661531
BFGS:    5 08:06:05    -1916.777900       19.209205
BFGS:    6 08:06:06    -1960.908862       11.101493
BFGS:    7 08:06:07    -1991.567591        3.468733
BFGS:    8 08:06:08    -2006.816367        4.173965
BFGS:    9 08:06:09    -2019.176174        3.934111
BFGS:   10 08:06:10    -2024.441475        3.587878
BFGS:   11 08:06:11    -2031.040448        2.577332
BFGS:   12 08:06:12    -2034.002731        2.264337
BFGS:   13 08:06:13    -2038.181066        1.933541
BFGS:   14 08:06:14

  0%|          | 0/3001 [00:00<?, ?it/s]

    -> ✔️ Matlantis最適化完了。最終エネルギー: -2140.100 eV
  ✅ 表面保存完了: NMC_pristine_104_surface.cif
  表面作成中: NMC_pristine_012
=== NMC_pristine_012の構造最適化開始 (BFGS → Matlantis) ===
  -> Step 1: BFGSによる予備最適化 (fmax=0.05)
      Step     Time          Energy          fmax
BFGS:    0 08:10:43    -1219.052697       40.922074
BFGS:    1 08:10:43    -1258.424006       10.715510
BFGS:    2 08:10:43    -1285.656765        5.632798
BFGS:    3 08:10:43    -1303.384842        5.196912
BFGS:    4 08:10:44    -1310.782375        4.218248
BFGS:    5 08:10:44    -1314.714931        3.258187
BFGS:    6 08:10:44    -1318.430577        2.472035
BFGS:    7 08:10:44    -1320.329948        1.487322
BFGS:    8 08:10:44    -1321.332753        2.118992
BFGS:    9 08:10:45    -1322.517698        1.594619
BFGS:   10 08:10:45    -1323.139228        0.880368
BFGS:   11 08:10:45    -1323.723185        0.853981
BFGS:   12 08:10:45    -1324.238217        1.205280
BFGS:   13 08:10:46    -1324.647708        0.870004
BFGS:   14 08:10:4

...

    -> ✔️ Matlantis最適化完了。最終エネルギー: -2096.954 eV
  ✅ 表面保存完了: NMC_NMC811_approx_104_surface.cif
  表面作成中: NMC_NMC811_approx_012
=== NMC_NMC811_approx_012の構造最適化開始 (BFGS → Matlantis) ===
  -> Step 1: BFGSによる予備最適化 (fmax=0.05)
      Step     Time          Energy          fmax
BFGS:    0 09:06:02    -1206.740265       40.478998
BFGS:    1 09:06:02    -1244.289552        8.880888
BFGS:    2 09:06:02    -1267.446194        4.396027
BFGS:    3 09:06:03    -1282.074937        2.955939
BFGS:    4 09:06:03    -1287.997635        2.720671
BFGS:    5 09:06:03    -1291.579189        2.565191
BFGS:    6 09:06:03    -1295.348883        2.144408
BFGS:    7 09:06:03    -1297.143934        1.836281
BFGS:    8 09:06:04    -1298.830678        1.279012
BFGS:    9 09:06:04    -1299.579655        1.281209
BFGS:   10 09:06:04    -1300.552626        1.066716
BFGS:   11 09:06:04    -1301.157934        1.076612
BFGS:   12 09:06:04    -1301.733310        1.149977
BFGS:   13 09:06:05    -1302.213311        0.788440
BFG

  0%|          | 0/3001 [00:00<?, ?it/s]

    -> ✔️ Matlantis最適化完了。最終エネルギー: -1311.512 eV
  ✅ 表面保存完了: NMC_NMC811_approx_012_surface.cif
  表面作成中: NMC_NMC811_approx_010
=== NMC_NMC811_approx_010の構造最適化開始 (BFGS → Matlantis) ===
  -> Step 1: BFGSによる予備最適化 (fmax=0.05)
      Step     Time          Energy          fmax
BFGS:    0 09:06:57     -445.115224        2.408930
BFGS:    1 09:06:58     -445.691289        1.722323
BFGS:    2 09:06:58     -447.100201        4.265989
BFGS:    3 09:06:58     -447.491770        1.432608
BFGS:    4 09:06:58     -447.619775        1.042024
BFGS:    5 09:06:58     -447.841634        0.577386
BFGS:    6 09:06:58     -447.875749        0.577346
BFGS:    7 09:06:58     -447.964806        0.321060
BFGS:    8 09:06:58     -447.983034        0.309314
BFGS:    9 09:06:59     -448.008298        0.398546
BFGS:   10 09:06:59     -448.034184        0.505023
BFGS:   11 09:06:59     -448.062410        0.329585
BFGS:   12 09:06:59     -448.079079        0.191606
BFGS:   13 09:06:59     -448.094167        0.180008
BFG

  0%|          | 0/3001 [00:00<?, ?it/s]

    -> ✔️ Matlantis最適化完了。最終エネルギー: -448.849 eV
  ✅ 表面保存完了: NMC_NMC811_approx_010_surface.cif

⚗️ Al表面処理

🔗 界面構造作成
  進行状況: 1/80
  界面作成中: Interface_Al2O3_on_NMC_pristine_001

❌ ワークフロー実行エラー: name 'unit_thickness_xy' is not defined

⚠️ 一部処理でエラーが発生しましたが、
   完了した結果は保存されています


In [2]:
#!/usr/bin/env python3
"""
メモリ効率改善版 統合NMC-Al界面構造生成・最適化システム
リチウムイオン電池リサイクル研究用

主な改善点:
- メモリ使用量を最小化
- 一つずつ処理して即座に保存
- 大量のデータを同時にメモリに保持しない
- エラー耐性の向上
- 必要最小限の構造のみ生成
"""

import numpy as np
import os, sys, time
from pathlib import Path
import random
import math
import gc  # ガベージコレクション用
# ASE
import ase
from ase import Atoms, units
from ase.io import read, write
from ase.build import surface, fcc100,bulk
from ase.constraints import FixAtoms
from ase.optimize import BFGS

# PFP (Matlantis) - エラー耐性版
try:
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    from pfp_api_client.pfp.estimator import Estimator
    from matlantis_features.atoms import MatlantisAtoms
    from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
    from matlantis_features.utils.calculators import pfp_estimator_fn
    from pfp_api_client.pfp.estimator import EstimatorCalcMode
    
    estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
    calculator = ASECalculator(estimator)
    MATLANTIS_AVAILABLE = True
    print("✅ PFP/Matlantis計算器を正常に初期化")
    
except ImportError as e:
    print(f"⚠️ PFP/Matlantisが利用できません: {e}")
    print("  基本ASE最適化のみを使用")
    calculator = None
    MATLANTIS_AVAILABLE = False

class MemoryEfficientNMCSystem:
    """メモリ効率重視のNMC-Al界面構造システム"""
    
    def __init__(self, base_dir="/home/jovyan/Kaori/MD/LiB_2"):
        self.base_dir = Path(base_dir)
        self.output_dir = self.base_dir / "structure" 
        self.cif_dir = self.output_dir / "opt"
        
        # ディレクトリ作成
        for dir_path in [self.output_dir, self.cif_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)
        
        # 計算器設定
        self.calculator = calculator
        
        # Miller指数リスト（重要度順）
        self.miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 0, 4),  # NMC(104) - 中活性、多面体形状
            (0, 1, 2),  # NMC(012) - 高活性、切頭八面体形状
            (0, 1, 0),  # NMC(010) - Li拡散活性面
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        self.al_miller_indices = [
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        self.al2o3_miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
        ]
        self.alf3_miller_indices = [
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
        ]
        
        # シミュレーション計画の定義
        self.simulation_plan = [
            # ニッケル(Ni)増減の影響
            {"name": "Ni_increase", "composition": {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}},
            {"name": "Ni_decrease", "composition": {"Ni": 0.2, "Mn": 0.4, "Co": 0.4}},
            
            # マンガン(Mn)増減の影響
            {"name": "Mn_increase", "composition": {"Ni": 0.2, "Mn": 0.6, "Co": 0.2}},
            {"name": "Mn_decrease", "composition": {"Ni": 0.4, "Mn": 0.2, "Co": 0.4}},
            
            # コバルト(Co)増減の影響
            {"name": "Co_increase", "composition": {"Ni": 0.2, "Mn": 0.2, "Co": 0.6}},
            {"name": "Co_decrease", "composition": {"Ni": 0.4, "Mn": 0.4, "Co": 0.2}},
            
            # ハイニッケル化トレンド
            {"name": "NMC523", "composition": {"Ni": 0.50, "Mn": 0.25, "Co": 0.25}},
            {"name": "NMC622_approx", "composition": {"Ni": 0.60, "Mn": 0.20, "Co": 0.20}},
            {"name": "NMC811_approx", "composition": {"Ni": 0.80, "Mn": 0.10, "Co": 0.10}},
        ]
        
        print(f"✅ メモリ効率システム初期化完了")
        print(f"  出力ディレクトリ: {self.output_dir}")
        print(f"  CIF保存先: {self.cif_dir}")
    
    def load_or_create_nmc_base(self):
        """NMC基本構造の読み込みまたは作成（元の機能を復元）"""
        input_dir = Path("/home/jovyan/Kaori/MD/input")
        cif_path = input_dir / "Li3MnCoNiO6.cif"
        
        if cif_path.exists():
            print(f"CIFファイルから読み込み: {cif_path}")
            try:
                base_atoms = read(str(cif_path))
                print(f"✅ CIF読み込み成功: {len(base_atoms)} atoms")
                return base_atoms
            except Exception as e:
                print(f"❌ CIF読み込み失敗: {e}")
        
        print("CIFファイルがないため、基本構造を生成します")
        return self.create_nmc_base_structure()
    
    # def create_nmc_base_structure(self):
    #     """NMC基本構造の作成（簡素版）"""
    #     print("NMC基本構造を作成中...")
        
    #     # 簡素化したNMC構造
    #     a = 2.87  # 格子定数
    #     c = 14.2
        
    #     positions = [
    #         [0.0, 0.0, 0.0],      # Li
    #         [0.5, 0.5, 0.0],      # Li
    #         [0.0, 0.5, 0.25],     # Ni
    #         [0.5, 0.0, 0.25],     # Mn
    #         [0.0, 0.0, 0.125],    # O
    #         [0.5, 0.5, 0.125],    # O
    #         [0.0, 0.5, 0.375],    # O
    #         [0.5, 0.0, 0.375],    # O
    #         [0.0, 0.0, 0.5],      # Co
    #         [0.5, 0.5, 0.5],      # Ni
    #     ]
        
    #     symbols = ['Li', 'Li', 'Ni', 'Mn', 'O', 'O', 'O', 'O', 'Co', 'Ni']
    #     cell = [[a, 0, 0], [0, a, 0], [0, 0, c]]
        
    #     atoms = Atoms(symbols=symbols, 
    #                  positions=np.array(positions) * [a, a, c],
    #                  cell=cell, 
    #                  pbc=True)
        
    #     print(f"✅ NMC基本構造作成完了: {len(atoms)} atoms")
    #     return atoms
    
    def create_al_structure(self):
        """Al基本構造の作成"""
        print("Al基本構造を作成中...")
        al_atoms = bulk("Al", "fcc", a=4.05, cubic=True)
        # al2o3_atoms = read('/home/jovyan/Kaori/MD/input/Al2O3.cif') 
        # alf3_atoms = read('/home/jovyan/Kaori/MD/input/AlF3_cubic.cif') 
        print(f"✅ Al構造作成完了: {len(al_atoms)} atoms")
        return al_atoms
    
    def modify_composition(self, atoms, target_composition, name="variant"):
        """組成変更（即座に保存）"""
        print(f"組成変更中: {name}")
        # if cif_path.exists():
        #     print(f"  既存ファイルをスキップ: {cif_path.name}")
        #     return str(cif_path)
        try:
            variant_atoms = atoms.copy()
            tm_indices = [i for i, atom in enumerate(variant_atoms) 
                         if atom.symbol in ["Ni", "Mn", "Co"]]
            
            if len(tm_indices) == 0:
                print(f"  警告: 遷移金属が見つかりません")
                return None
            
            total_tm_sites = len(tm_indices)
            target_counts = {symbol: round(fraction * total_tm_sites) 
                           for symbol, fraction in target_composition.items()}
            
            # 四捨五入調整
            current_total = sum(target_counts.values())
            diff = total_tm_sites - current_total
            if diff != 0:
                most_abundant = max(target_composition, key=target_composition.get)
                target_counts[most_abundant] += diff
            
            # 新しい元素リストを作成してシャッフル
            new_symbols = []
            for symbol, count in target_counts.items():
                new_symbols.extend([symbol] * count)
            
            random.seed(42)
            random.shuffle(tm_indices)
            
            for i, atom_index in enumerate(tm_indices):
                variant_atoms[atom_index].symbol = new_symbols[i]
            
            # 即座にCIF保存
            cif_path = self.cif_dir / f"NMC_{name}_bulk.cif"
            write(str(cif_path), variant_atoms)
            print(f"  ✅ 保存完了: {cif_path.name}")
            
            return variant_atoms
            
        except Exception as e:
            print(f"  ❌ 組成変更エラー: {e}")
            return None
        finally:
            # メモリクリーンアップ
            gc.collect()
    
    def optimize_structure_systematic(self, atoms, name="structure"):
        """
        構造最適化（BFGS + Matlantis 2段階版）
        1. ASEのBFGSで格子と原子位置を予備的に最適化
        2. Matlantis Featuresで最終的な精密最適化
        """
        print(f"=== {name}の構造最適化開始 (BFGS → Matlantis) ===")

        if self.calculator is None or not MATLANTIS_AVAILABLE:
            print("  -> ⚠️ 計算器が利用できないため、最適化をスキップします")
            return atoms

        try:
            # --- Step 1: ASEのBFGSによる予備最適化 ---
            print("  -> Step 1: BFGSによる予備最適化 (fmax=0.05)")
            write(f"{name}.xyz",atoms)
            # 計算機をatomsオブジェクトにセット
            temp_estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
            atoms.calc = ASECalculator(temp_estimator)

            # UnitCellFilterを使って格子と原子位置を同時に最適化
            bfgs_optimizer = BFGS(atoms)
            bfgs_optimizer.run(fmax=0.05, steps=100) # やや緩めの条件で実行
            
            print(f"    -> BFGS最適化完了。中間エネルギー: {atoms.get_potential_energy():.3f} eV")

            # --- Step 2: Matlantis Featuresによる精密最適化 ---
            print("  -> Step 2: Matlantis Featuresによる精密最適化 (fmax=0.01)")
            
            # BFGSで最適化された構造をMatlantisオプティマイザに入力
            estimator_function = pfp_estimator_fn(
                model_version='v7.0.0',
                calc_mode=EstimatorCalcMode.CRYSTAL_U0_PLUS_D3
            )

            matlantis_optimizer = FireLBFGSASEOptFeature(
                estimator_fn=estimator_function,
                n_run=3000,
                fmax=0.01,  # より厳密な条件
                show_progress_bar=True
            )
                            # filter=True,  # 格子も最終調整する
                # trajectory=str(self.output_dir / f"{name}_matlantis_final.traj"),
            result = matlantis_optimizer(MatlantisAtoms(atoms))
            optimized_atoms = result.atoms.ase_atoms
            final_energy = result.output.energy_log[-1]
            
            print(f"    -> ✔️ Matlantis最適化完了。最終エネルギー: {final_energy:.3f} eV")

            return optimized_atoms

        except Exception as e:
            print(f"  -> ❌ 最適化中にエラーが発生しました: {e}")
            # エラーが発生した場合は、元の構造をそのまま返す
            return atoms
        # finally:
        #     # メモリクリーンアップ
        #     gc.collect()
    
    def create_surface_and_save(self, atoms, miller, base_name):
        """表面作成と即座保存"""
        miller_str = f"{miller[0]}{miller[1]}{miller[2]}"
        surface_name = f"{base_name}_{miller_str}"
        cif_path = self.cif_dir / f"{surface_name}_surface.cif"
        
        # 既存ファイルのチェック
        if cif_path.exists():
            print(f"  既存ファイルをスキップ: {cif_path.name}")
            return str(cif_path)
        
        # try:
        print(f"  表面作成中: {surface_name}")
        
        # 元の設定値に戻す
        slab_unit = surface(atoms, miller, layers=4, vacuum=0.0)
        initial_dims=slab_unit.cell.lengths()
        target_dims = np.array([12, 12, 25])

        
        repean_num=np.ceil(target_dims / initial_dims).astype(int)
        slab = slab_unit.repeat(repeat_num)  # 元のリピート数に戻す
        
        # 位置調整
        positions = slab.get_positions()
        z_min = positions[:, 2].min()
        slab.set_positions(positions - [0, 0, z_min])
        
        # 下部原子の固定（元の固定比率）
        z_coords = slab.positions[:, 2]
        z_threshold = z_coords.min() + (z_coords.max() - z_coords.min()) * 0.3
        fixed_indices = [i for i, z in enumerate(z_coords) if z <= z_threshold]
        
        if fixed_indices:
            constraint = FixAtoms(indices=fixed_indices)
            slab.set_constraint(constraint)
        
        # 軽い最適化
        slab = self.optimize_structure_systematic(slab, surface_name)
        
        # 即座に保存
        write(str(cif_path), slab)
        print(f"  ✅ 表面保存完了: {cif_path.name}")
        
        return str(cif_path)
            
        # except Exception as e:
        #     print(f"  ❌ 表面作成失敗: {surface_name} - {e}")
        #     return None
        # finally:
        #     # メモリクリーンアップ
        #     # del slab if 'slab' in locals() else None
        #     gc.collect()
    
    def create_interface_and_save(self, surface1_path, surface2_path):
        """界面作成と即座保存（元の複雑なトリミング処理を復元）"""
        try:
            # ファイル名から界面名を生成
            name1 = Path(surface1_path).stem.replace('_surface', '')
            name2 = Path(surface2_path).stem.replace('_surface', '')
            interface_name = f"Interface_{name2}_on_{name1}"
            cif_path = self.cif_dir / f"{interface_name}.cif"
            
            # 既存ファイルのチェック
            if cif_path.exists():
                print(f"  既存界面をスキップ: {cif_path.name}")
                return str(cif_path)
            
            print(f"  界面作成中: {interface_name}")
            
            # 表面構造の読み込み
            slab1 = read(surface1_path)
            slab2 = read(surface2_path)
            separation = 2.0
            # XY方向のトリミング
            unit_thickness1 = slab1.cell.lengths()[2]
            # 目標とする厚みを設定
            target_thickness = 19.5
            # 必要なZ方向の繰り返し回数を計算 (小数点以下を切り上げ)
            if unit_thickness1 > 0:
                repeat_z1 = math.ceil(target_thickness / unit_thickness1)
            else:
                repeat_z = 1 # 厚みが0の場合は1回とする
            # 計算した回数でslab1をZ方向に繰り返す
            final_slab1 = slab1.repeat((1, 1, repeat_z1))
            
            unit_thickness2 = slab2.cell.lengths()[2]
            # 必要なZ方向の繰り返し回数を計算 (小数点以下を切り上げ)
            if unit_thickness2 > 0:
                repeat_z2 = math.ceil(target_thickness / unit_thickness2)
            else:
                repeat_z = 1 # 厚みが0の場合は1回とする
            # 計算した回数でslab1をZ方向に繰り返す
            final_slab2 = slab2.repeat((1, 1, repeat_z2))
            
            pos1 = final_slab1.get_positions()
            pos2 = final_slab2.get_positions()
            x_max=pos2[:,0].max()
            y_max=pos2[:,1].max()
            target_xy=(x_max,y_max)
            indices_to_delete1 = np.where((pos1[:, 1] < 0)|(pos1[:, 0] < 0)| (pos1[:, 0] >= target_xy[0]) | 
                                         (pos1[:, 1] >= target_xy[1]) | (pos1[:, 2] > 25))[0]
            indices_to_delete2 = np.where((pos2[:, 1] < 0)|(pos2[:, 0] < 0)|(pos2[:, 0] >= target_xy[0]) | 
                                         (pos2[:, 1] >= target_xy[1]) | (pos2[:, 2] > 25))[0]
            
            cut_slab1, cut_slab2 = final_slab1.copy(), final_slab2.copy()
            if len(indices_to_delete1) > 0: 
                del cut_slab1[indices_to_delete1]
            if len(indices_to_delete2) > 0: 
                del cut_slab2[indices_to_delete2]
            
            # Z方向の位置調整
            z1_max = cut_slab1.positions[:, 2].max()
            z2_min = cut_slab2.positions[:, 2].min()
            cut_slab2.positions[:, 2] += z1_max - z2_min + separation
            
            # 界面の結合
            interface = cut_slab1 + cut_slab2
            interface.cell = [x_max,y_max , 53]
            interface.pbc = [True, True, False]
            interface.center(vacuum=1.0, axis=2)
            
            # STEP 2: 界面のトリミング（元の複雑な処理）
            # interface = self.trim_interface(interface, thickness_per_slab=25.0)
            # if interface is None:
            #     print("   -> ❌ トリミングに失敗")
            #     return None
            
            # STEP 3: 構造最適化
            indices_to_fix = [atom.index for atom in interface if atom.position[2] <= 3.0]

            if indices_to_fix:
                print(f"    下部 {len(indices_to_fix)} 個の原子を固定します。")
                constraint = FixAtoms(indices=indices_to_fix)
                interface.set_constraint(constraint)
            else:
                print("    警告: 固定する原子が見つかりませんでした。")

            interface = self.optimize_structure_systematic(interface, interface_name)
            interface.set_constraint() 
            # 即座に保存
            write(str(cif_path), interface)
            print(f"  ✅ 界面保存完了: {cif_path.name}")
            
            return str(cif_path)
        except Exception as e:
            print(f"  ❌ 界面作成失敗: {e}")
            return None
        finally:
            # メモリクリーンアップ
            for var in ['slab1', 'slab2', 'cut_slab1', 'cut_slab2', 'interface']:
                if var in locals():
                    del locals()[var]
            gc.collect()
    
    def trim_interface(self, atoms, thickness_per_slab=25.0):
        """界面のトリミング（元の複雑な処理を復元）"""
        print(f"    2. 界面をトリミング中 (各スラブ厚み: {thickness_per_slab} Å)...")
        
        # 界面のZ座標を推定
        al_indices = [a.index for a in atoms if a.symbol == 'Al']
        nmc_indices = [a.index for a in atoms if a.symbol not in ['Al']]
        
        if not al_indices or not nmc_indices: 
            return None
            
        al_pos = atoms.get_positions()[al_indices]
        nmc_pos = atoms.get_positions()[nmc_indices]
        
        if np.mean(al_pos[:, 2]) < np.mean(nmc_pos[:, 2]):
            interface_z = (al_pos[:, 2].max() + nmc_pos[:, 2].min()) / 2
            is_al_bottom = True
        else:
            interface_z = (nmc_pos[:, 2].max() + al_pos[:, 2].min()) / 2
            is_al_bottom = False

        # 削除する原子を特定
        indices_to_delete = []
        for atom in atoms:
            if is_al_bottom:
                if atom.symbol == 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol != 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)
            else:
                if atom.symbol != 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol == 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)

        print(f"       -> {len(indices_to_delete)} 個の原子を削除")
        if len(indices_to_delete) > 0:
            del atoms[indices_to_delete]
        
        return atoms
    
    def process_single_nmc_variant(self, base_atoms, composition_info):
        """単一のNMC変異体の処理（メモリ効率版）"""
        name = composition_info["name"]
        composition = composition_info["composition"]
        
        print(f"\n--- {name}の処理開始 ---")
        
        # 1. 組成変更と保存
        variant_atoms = self.modify_composition(base_atoms, composition, name)
        if variant_atoms is None:
            return []
        
        # 2. 表面作成と保存（全Miller指数対応）
        surface_files = []
        for miller in self.miller_indices[:4]:  # NMC面のみ
            surface_file = self.create_surface_and_save(variant_atoms, miller, f"NMC_{name}")
            if surface_file:
                surface_files.append(surface_file)
        
        # メモリクリーンアップ
        del variant_atoms
        gc.collect()
        
        return surface_files
    
    def run_memory_efficient_workflow(self):
        """メモリ効率重視のワークフロー実行"""
        print("🚀 メモリ効率NMC-Al界面システム 開始")
        print("=" * 60)
        
        start_time = time.time()
        all_surface_files = []
        
        try:
            # 基本構造作成
            print("\n📐 基本構造作成")
            base_nmc = self.load_or_create_nmc_base()  # 元の関数名に修正
            base_al = self.create_al_structure()
            
            # NMC組成変異体の処理（一つずつ）
            print("\n🔬 NMC組成変異体処理")
            nmc_surfaces = []
            
            # pristine版も追加
            pristine_surfaces = self.process_single_nmc_variant(
            base_nmc, {"name": "pristine", "composition": {"Ni": 1/3, "Mn": 1/3, "Co": 1/3}})
            nmc_surfaces.extend(pristine_surfaces)
            
            # 全組成変異体の処理
            for comp_info in self.simulation_plan:
                surfaces = self.process_single_nmc_variant(base_nmc, comp_info)
                nmc_surfaces.extend(surfaces)
                # 各処理後にメモリクリーンアップ
                gc.collect()
            
            # Al表面の処理（全Miller指数）
            print("\n⚗️ Al表面処理")
            al_surfaces = []
            for miller in self.al_miller_indices[3:]:  # Al面のみ
                surface_file = self.create_surface_and_save(base_al, miller, "Al")
                if surface_file:
                    al_surfaces.append(surface_file)
            al2o3_atoms=read("/home/jovyan/Kaori/MD/LiB_2/structure/output/Al2O3_pos_opt_final.xyz")
            al2o3_path = self.cif_dir / "Al2O3_surface.cif" 
            write(al2o3_path, al2o3_atoms)
            # Atomsオブジェクトではなく、ファイルパスをリストに追加
            al_surfaces.append(str(al2o3_path))
            alf3_atoms=read("/home/jovyan/Kaori/MD/LiB_2/structure/output/AlF3_pos_opt_final.xyz")
            alf3_path = self.cif_dir / "AlF3_surface.cif" 
            write(alf3_path, alf3_atoms)
            al_surfaces.append(str(alf3_path))
            # Atomsオブジェクトではなく、ファイルパスをリストに追加
            # al_surfaces.append(al2o3)
            # al_surfaces.append(alf3)
            # メモリクリーンアップ
            del base_nmc, base_al
            gc.collect()
            
            # 界面作成（一つずつ処理）
            print("\n🔗 界面構造作成")
            interface_count = 0
            total_combinations = len(nmc_surfaces) * len(al_surfaces)
            current = 0
            
            for nmc_surface in nmc_surfaces:
                for al_surface in al_surfaces:
                    current += 1
                    print(f"  進行状況: {current}/{total_combinations}")
                    interface_file = self.create_interface_and_save(nmc_surface, al_surface)
                    if interface_file:
                        interface_count += 1
                    # 各界面作成後にメモリクリーンアップ
                    gc.collect()
            
            # 最終結果
            execution_time = time.time() - start_time
            
            print(f"\n✅ 処理完了!")
            print(f"  実行時間: {execution_time:.1f}秒")
            print(f"  NMC表面: {len(nmc_surfaces)}個")
            print(f"  Al表面: {len(al_surfaces)}個") 
            print(f"  界面構造: {interface_count}個")
            
            # 保存されたファイルの確認
            cif_files = list(self.cif_dir.glob("*.cif"))
            print(f"  保存されたCIFファイル総数: {len(cif_files)}個")
            
            # サマリー保存
            self.save_summary(len(nmc_surfaces), len(al_surfaces), interface_count, execution_time)
            
            return True
            
        except Exception as e:
            print(f"\n❌ ワークフロー実行エラー: {e}")
            return False
        finally:
            # 最終メモリクリーンアップ
            gc.collect()
    
    def save_summary(self, nmc_count, al_count, interface_count, exec_time):
        """実行サマリーの保存"""
        summary = f"""メモリ効率NMC-Al界面構造システム 実行結果
========================================
実行日時: {time.strftime('%Y-%m-%d %H:%M:%S')}
実行時間: {exec_time:.1f}秒

生成された構造:
- NMC表面構造: {nmc_count}個
- Al表面構造: {al_count}個  
- 界面構造: {interface_count}個

保存先: {self.cif_dir}

処理した組成:
- NMC pristine (Ni:Mn:Co = 1:1:1)
- Ni増減バリアント (高活性/低活性)
- Mn増減バリアント (構造安定性) 
- Co増減バリアント (電気化学特性)
- ハイニッケル化トレンド (NMC523/622/811)

処理したMiller指数:
- (001) NMC最安定面、プレート形状
- (104) NMC中活性、多面体形状
- (012) NMC高活性、切頭八面体形状
- (010) NMC Li拡散活性面
- (111) Al最安定面
- (100) 基本面
- (110) 副次面
"""
        
        summary_path = self.output_dir / "execution_summary.txt"
        with open(summary_path, "w", encoding="utf-8") as f:
            f.write(summary)
        
        print(f"📄 サマリー保存: {summary_path}")

def main():
    """メイン実行関数"""
    print("=" * 70)
    print("🧪 メモリ効率NMC-Al界面構造システム")
    print("   リチウムイオン電池リサイクル研究用")
    print("=" * 70)
    
    try:
        # システム初期化
        system = MemoryEfficientNMCSystem()
        
        # ワークフロー実行
        success = system.run_memory_efficient_workflow()
        
        if success:
            print("\n🎉 全処理が正常完了!")
            print(f"📂 結果: {system.cif_dir}")
        else:
            print("\n⚠️ 一部処理でエラーが発生しましたが、")
            print("   完了した結果は保存されています")
            
    except Exception as e:
        print(f"\n💥 システムエラー: {e}")
        
        # 最小限のフォールバック
        try:
            print("\n🔄 フォールバック実行...")
            fallback_system = MemoryEfficientNMCSystem()
            base_nmc = fallback_system.load_or_create_nmc_base()  # 元の関数名に修正
            
            fallback_path = fallback_system.cif_dir / "NMC_fallback.cif"
            write(str(fallback_path), base_nmc)
            print(f"✅ フォールバック完了: {fallback_path}")
            
        except Exception as fb_error:
            print(f"❌ フォールバックも失敗: {fb_error}")

def quick_test():
    """動作確認用クイックテスト"""
    print("🧪 クイックテスト開始")
    
    try:
        system = MemoryEfficientNMCSystem()
        
        # 基本構造テスト
        nmc = system.create_nmc_base_structure()
        al = system.create_al_structure()
        
        print(f"✅ NMC: {len(nmc)} atoms")
        print(f"✅ Al: {len(al)} atoms")
        
        # テスト組成変更
        test_comp = {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}
        variant = system.modify_composition(nmc, test_comp, "test")
        
        if variant:
            print("✅ 組成変更テスト成功")
        
        return True
        
    except Exception as e:
        print(f"❌ テスト失敗: {e}")
        return False

if __name__ == "__main__":
    # 通常実行
    main()
    
    # テスト実行の場合は以下をコメントアウト
    # quick_test()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 426)

In [14]:
#!/usr/bin/env python3
"""
メモリ効率改善版 統合NMC-Al界面構造生成・最適化システム
リチウムイオン電池リサイクル研究用

主な改善点:
- メモリ使用量を最小化
- 一つずつ処理して即座に保存
- 大量のデータを同時にメモリに保持しない
- エラー耐性の向上
- 必要最小限の構造のみ生成
"""

import numpy as np
import os, sys, time
from pathlib import Path
import random
import math
import gc  # ガベージコレクション用
# ASE
import ase
from ase import Atoms, units
from ase.io import read, write
from ase.build import surface, fcc100,bulk
from ase.constraints import FixAtoms
from ase.optimize import BFGS

# PFP (Matlantis) - エラー耐性版
try:
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    from pfp_api_client.pfp.estimator import Estimator
    from matlantis_features.atoms import MatlantisAtoms
    from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
    from matlantis_features.utils.calculators import pfp_estimator_fn
    from pfp_api_client.pfp.estimator import EstimatorCalcMode
    
    estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
    calculator = ASECalculator(estimator)
    MATLANTIS_AVAILABLE = True
    print("✅ PFP/Matlantis計算器を正常に初期化")
    
except ImportError as e:
    print(f"⚠️ PFP/Matlantisが利用できません: {e}")
    print("  基本ASE最適化のみを使用")
    calculator = None
    MATLANTIS_AVAILABLE = False

class MemoryEfficientNMCSystem:
    """メモリ効率重視のNMC-Al界面構造システム"""
    
    def __init__(self, base_dir="/home/jovyan/Kaori/MD/LiB_2"):
        self.base_dir = Path(base_dir)
        self.output_dir = self.base_dir / "structure" 
        self.cif_dir = self.output_dir / "opt"
        
        # ディレクトリ作成
        for dir_path in [self.output_dir, self.cif_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)
        
        # 計算器設定
        self.calculator = calculator
        
        # Miller指数リスト（重要度順）
        self.miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 0, 4),  # NMC(104) - 中活性、多面体形状
            (0, 1, 2),  # NMC(012) - 高活性、切頭八面体形状
            (0, 1, 0),  # NMC(010) - Li拡散活性面
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        self.al_miller_indices = [
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        self.al2o3_miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
        ]
        self.alf3_miller_indices = [
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
        ]
        
        # シミュレーション計画の定義
        self.simulation_plan = [
            # ニッケル(Ni)増減の影響
            {"name": "Ni_increase", "composition": {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}},
            {"name": "Ni_decrease", "composition": {"Ni": 0.2, "Mn": 0.4, "Co": 0.4}},
            
            # マンガン(Mn)増減の影響
            {"name": "Mn_increase", "composition": {"Ni": 0.2, "Mn": 0.6, "Co": 0.2}},
            {"name": "Mn_decrease", "composition": {"Ni": 0.4, "Mn": 0.2, "Co": 0.4}},
            
            # コバルト(Co)増減の影響
            {"name": "Co_increase", "composition": {"Ni": 0.2, "Mn": 0.2, "Co": 0.6}},
            {"name": "Co_decrease", "composition": {"Ni": 0.4, "Mn": 0.4, "Co": 0.2}},
            
            # ハイニッケル化トレンド
            {"name": "NMC523_approx", "composition": {"Ni": 0.50, "Mn": 0.25, "Co": 0.25}},
            {"name": "NMC622_approx", "composition": {"Ni": 0.60, "Mn": 0.20, "Co": 0.20}},
            {"name": "NMC811_approx", "composition": {"Ni": 0.80, "Mn": 0.10, "Co": 0.10}},
        ]
        
        print(f"✅ メモリ効率システム初期化完了")
        print(f"  出力ディレクトリ: {self.output_dir}")
        print(f"  CIF保存先: {self.cif_dir}")
    
    def load_or_create_nmc_base(self):
        """NMC基本構造の読み込みまたは作成（元の機能を復元）"""
        input_dir = Path("/home/jovyan/Kaori/MD/input")
        cif_path = input_dir / "Li3MnCoNiO6.cif"
        
        if cif_path.exists():
            print(f"CIFファイルから読み込み: {cif_path}")
            try:
                base_atoms = read(str(cif_path))
                print(f"✅ CIF読み込み成功: {len(base_atoms)} atoms")
                return base_atoms
            except Exception as e:
                print(f"❌ CIF読み込み失敗: {e}")
        
        print("CIFファイルがないため、基本構造を生成します")
        return self.create_nmc_base_structure()
    
    # def create_nmc_base_structure(self):
    #     """NMC基本構造の作成（簡素版）"""
    #     print("NMC基本構造を作成中...")
        
    #     # 簡素化したNMC構造
    #     a = 2.87  # 格子定数
    #     c = 14.2
        
    #     positions = [
    #         [0.0, 0.0, 0.0],      # Li
    #         [0.5, 0.5, 0.0],      # Li
    #         [0.0, 0.5, 0.25],     # Ni
    #         [0.5, 0.0, 0.25],     # Mn
    #         [0.0, 0.0, 0.125],    # O
    #         [0.5, 0.5, 0.125],    # O
    #         [0.0, 0.5, 0.375],    # O
    #         [0.5, 0.0, 0.375],    # O
    #         [0.0, 0.0, 0.5],      # Co
    #         [0.5, 0.5, 0.5],      # Ni
    #     ]
        
    #     symbols = ['Li', 'Li', 'Ni', 'Mn', 'O', 'O', 'O', 'O', 'Co', 'Ni']
    #     cell = [[a, 0, 0], [0, a, 0], [0, 0, c]]
        
    #     atoms = Atoms(symbols=symbols, 
    #                  positions=np.array(positions) * [a, a, c],
    #                  cell=cell, 
    #                  pbc=True)
        
    #     print(f"✅ NMC基本構造作成完了: {len(atoms)} atoms")
    #     return atoms
    
    def create_al_structure(self):
        """Al基本構造の作成"""
        print("Al基本構造を作成中...")
        al_atoms = bulk("Al", "fcc", a=4.05, cubic=True)
        # al2o3_atoms = read('/home/jovyan/Kaori/MD/input/Al2O3.cif') 
        # alf3_atoms = read('/home/jovyan/Kaori/MD/input/AlF3_cubic.cif') 
        print(f"✅ Al構造作成完了: {len(al_atoms)} atoms")
        return al_atoms
    
    def modify_composition(self, atoms, target_composition, name="variant"):
        """組成変更（即座に保存）"""
        print(f"組成変更中: {name}")
        # if cif_path.exists():
        #     print(f"  既存ファイルをスキップ: {cif_path.name}")
        #     return str(cif_path)
        try:
            variant_atoms = atoms.copy()
            tm_indices = [i for i, atom in enumerate(variant_atoms) 
                         if atom.symbol in ["Ni", "Mn", "Co"]]
            
            if len(tm_indices) == 0:
                print(f"  警告: 遷移金属が見つかりません")
                return None
            
            total_tm_sites = len(tm_indices)
            target_counts = {symbol: round(fraction * total_tm_sites) 
                           for symbol, fraction in target_composition.items()}
            
            # 四捨五入調整
            current_total = sum(target_counts.values())
            diff = total_tm_sites - current_total
            if diff != 0:
                most_abundant = max(target_composition, key=target_composition.get)
                target_counts[most_abundant] += diff
            
            # 新しい元素リストを作成してシャッフル
            new_symbols = []
            for symbol, count in target_counts.items():
                new_symbols.extend([symbol] * count)
            
            random.seed(42)
            random.shuffle(tm_indices)
            
            for i, atom_index in enumerate(tm_indices):
                variant_atoms[atom_index].symbol = new_symbols[i]
            
            # 即座にCIF保存
            cif_path = self.cif_dir / f"NMC_{name}_bulk.cif"
            write(str(cif_path), variant_atoms)
            print(f"  ✅ 保存完了: {cif_path.name}")
            
            return variant_atoms
            
        except Exception as e:
            print(f"  ❌ 組成変更エラー: {e}")
            return None
        finally:
            # メモリクリーンアップ
            gc.collect()
    
    def optimize_structure_systematic(self, atoms, name="structure"):
        """
        構造最適化（BFGS + Matlantis 2段階版）
        1. ASEのBFGSで格子と原子位置を予備的に最適化
        2. Matlantis Featuresで最終的な精密最適化
        """
        print(f"=== {name}の構造最適化開始 (BFGS → Matlantis) ===")

        if self.calculator is None or not MATLANTIS_AVAILABLE:
            print("  -> ⚠️ 計算器が利用できないため、最適化をスキップします")
            return atoms

        try:
            # --- Step 1: ASEのBFGSによる予備最適化 ---
            print("  -> Step 1: BFGSによる予備最適化 (fmax=0.05)")
            write(f"{name}.xyz",atoms)
            # 計算機をatomsオブジェクトにセット
            temp_estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
            atoms.calc = ASECalculator(temp_estimator)

            # UnitCellFilterを使って格子と原子位置を同時に最適化
            bfgs_optimizer = BFGS(atoms)
            bfgs_optimizer.run(fmax=0.05, steps=100) # やや緩めの条件で実行
            
            print(f"    -> BFGS最適化完了。中間エネルギー: {atoms.get_potential_energy():.3f} eV")

            # --- Step 2: Matlantis Featuresによる精密最適化 ---
            print("  -> Step 2: Matlantis Featuresによる精密最適化 (fmax=0.01)")
            
            # BFGSで最適化された構造をMatlantisオプティマイザに入力
            estimator_function = pfp_estimator_fn(
                model_version='v7.0.0',
                calc_mode=EstimatorCalcMode.CRYSTAL_U0_PLUS_D3
            )

            matlantis_optimizer = FireLBFGSASEOptFeature(
                estimator_fn=estimator_function,
                n_run=3000,
                fmax=0.01,  # より厳密な条件
                show_progress_bar=True
            )
                            # filter=True,  # 格子も最終調整する
                # trajectory=str(self.output_dir / f"{name}_matlantis_final.traj"),
            result = matlantis_optimizer(MatlantisAtoms(atoms))
            optimized_atoms = result.atoms.ase_atoms
            final_energy = result.output.energy_log[-1]
            
            print(f"    -> ✔️ Matlantis最適化完了。最終エネルギー: {final_energy:.3f} eV")

            return optimized_atoms

        except Exception as e:
            print(f"  -> ❌ 最適化中にエラーが発生しました: {e}")
            # エラーが発生した場合は、元の構造をそのまま返す
            return atoms
        # finally:
        #     # メモリクリーンアップ
        #     gc.collect()
    
    def create_surface_and_save(self, atoms, miller, base_name):
        """表面作成と即座保存"""
        miller_str = f"{miller[0]}{miller[1]}{miller[2]}"
        surface_name = f"{base_name}_{miller_str}"
        cif_path = self.cif_dir / f"{surface_name}_surface.cif"
        
        # 既存ファイルのチェック
        if cif_path.exists():
            print(f"  既存ファイルをスキップ: {cif_path.name}")
            return str(cif_path)
        
        # try:
        print(f"  表面作成中: {surface_name}")
        
        # 元の設定値に戻す
        slab_unit = surface(atoms, miller, layers=4, vacuum=0.0)
        initial_dims=slab_unit.cell.lengths()
        target_dims = np.array([12, 12, 25])

        
        repean_num=np.ceil(target_dims / initial_dims).astype(int)
        slab = slab_unit.repeat(repeat_num)  # 元のリピート数に戻す
        
        # 位置調整
        positions = slab.get_positions()
        z_min = positions[:, 2].min()
        slab.set_positions(positions - [0, 0, z_min])
        
        # 下部原子の固定（元の固定比率）
        z_coords = slab.positions[:, 2]
        z_threshold = z_coords.min() + (z_coords.max() - z_coords.min()) * 0.3
        fixed_indices = [i for i, z in enumerate(z_coords) if z <= z_threshold]
        
        if fixed_indices:
            constraint = FixAtoms(indices=fixed_indices)
            slab.set_constraint(constraint)
        
        # 軽い最適化
        slab = self.optimize_structure_systematic(slab, surface_name)
        
        # 即座に保存
        write(str(cif_path), slab)
        print(f"  ✅ 表面保存完了: {cif_path.name}")
        
        return str(cif_path)
            
        # except Exception as e:
        #     print(f"  ❌ 表面作成失敗: {surface_name} - {e}")
        #     return None
        # finally:
        #     # メモリクリーンアップ
        #     # del slab if 'slab' in locals() else None
        #     gc.collect()
    
    def create_interface_and_save(self, surface1_path, surface2_path):
        """界面作成と即座保存（元の複雑なトリミング処理を復元）"""
        try:
            # ファイル名から界面名を生成
            name1 = Path(surface1_path).stem.replace('_surface', '')
            name2 = Path(surface2_path).stem.replace('_surface', '')
            interface_name = f"Interface_{name2}_on_{name1}"
            cif_path = self.cif_dir / f"{interface_name}.cif"
            
            # 既存ファイルのチェック
            if cif_path.exists():
                print(f"  既存界面をスキップ: {cif_path.name}")
                return str(cif_path)
            
            print(f"  界面作成中: {interface_name}")
            
            # 表面構造の読み込み
            slab1 = read(surface1_path)
            slab2 = read(surface2_path)
            separation = 2.0
            # XY方向のトリミング
            unit_thickness1 = slab1.cell.lengths()[2]
            # 目標とする厚みを設定
            target_thickness = 19.5
            # 必要なZ方向の繰り返し回数を計算 (小数点以下を切り上げ)
            if unit_thickness1 > 0:
                repeat_z1 = math.ceil(target_thickness / unit_thickness1)
            else:
                repeat_z = 1 # 厚みが0の場合は1回とする
            # 計算した回数でslab1をZ方向に繰り返す
            final_slab1 = slab1.repeat((1, 1, repeat_z1))
            
            unit_thickness2 = slab2.cell.lengths()[2]
            # 必要なZ方向の繰り返し回数を計算 (小数点以下を切り上げ)
            if unit_thickness2 > 0:
                repeat_z2 = math.ceil(target_thickness / unit_thickness2)
            else:
                repeat_z = 1 # 厚みが0の場合は1回とする
            # 計算した回数でslab1をZ方向に繰り返す
            final_slab2 = slab2.repeat((1, 1, repeat_z2))
            
            pos1 = final_slab1.get_positions()
            pos2 = final_slab2.get_positions()
            x_max=pos2[:,0].max()
            y_max=pos2[:,1].max()
            target_xy=(x_max,y_max)
            indices_to_delete1 = np.where((pos1[:, 1] < 0)|(pos1[:, 0] < 0)| (pos1[:, 0] >= target_xy[0]) | 
                                         (pos1[:, 1] >= target_xy[1]) | (pos1[:, 2] > 25))[0]
            indices_to_delete2 = np.where((pos2[:, 1] < 0)|(pos2[:, 0] < 0)|(pos2[:, 0] >= target_xy[0]) | 
                                         (pos2[:, 1] >= target_xy[1]) | (pos2[:, 2] > 25))[0]
            
            cut_slab1, cut_slab2 = final_slab1.copy(), final_slab2.copy()
            if len(indices_to_delete1) > 0: 
                del cut_slab1[indices_to_delete1]
            if len(indices_to_delete2) > 0: 
                del cut_slab2[indices_to_delete2]
            
            # Z方向の位置調整
            z1_max = cut_slab1.positions[:, 2].max()
            z2_min = cut_slab2.positions[:, 2].min()
            cut_slab2.positions[:, 2] += z1_max - z2_min + separation
            
            # 界面の結合
            interface = cut_slab1 + cut_slab2
            interface.cell = [x_max,y_max , 53]
            interface.pbc = [True, True, False]
            interface.center(vacuum=1.0, axis=2)
            
            # STEP 2: 界面のトリミング（元の複雑な処理）
            # interface = self.trim_interface(interface, thickness_per_slab=25.0)
            # if interface is None:
            #     print("   -> ❌ トリミングに失敗")
            #     return None
            
            # STEP 3: 構造最適化
            indices_to_fix = [atom.index for atom in interface if atom.position[2] <= 3.0]

            if indices_to_fix:
                print(f"    下部 {len(indices_to_fix)} 個の原子を固定します。")
                constraint = FixAtoms(indices=indices_to_fix)
                interface.set_constraint(constraint)
            else:
                print("    警告: 固定する原子が見つかりませんでした。")

            interface = self.optimize_structure_systematic(interface, interface_name)
            interface.set_constraint() 
            # 即座に保存
            write(str(cif_path), interface)
            print(f"  ✅ 界面保存完了: {cif_path.name}")
            
            return str(cif_path)
        except Exception as e:
            print(f"  ❌ 界面作成失敗: {e}")
            return None
        finally:
            # メモリクリーンアップ
            for var in ['slab1', 'slab2', 'cut_slab1', 'cut_slab2', 'interface']:
                if var in locals():
                    del locals()[var]
            gc.collect()
    
    def trim_interface(self, atoms, thickness_per_slab=25.0):
        """界面のトリミング（元の複雑な処理を復元）"""
        print(f"    2. 界面をトリミング中 (各スラブ厚み: {thickness_per_slab} Å)...")
        
        # 界面のZ座標を推定
        al_indices = [a.index for a in atoms if a.symbol == 'Al']
        nmc_indices = [a.index for a in atoms if a.symbol not in ['Al']]
        
        if not al_indices or not nmc_indices: 
            return None
            
        al_pos = atoms.get_positions()[al_indices]
        nmc_pos = atoms.get_positions()[nmc_indices]
        
        if np.mean(al_pos[:, 2]) < np.mean(nmc_pos[:, 2]):
            interface_z = (al_pos[:, 2].max() + nmc_pos[:, 2].min()) / 2
            is_al_bottom = True
        else:
            interface_z = (nmc_pos[:, 2].max() + al_pos[:, 2].min()) / 2
            is_al_bottom = False

        # 削除する原子を特定
        indices_to_delete = []
        for atom in atoms:
            if is_al_bottom:
                if atom.symbol == 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol != 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)
            else:
                if atom.symbol != 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol == 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)

        print(f"       -> {len(indices_to_delete)} 個の原子を削除")
        if len(indices_to_delete) > 0:
            del atoms[indices_to_delete]
        
        return atoms
    
    def process_single_nmc_variant(self, base_atoms, composition_info):
        """単一のNMC変異体の処理（メモリ効率版）"""
        name = composition_info["name"]
        composition = composition_info["composition"]
        
        print(f"\n--- {name}の処理開始 ---")
        
        # 1. 組成変更と保存
        variant_atoms = self.modify_composition(base_atoms, composition, name)
        if variant_atoms is None:
            return []
        
        # 2. 表面作成と保存（全Miller指数対応）
        surface_files = []
        for miller in self.miller_indices[:4]:  # NMC面のみ
            surface_file = self.create_surface_and_save(variant_atoms, miller, f"NMC_{name}")
            if surface_file:
                surface_files.append(surface_file)
        
        # メモリクリーンアップ
        del variant_atoms
        gc.collect()
        
        return surface_files
    
    def run_memory_efficient_workflow(self):
        """メモリ効率重視のワークフロー実行"""
        print("🚀 メモリ効率NMC-Al界面システム 開始")
        print("=" * 60)
        
        start_time = time.time()
        all_surface_files = []
        
        try:
            # 基本構造作成
            print("\n📐 基本構造作成")
            base_nmc = self.load_or_create_nmc_base()  # 元の関数名に修正
            base_al = self.create_al_structure()
            
            # NMC組成変異体の処理（一つずつ）
            print("\n🔬 NMC組成変異体処理")
            nmc_surfaces = []
            
            # pristine版も追加
            pristine_surfaces = self.process_single_nmc_variant(
            base_nmc, {"name": "pristine", "composition": {"Ni": 1/3, "Mn": 1/3, "Co": 1/3}})
            nmc_surfaces.extend(pristine_surfaces)
            
            # 全組成変異体の処理
            for comp_info in self.simulation_plan:
                surfaces = self.process_single_nmc_variant(base_nmc, comp_info)
                nmc_surfaces.extend(surfaces)
                # 各処理後にメモリクリーンアップ
                gc.collect()
            
            # Al表面の処理（全Miller指数）
            print("\n⚗️ Al表面処理")
            al_surfaces = []
            for miller in self.al_miller_indices[3:]:  # Al面のみ
                surface_file = self.create_surface_and_save(base_al, miller, "Al")
                if surface_file:
                    al_surfaces.append(surface_file)
            al2o3_atoms=read("/home/jovyan/Kaori/MD/LiB_2/structure/output/Al2O3_pos_opt_final.xyz")
            al2o3_path = self.cif_dir / "Al2O3_surface.cif" 
            write(al2o3_path, al2o3_atoms)
            # Atomsオブジェクトではなく、ファイルパスをリストに追加
            al_surfaces.append(str(al2o3_path))
            alf3_atoms=read("/home/jovyan/Kaori/MD/LiB_2/structure/output/AlF3_pos_opt_final.xyz")
            alf3_path = self.cif_dir / "AlF3_surface.cif" 
            write(alf3_path, alf3_atoms)
            al_surfaces.append(str(alf3_path))
            # Atomsオブジェクトではなく、ファイルパスをリストに追加
            # al_surfaces.append(al2o3)
            # al_surfaces.append(alf3)
            # メモリクリーンアップ
            del base_nmc, base_al
            gc.collect()
            
            # 界面作成（一つずつ処理）
            print("\n🔗 界面構造作成")
            interface_count = 0
            total_combinations = len(nmc_surfaces) * len(al_surfaces)
            current = 0
            
            for nmc_surface in nmc_surfaces:
                for al_surface in al_surfaces:
                    current += 1
                    print(f"  進行状況: {current}/{total_combinations}")
                    interface_file = self.create_interface_and_save(nmc_surface, al_surface)
                    if interface_file:
                        interface_count += 1
                    # 各界面作成後にメモリクリーンアップ
                    gc.collect()
            
            # 最終結果
            execution_time = time.time() - start_time
            
            print(f"\n✅ 処理完了!")
            print(f"  実行時間: {execution_time:.1f}秒")
            print(f"  NMC表面: {len(nmc_surfaces)}個")
            print(f"  Al表面: {len(al_surfaces)}個") 
            print(f"  界面構造: {interface_count}個")
            
            # 保存されたファイルの確認
            cif_files = list(self.cif_dir.glob("*.cif"))
            print(f"  保存されたCIFファイル総数: {len(cif_files)}個")
            
            # サマリー保存
            self.save_summary(len(nmc_surfaces), len(al_surfaces), interface_count, execution_time)
            
            return True
            
        except Exception as e:
            print(f"\n❌ ワークフロー実行エラー: {e}")
            return False
        finally:
            # 最終メモリクリーンアップ
            gc.collect()
    
    def save_summary(self, nmc_count, al_count, interface_count, exec_time):
        """実行サマリーの保存"""
        summary = f"""メモリ効率NMC-Al界面構造システム 実行結果
========================================
実行日時: {time.strftime('%Y-%m-%d %H:%M:%S')}
実行時間: {exec_time:.1f}秒

生成された構造:
- NMC表面構造: {nmc_count}個
- Al表面構造: {al_count}個  
- 界面構造: {interface_count}個

保存先: {self.cif_dir}

処理した組成:
- NMC pristine (Ni:Mn:Co = 1:1:1)
- Ni増減バリアント (高活性/低活性)
- Mn増減バリアント (構造安定性) 
- Co増減バリアント (電気化学特性)
- ハイニッケル化トレンド (NMC523/622/811)

処理したMiller指数:
- (001) NMC最安定面、プレート形状
- (104) NMC中活性、多面体形状
- (012) NMC高活性、切頭八面体形状
- (010) NMC Li拡散活性面
- (111) Al最安定面
- (100) 基本面
- (110) 副次面
"""
        
        summary_path = self.output_dir / "execution_summary.txt"
        with open(summary_path, "w", encoding="utf-8") as f:
            f.write(summary)
        
        print(f"📄 サマリー保存: {summary_path}")

def main():
    """メイン実行関数"""
    print("=" * 70)
    print("🧪 メモリ効率NMC-Al界面構造システム")
    print("   リチウムイオン電池リサイクル研究用")
    print("=" * 70)
    
    try:
        # システム初期化
        system = MemoryEfficientNMCSystem()
        
        # ワークフロー実行
        success = system.run_memory_efficient_workflow()
        
        if success:
            print("\n🎉 全処理が正常完了!")
            print(f"📂 結果: {system.cif_dir}")
        else:
            print("\n⚠️ 一部処理でエラーが発生しましたが、")
            print("   完了した結果は保存されています")
            
    except Exception as e:
        print(f"\n💥 システムエラー: {e}")
        
        # 最小限のフォールバック
        try:
            print("\n🔄 フォールバック実行...")
            fallback_system = MemoryEfficientNMCSystem()
            base_nmc = fallback_system.load_or_create_nmc_base()  # 元の関数名に修正
            
            fallback_path = fallback_system.cif_dir / "NMC_fallback.cif"
            write(str(fallback_path), base_nmc)
            print(f"✅ フォールバック完了: {fallback_path}")
            
        except Exception as fb_error:
            print(f"❌ フォールバックも失敗: {fb_error}")

def quick_test():
    """動作確認用クイックテスト"""
    print("🧪 クイックテスト開始")
    
    try:
        system = MemoryEfficientNMCSystem()
        
        # 基本構造テスト
        nmc = system.create_nmc_base_structure()
        al = system.create_al_structure()
        
        print(f"✅ NMC: {len(nmc)} atoms")
        print(f"✅ Al: {len(al)} atoms")
        
        # テスト組成変更
        test_comp = {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}
        variant = system.modify_composition(nmc, test_comp, "test")
        
        if variant:
            print("✅ 組成変更テスト成功")
        
        return True
        
    except Exception as e:
        print(f"❌ テスト失敗: {e}")
        return False

if __name__ == "__main__":
    # 通常実行
    main()
    
    # テスト実行の場合は以下をコメントアウト
    # quick_test()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 426)

In [55]:
from pfcc_extras import show_gui, view_ngl
from ase.io import Trajectory,read
# show_gui(read("/home/jovyan/Kaori/MD/LiB_2/structure/Interface_NMC_pristine_010_on_pre-optimized_AlF3.xyz"))
# traj=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/MD/compression/AlF3_NMC_Ni_increase_P0.001_T300K_compression.traj")
#traj=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/MD/tensile/AlF3_NMC_pristine_P0.001_T300K_HT500K_tensile300K.traj")

# cif=read("/home/jovyan/Kaori/MD/LiB_2/structure/output/cif_files/Interface_AlF3_on_NMC_NMC811_approx_001.cif")
# traj[1].cellshow_gui(traj
# traj=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/MD/heat/Al2O3_NMC_Mn_decrease_P0.010_T300K_HT500K.traj")
# traj=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/MD/compression/Al2O3_NMC_Co_decrease_P0.010_T300K_compression.traj")
# traj=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/MD/tensile/AlF3_NMC_pristine_P0.000_T300K_HT500K_tensile300K.traj")
# traj=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/MD/tensile/AlF3_NMC_NMC811_approx_P0.000_T300K_HT500K_tensile300K.traj")
# traj=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/MD/tensile1/AlF3_NMC_pristine_P0.000_T300K_HT500K_tensile300K.traj")
# traj=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/MD_P0/compression/AlF3_NMC_pristine_P0.000_T300K_compression.traj")
# traj=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/MD_P0/compression/AlF3_bulk_NMC_012_P10.000_T300K_compression.traj")
traj=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/MD4/tensile01/AlF3_NMC_pristine_P0.000_T300K_HT500K_tensile300K.traj")
# traj=read("/home/jovyan/Kaori/MD/LiB_2/structure/MD4/md_before.xyz")
traj=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/MD4/tensile01/AlF3_NMC_Ni_increase_P0.000_T300K_HT500K_tensile300K.traj")
# traj=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/MD_P0/tensile1/Al2O3_bulk_NMC_010_P10.000_T300K_HT800K_tensile300K.traj")
show_gui(traj)
# Kaori/MD/LiB_2/structure/MD4/heat/AlF3_NMC_pristine_P0.010_T300K_equil.traj
# traj[0].cell

In [2]:
#!/usr/bin/env python3
"""
メモリ効率改善版 統合NMC-Al界面構造生成・最適化システム
リチウムイオン電池リサイクル研究用

主な改善点:
- メモリ使用量を最小化
- 一つずつ処理して即座に保存
- 大量のデータを同時にメモリに保持しない
- エラー耐性の向上
- 必要最小限の構造のみ生成
"""

import numpy as np
import os, sys, time
from pathlib import Path
import random
import math
import gc  # ガベージコレクション用

# ASE
import ase
from ase import Atoms, units
from ase.io import read, write
from ase.build import surface, fcc100
from ase.constraints import FixAtoms
from ase.optimize import BFGS

# PFP (Matlantis) - エラー耐性版
try:
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    from pfp_api_client.pfp.estimator import Estimator
    from matlantis_features.atoms import MatlantisAtoms
    from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
    from matlantis_features.utils.calculators import pfp_estimator_fn
    from pfp_api_client.pfp.estimator import EstimatorCalcMode
    
    estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
    calculator = ASECalculator(estimator)
    MATLANTIS_AVAILABLE = True
    print("✅ PFP/Matlantis計算器を正常に初期化")
    
except ImportError as e:
    print(f"⚠️ PFP/Matlantisが利用できません: {e}")
    print("  基本ASE最適化のみを使用")
    calculator = None
    MATLANTIS_AVAILABLE = False

class MemoryEfficientNMCSystem:
    """メモリ効率重視のNMC-Al界面構造システム"""
    
    def __init__(self, base_dir="/home/jovyan/Kaori/MD/LiB_2"):
        self.base_dir = Path(base_dir)
        self.output_dir = self.base_dir / "structure" / "output"
        self.cif_dir = self.output_dir / "cif_files"
        
        # ディレクトリ作成
        for dir_path in [self.output_dir, self.cif_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)
        
        # 計算器設定
        self.calculator = calculator
        
        # Miller指数リスト（重要度順）
        self.miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 0, 4),  # NMC(104) - 中活性、多面体形状
            (0, 1, 2),  # NMC(012) - 高活性、切頭八面体形状
            (0, 1, 0),  # NMC(010) - Li拡散活性面
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        
        # シミュレーション計画の定義
        self.simulation_plan = [
            # ニッケル(Ni)増減の影響
            {"name": "Ni_increase", "composition": {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}},
            {"name": "Ni_decrease", "composition": {"Ni": 0.2, "Mn": 0.4, "Co": 0.4}},
            
            # マンガン(Mn)増減の影響
            {"name": "Mn_increase", "composition": {"Ni": 0.2, "Mn": 0.6, "Co": 0.2}},
            {"name": "Mn_decrease", "composition": {"Ni": 0.4, "Mn": 0.2, "Co": 0.4}},
            
            # コバルト(Co)増減の影響
            {"name": "Co_increase", "composition": {"Ni": 0.2, "Mn": 0.2, "Co": 0.6}},
            {"name": "Co_decrease", "composition": {"Ni": 0.4, "Mn": 0.4, "Co": 0.2}},
            
            # ハイニッケル化トレンド
            {"name": "NMC523_approx", "composition": {"Ni": 0.50, "Mn": 0.25, "Co": 0.25}},
            {"name": "NMC622_approx", "composition": {"Ni": 0.60, "Mn": 0.20, "Co": 0.20}},
            {"name": "NMC811_approx", "composition": {"Ni": 0.80, "Mn": 0.10, "Co": 0.10}},
        ]
        
        print(f"✅ メモリ効率システム初期化完了")
        print(f"  出力ディレクトリ: {self.output_dir}")
        print(f"  CIF保存先: {self.cif_dir}")
    
    def load_or_create_nmc_base(self):
        """NMC基本構造の読み込みまたは作成（元の機能を復元）"""
        input_dir = Path("/home/jovyan/Kaori/MD/input")
        cif_path = input_dir / "Li3MnCoNiO6.cif"
        
        if cif_path.exists():
            print(f"CIFファイルから読み込み: {cif_path}")
            try:
                base_atoms = read(str(cif_path))
                print(f"✅ CIF読み込み成功: {len(base_atoms)} atoms")
                return base_atoms
            except Exception as e:
                print(f"❌ CIF読み込み失敗: {e}")
        
        print("CIFファイルがないため、基本構造を生成します")
        return self.create_nmc_base_structure()
    
    # def create_nmc_base_structure(self):
    #     """NMC基本構造の作成（簡素版）"""
    #     print("NMC基本構造を作成中...")
        
    #     # 簡素化したNMC構造
    #     a = 2.87  # 格子定数
    #     c = 14.2
        
    #     positions = [
    #         [0.0, 0.0, 0.0],      # Li
    #         [0.5, 0.5, 0.0],      # Li
    #         [0.0, 0.5, 0.25],     # Ni
    #         [0.5, 0.0, 0.25],     # Mn
    #         [0.0, 0.0, 0.125],    # O
    #         [0.5, 0.5, 0.125],    # O
    #         [0.0, 0.5, 0.375],    # O
    #         [0.5, 0.0, 0.375],    # O
    #         [0.0, 0.0, 0.5],      # Co
    #         [0.5, 0.5, 0.5],      # Ni
    #     ]
        
    #     symbols = ['Li', 'Li', 'Ni', 'Mn', 'O', 'O', 'O', 'O', 'Co', 'Ni']
    #     cell = [[a, 0, 0], [0, a, 0], [0, 0, c]]
        
    #     atoms = Atoms(symbols=symbols, 
    #                  positions=np.array(positions) * [a, a, c],
    #                  cell=cell, 
    #                  pbc=True)
        
    #     print(f"✅ NMC基本構造作成完了: {len(atoms)} atoms")
    #     return atoms
    
    def create_al_structure(self):
        """Al基本構造の作成"""
        print("Al基本構造を作成中...")
        al_atoms = fcc100("Al", size=(6, 6, 10), orthogonal=True, periodic=True, vacuum=1.0)
        print(f"✅ Al構造作成完了: {len(al_atoms)} atoms")
        return al_atoms
    
    def modify_composition(self, atoms, target_composition, name="variant"):
        """組成変更（即座に保存）"""
        print(f"組成変更中: {name}")
        
        try:
            variant_atoms = atoms.copy()
            tm_indices = [i for i, atom in enumerate(variant_atoms) 
                         if atom.symbol in ["Ni", "Mn", "Co"]]
            
            if len(tm_indices) == 0:
                print(f"  警告: 遷移金属が見つかりません")
                return None
            
            total_tm_sites = len(tm_indices)
            target_counts = {symbol: round(fraction * total_tm_sites) 
                           for symbol, fraction in target_composition.items()}
            
            # 四捨五入調整
            current_total = sum(target_counts.values())
            diff = total_tm_sites - current_total
            if diff != 0:
                most_abundant = max(target_composition, key=target_composition.get)
                target_counts[most_abundant] += diff
            
            # 新しい元素リストを作成してシャッフル
            new_symbols = []
            for symbol, count in target_counts.items():
                new_symbols.extend([symbol] * count)
            
            random.seed(42)
            random.shuffle(tm_indices)
            
            for i, atom_index in enumerate(tm_indices):
                variant_atoms[atom_index].symbol = new_symbols[i]
            
            # 即座にCIF保存
            cif_path = self.cif_dir / f"NMC_{name}_bulk.cif"
            write(str(cif_path), variant_atoms)
            print(f"  ✅ 保存完了: {cif_path.name}")
            
            return variant_atoms
            
        except Exception as e:
            print(f"  ❌ 組成変更エラー: {e}")
            return None
        finally:
            # メモリクリーンアップ
            gc.collect()
    
    def optimize_if_possible(self, atoms, name="structure"):
        """最適化（可能な場合のみ）"""
        if not MATLANTIS_AVAILABLE or self.calculator is None:
            print(f"  計算器が利用できないため最適化をスキップ: {name}")
            return atoms
        
        try:
            print(f"  最適化実行中: {name}")
            atoms.calc = self.calculator
            
            # 簡単なBFGS最適化のみ
            optimizer = BFGS(atoms)
            optimizer.run(fmax=0.1, steps=50)  # 軽い最適化
            
            print(f"  ✅ 最適化完了: {name}")
            return atoms
            
        except Exception as e:
            print(f"  ⚠️ 最適化失敗、元の構造を使用: {e}")
            return atoms
        finally:
            gc.collect()
    
    def create_surface_and_save(self, atoms, miller, base_name):
        """表面作成と即座保存"""
        miller_str = f"{miller[0]}{miller[1]}{miller[2]}"
        surface_name = f"{base_name}_{miller_str}"
        cif_path = self.cif_dir / f"{surface_name}_surface.cif"
        
        # 既存ファイルのチェック
        if cif_path.exists():
            print(f"  既存ファイルをスキップ: {cif_path.name}")
            return str(cif_path)
        
        try:
            print(f"  表面作成中: {surface_name}")
            
            # 元の設定値に戻す
            slab = surface(atoms, miller, layers=4, vacuum=1.0)
            slab = slab.repeat([4, 4, 1])  # 元のリピート数に戻す
            
            # 位置調整
            positions = slab.get_positions()
            z_min = positions[:, 2].min()
            slab.set_positions(positions - [0, 0, z_min])
            
            # 下部原子の固定（元の固定比率）
            z_coords = slab.positions[:, 2]
            z_threshold = z_coords.min() + (z_coords.max() - z_coords.min()) * 0.3
            fixed_indices = [i for i, z in enumerate(z_coords) if z <= z_threshold]
            
            if fixed_indices:
                constraint = FixAtoms(indices=fixed_indices)
                slab.set_constraint(constraint)
            
            # 軽い最適化
            slab = self.optimize_if_possible(slab, surface_name)
            
            # 即座に保存
            write(str(cif_path), slab)
            print(f"  ✅ 表面保存完了: {cif_path.name}")
            
            return str(cif_path)
            
        except Exception as e:
            print(f"  ❌ 表面作成失敗: {surface_name} - {e}")
            return None
        finally:
            # メモリクリーンアップ
            # del slab if 'slab' in locals() else None
            gc.collect()
    
    def create_interface_and_save(self, surface1_path, surface2_path):
        """界面作成と即座保存"""
        try:
            # ファイル名から界面名を生成
            name1 = Path(surface1_path).stem.replace('_surface', '')
            name2 = Path(surface2_path).stem.replace('_surface', '')
            interface_name = f"Interface_{name1}_on_{name2}"
            cif_path = self.cif_dir / f"{interface_name}.cif"
            
            # 既存ファイルのチェック
            if cif_path.exists():
                print(f"  既存界面をスキップ: {cif_path.name}")
                return str(cif_path)
            
            print(f"  界面作成中: {interface_name}")
            
            # 表面構造の読み込み
            slab1 = read(surface1_path)
            slab2 = read(surface2_path)
            
            # 元のサイズ設定に戻す
            target_xy = (12.0, 12.0)  # 元の設定値
            separation = 2.0
            
            # XY方向のトリミング
            pos1 = slab1.get_positions()
            pos2 = slab2.get_positions()
            indices_to_delete1 = np.where((pos1[:, 0] >= target_xy[0]) | 
                                         (pos1[:, 1] >= target_xy[1]))[0]
            indices_to_delete2 = np.where((pos2[:, 0] >= target_xy[0]) | 
                                         (pos2[:, 1] >= target_xy[1]))[0]
            
            if len(indices_to_delete1) > 0: 
                del slab1[indices_to_delete1]
            if len(indices_to_delete2) > 0: 
                del slab2[indices_to_delete2]
            
            # Z方向の位置調整
            z1_max = slab1.positions[:, 2].max()
            z2_min = slab2.positions[:, 2].min()
            slab2.positions[:, 2] += z1_max - z2_min + separation
            
            # 界面結合
            interface = slab1 + slab2
            interface.cell = [12, 12, 53]  # 元のセルサイズ
            interface.pbc = [True, True, False]
            
            # 軽い最適化
            interface = self.optimize_if_possible(interface, interface_name)
            
            # 即座に保存
            write(str(cif_path), interface)
            print(f"  ✅ 界面保存完了: {cif_path.name}")
            
            return str(cif_path)
            
        except Exception as e:
            print(f"  ❌ 界面作成失敗: {e}")
            return None
        finally:
            # メモリクリーンアップ
            for var in ['slab1', 'slab2', 'interface']:
                if var in locals():
                    del locals()[var]
            gc.collect()
    
    def process_single_nmc_variant(self, base_atoms, composition_info):
        """単一のNMC変異体の処理（メモリ効率版）"""
        name = composition_info["name"]
        composition = composition_info["composition"]
        
        print(f"\n--- {name}の処理開始 ---")
        
        # 1. 組成変更と保存
        variant_atoms = self.modify_composition(base_atoms, composition, name)
        if variant_atoms is None:
            return []
        
        # 2. 表面作成と保存（全Miller指数対応）
        surface_files = []
        for miller in self.miller_indices[:4]:  # NMC面のみ
            surface_file = self.create_surface_and_save(variant_atoms, miller, f"NMC_{name}")
            if surface_file:
                surface_files.append(surface_file)
        
        # メモリクリーンアップ
        del variant_atoms
        gc.collect()
        
        return surface_files
    
    def run_memory_efficient_workflow(self):
        """メモリ効率重視のワークフロー実行"""
        print("🚀 メモリ効率NMC-Al界面システム 開始")
        print("=" * 60)
        
        start_time = time.time()
        all_surface_files = []
        
        try:
            # 基本構造作成
            print("\n📐 基本構造作成")
            base_nmc = self.load_or_create_nmc_base()  # 元の関数名に修正
            base_al = self.create_al_structure()
            
            # NMC組成変異体の処理（一つずつ）
            print("\n🔬 NMC組成変異体処理")
            nmc_surfaces = []
            
            # pristine版も追加
            pristine_surfaces = self.process_single_nmc_variant(
                base_nmc, {"name": "pristine", "composition": {"Ni": 1/3, "Mn": 1/3, "Co": 1/3}})
            nmc_surfaces.extend(pristine_surfaces)
            
            # 全組成変異体の処理
            for comp_info in self.simulation_plan:
                surfaces = self.process_single_nmc_variant(base_nmc, comp_info)
                nmc_surfaces.extend(surfaces)
                # 各処理後にメモリクリーンアップ
                gc.collect()
            
            # Al表面の処理（全Miller指数）
            print("\n⚗️ Al表面処理")
            al_surfaces = []
            for miller in self.miller_indices[4:]:  # Al面のみ
                surface_file = self.create_surface_and_save(base_al, miller, "Al")
                if surface_file:
                    al_surfaces.append(surface_file)
            
            # メモリクリーンアップ
            del base_nmc, base_al
            gc.collect()
            
            # 界面作成（一つずつ処理）
            print("\n🔗 界面構造作成")
            interface_count = 0
            total_combinations = len(nmc_surfaces) * len(al_surfaces)
            current = 0
            
            for nmc_surface in nmc_surfaces:
                for al_surface in al_surfaces:
                    current += 1
                    print(f"  進行状況: {current}/{total_combinations}")
                    interface_file = self.create_interface_and_save(nmc_surface, al_surface)
                    if interface_file:
                        interface_count += 1
                    # 各界面作成後にメモリクリーンアップ
                    gc.collect()
            
            # 最終結果
            execution_time = time.time() - start_time
            
            print(f"\n✅ 処理完了!")
            print(f"  実行時間: {execution_time:.1f}秒")
            print(f"  NMC表面: {len(nmc_surfaces)}個")
            print(f"  Al表面: {len(al_surfaces)}個") 
            print(f"  界面構造: {interface_count}個")
            
            # 保存されたファイルの確認
            cif_files = list(self.cif_dir.glob("*.cif"))
            print(f"  保存されたCIFファイル総数: {len(cif_files)}個")
            
            # サマリー保存
            self.save_summary(len(nmc_surfaces), len(al_surfaces), interface_count, execution_time)
            
            return True
            
        except Exception as e:
            print(f"\n❌ ワークフロー実行エラー: {e}")
            return False
        finally:
            # 最終メモリクリーンアップ
            gc.collect()
    
    def save_summary(self, nmc_count, al_count, interface_count, exec_time):
        """実行サマリーの保存"""
        summary = f"""メモリ効率NMC-Al界面構造システム 実行結果
========================================
実行日時: {time.strftime('%Y-%m-%d %H:%M:%S')}
実行時間: {exec_time:.1f}秒

生成された構造:
- NMC表面構造: {nmc_count}個
- Al表面構造: {al_count}個  
- 界面構造: {interface_count}個

保存先: {self.cif_dir}

処理した組成:
- NMC pristine (Ni:Mn:Co = 1:1:1)
- Ni増減バリアント (高活性/低活性)
- Mn増減バリアント (構造安定性) 
- Co増減バリアント (電気化学特性)
- ハイニッケル化トレンド (NMC523/622/811)

処理したMiller指数:
- (001) NMC最安定面、プレート形状
- (104) NMC中活性、多面体形状
- (012) NMC高活性、切頭八面体形状
- (010) NMC Li拡散活性面
- (111) Al最安定面
- (100) 基本面
- (110) 副次面
"""
        
        summary_path = self.output_dir / "execution_summary.txt"
        with open(summary_path, "w", encoding="utf-8") as f:
            f.write(summary)
        
        print(f"📄 サマリー保存: {summary_path}")

def main():
    """メイン実行関数"""
    print("=" * 70)
    print("🧪 メモリ効率NMC-Al界面構造システム")
    print("   リチウムイオン電池リサイクル研究用")
    print("=" * 70)
    
    try:
        # システム初期化
        system = MemoryEfficientNMCSystem()
        
        # ワークフロー実行
        success = system.run_memory_efficient_workflow()
        
        if success:
            print("\n🎉 全処理が正常完了!")
            print(f"📂 結果: {system.cif_dir}")
        else:
            print("\n⚠️ 一部処理でエラーが発生しましたが、")
            print("   完了した結果は保存されています")
            
    except Exception as e:
        print(f"\n💥 システムエラー: {e}")
        
        # 最小限のフォールバック
        try:
            print("\n🔄 フォールバック実行...")
            fallback_system = MemoryEfficientNMCSystem()
            base_nmc = fallback_system.load_or_create_nmc_base()  # 元の関数名に修正
            
            fallback_path = fallback_system.cif_dir / "NMC_fallback.cif"
            write(str(fallback_path), base_nmc)
            print(f"✅ フォールバック完了: {fallback_path}")
            
        except Exception as fb_error:
            print(f"❌ フォールバックも失敗: {fb_error}")

def quick_test():
    """動作確認用クイックテスト"""
    print("🧪 クイックテスト開始")
    
    try:
        system = MemoryEfficientNMCSystem()
        
        # 基本構造テスト
        nmc = system.create_nmc_base_structure()
        al = system.create_al_structure()
        
        print(f"✅ NMC: {len(nmc)} atoms")
        print(f"✅ Al: {len(al)} atoms")
        
        # テスト組成変更
        test_comp = {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}
        variant = system.modify_composition(nmc, test_comp, "test")
        
        if variant:
            print("✅ 組成変更テスト成功")
        
        return True
        
    except Exception as e:
        print(f"❌ テスト失敗: {e}")
        return False

if __name__ == "__main__":
    # 通常実行
    main()
    
    # テスト実行の場合は以下をコメントアウト
    # quick_test()

/home/jovyan/.py311/lib/python3.11/site-packages/pfp_api_client/__init__.py:42: UserWarning: New version of pfp-api-client is available. Please consider upgrading by `pip install -U pfp-api-client`.
  warnings.warn(
/home/jovyan/.py311/lib/python3.11/site-packages/pfp_api_client/__init__.py:42: UserWarning: New version of matlantis-features is available. Please consider upgrading by `pip install -U matlantis-features`.
  warnings.warn(


✅ PFP/Matlantis計算器を正常に初期化
🧪 メモリ効率NMC-Al界面構造システム
   リチウムイオン電池リサイクル研究用
✅ メモリ効率システム初期化完了
  出力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output
  CIF保存先: /home/jovyan/Kaori/MD/LiB_2/structure/output/cif_files
🚀 メモリ効率NMC-Al界面システム 開始

📐 基本構造作成
CIFファイルから読み込み: /home/jovyan/Kaori/MD/input/Li3MnCoNiO6.cif
✅ CIF読み込み成功: 24 atoms
Al基本構造を作成中...
✅ Al構造作成完了: 360 atoms

🔬 NMC組成変異体処理

--- pristineの処理開始 ---
組成変更中: pristine
  ✅ 保存完了: NMC_pristine_bulk.cif
  表面作成中: NMC_pristine_001
  最適化実行中: NMC_pristine_001


/home/jovyan/.py311/lib/python3.11/site-packages/ase/io/cif.py:836: UserWarning: Occupancies present but no occupancy info for "{symbol}"
  warnings.warn(str(err))


      Step     Time          Energy          fmax
BFGS:    0 08:01:06    -6822.190174        1.800482
BFGS:    1 08:01:33    -6835.800928        1.700498
BFGS:    2 08:02:03    -6899.684346        1.650250
BFGS:    3 08:02:29    -6905.576625        1.124758
BFGS:    4 08:02:56    -6914.488844        0.747194
BFGS:    5 08:03:24    -6915.852436        0.661951
BFGS:    6 08:03:50    -6920.333188        0.411116
BFGS:    7 08:04:15    -6921.455463        0.447866
BFGS:    8 08:04:42    -6925.389772        0.762203
BFGS:    9 08:05:14    -6928.621698        0.721262
BFGS:   10 08:05:46    -6931.295401        0.659223
BFGS:   11 08:06:18    -6933.545772        0.562671
BFGS:   12 08:06:51    -6934.662268        0.469111
BFGS:   13 08:07:24    -6935.963362        0.292615
BFGS:   14 08:07:56    -6936.568250        0.277663
BFGS:   15 08:08:28    -6936.926061        0.320853
BFGS:   16 08:09:00    -6937.240708        0.281843
BFGS:   17 08:09:33    -6937.523403        0.204711
BFGS:   18 08:

KeyboardInterrupt: 

In [ ]:
#!/usr/bin/env python3
"""
統合NMC-Al界面構造生成・最適化システム
リチウムイオン電池リサイクル研究用

完全統合版: 構造生成 → 表面作成 → 界面構築 → トリミング → 最適化

Features:
- NMC組成変動構造の自動生成
- 体系的な表面・界面構築
- Trajectoryファイルの自動整理
- 界面構造のトリミングと最適化
- 複数ミラー指数対応
- エラー耐性とロバスト処理
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys, time
from pathlib import Path
import random
import math
from collections import Counter

# ASE
import ase
from ase import Atoms, units
from ase.io import read, write
from ase.build import surface, molecule, add_adsorbate, fcc100, stack
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, ExpCellFilter, UnitCellFilter
from ase.optimize import BFGS, LBFGS, FIRE
from ase.visualize import view

# PFP (Matlantis) - 統合版
try:
    # 基本PFP
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    from pfp_api_client.pfp.estimator import Estimator
    from pfp_api_client.pfp.estimator import EstimatorCalcMode
    
    # Matlantis機能
    from matlantis_features.atoms import MatlantisAtoms
    from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
    from matlantis_features.utils.calculators import pfp_estimator_fn
    
    # 計算器初期化
    estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
    calculator = ASECalculator(estimator)
    MATLANTIS_AVAILABLE = True
    print("✅ PFP/Matlantis計算器を正常に初期化しました")
    
except ImportError as e:
    print(f"❌ PFP/Matlantisが利用できません: {e}")
    print("  基本ASE最適化のみを使用します")
    calculator = None
    MATLANTIS_AVAILABLE = False

class IntegratedNMCInterfaceSystem:
    """統合NMC-Al界面構造システム"""
    
    def __init__(self, base_dir="/home/jovyan/Kaori/MD/LiB_2"):
        self.base_dir = Path(base_dir)
        self.input_dir = Path("/home/jovyan/Kaori/MD/input")
        self.output_dir = self.base_dir / "structure" / "output"
        self.interfaces_dir = self.output_dir / "final_interfaces"
        
        # ディレクトリ作成
        for dir_path in [self.input_dir, self.output_dir, self.interfaces_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)
        
        # 計算器設定
        self.calculator = calculator
        
        # Miller指数リスト（重要度順）
        self.miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 0, 4),  # NMC(104) - 中活性、多面体形状
            (0, 1, 2),  # NMC(012) - 高活性、切頭八面体形状
            (0, 1, 0),  # NMC(010) - Li拡散活性面
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        
        print(f"✅ 統合システム初期化完了")
        print(f"  ベースディレクトリ: {self.base_dir}")
        print(f"  出力ディレクトリ: {self.output_dir}")
        print(f"  界面保存先: {self.interfaces_dir}")
    
    # =================== PHASE 1: 構造生成フェーズ ===================
    
    # def create_nmc_structure(self):
    #     """NMC111基本構造の作成（CIFファイルがない場合の代替）"""
    #     print("NMC111基本構造を作成中...")
        
    #     # 層状岩塩構造のNMC111を近似的に作成
    #     # Li層、遷移金属層、酸素層の積層構造
    #     a = 2.87  # 格子定数 (Å)
    #     c = 14.2  # c軸方向格子定数
        
    #     # 基本セルの原子位置
    #     positions = [
    #         # Li層 (z=0)
    #         [0.0, 0.0, 0.0],      # Li
    #         [0.6, 0.5, 0.0],      # Li
            
    #         # 遷移金属層 (z=0.25c)
    #         [0.0, 0.5, 0.25],     # Ni
    #         [0.5, 0.0, 0.25],     # Mn
            
    #         # 酸素層 (z=0.125c, z=0.375c)
    #         [0.0, 0.0, 0.125],    # O
    #         [0.5, 0.5, 0.125],    # O
    #         [0.0, 0.5, 0.375],    # O
    #         [0.5, 0.0, 0.375],    # O
            
    #         # 遷移金属層 (z=0.5c)
    #         [0.0, 0.0, 0.5],      # Co
    #         [0.5, 0.5, 0.5],      # Ni
            
    #         # Li層 (z=0.75c)
    #         [0.0, 0.5, 0.75],     # Li
    #         [0.5, 0.0, 0.75],     # Li
    #     ]
        
    #     symbols = ['Li', 'Li', 'Ni', 'Mn', 'O', 'O', 'O', 'O', 'Co', 'Ni', 'Li', 'Li']
        
    #     cell = [[a, 0, 0], [0, a, 0], [0, 0, c]]
        
    #     nmc_atoms = Atoms(symbols=symbols, 
    #                      positions=np.array(positions) * [a, a, c],
    #                      cell=cell, 
    #                      pbc=True)
        
    #     print(f"✅ NMC111構造作成完了: {len(nmc_atoms)} atoms")
    #     return nmc_atoms
    
    def load_or_create_nmc_base(self):
        """NMC基本構造の読み込みまたは作成"""
        cif_path = self.input_dir / "Li3MnCoNiO6.cif"
        
        if cif_path.exists():
            print(f"CIFファイルから読み込み: {cif_path}")
            try:
                base_atoms = read(str(cif_path))
                print(f"✅ CIF読み込み成功: {len(base_atoms)} atoms")
                return base_atoms
                print(f"❌ CIF読み込み失敗: {e}")
            except Exception as e:
                print(f"CIF: {e}")
        # print("CIFファイルがないため、基本構造を生成します")
        return self.create_nmc_structure()
    
    def optimize_structure_systematic(self, atoms, name="structure"):
        """
        構造最適化（BFGS + Matlantis 2段階版）
        1. ASEのBFGSで格子と原子位置を予備的に最適化
        2. Matlantis Featuresで最終的な精密最適化
        """
        print(f"=== {name}の構造最適化開始 (BFGS → Matlantis) ===")

        if self.calculator is None or not MATLANTIS_AVAILABLE:
            print("  -> ⚠️ 計算器が利用できないため、最適化をスキップします")
            return atoms

        try:
            # --- Step 1: ASEのBFGSによる予備最適化 ---
            # Matlantisの計算機をASEのオプティマイザのバックエンドとして使用します。
            # 格子最適化を行うため、stressを計算するモードが必要です。
            print("  -> Step 1: BFGSによる予備最適化 (fmax=0.05)")
            
            # 計算機をatomsオブジェクトにセット
            # ★★★重要★★★: BFGSで格子を動かすため、stress計算モードのCalculatorを使います
            temp_estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
            atoms.calc = ASECalculator(temp_estimator)

            # UnitCellFilterを使って格子と原子位置を同時に最適化
            bfgs_optimizer = BFGS(atoms)
            bfgs_optimizer.run(fmax=0.05, steps=3000) # やや緩めの条件で実行
            
            print(f"    -> BFGS最適化完了。中間エネルギー: {atoms.get_potential_energy():.3f} eV")

            # --- Step 2: Matlantis Featuresによる精密最適化 ---
            print("  -> Step 2: Matlantis Featuresによる精密最適化 (fmax=0.01)")
            
            # BFGSで最適化された構造をMatlantisオプティマイザに入力
            # estimator_function = pfp_estimator_fn(
            #     model_version='v7.0.0',
            #     calc_mode=EstimatorCalcMode.CRYSTAL_U0_PLUS_D3
            # )

            # matlantis_optimizer = FireLBFGSASEOptFeature(
            #     estimator_fn=estimator_function,
            #     filter=True,  # 格子も最終調整する
            #     trajectory=str(self.output_dir / f"{name}_matlantis_final.traj"),
            #     n_run=5000,
            #     fmax=0.01,  # より厳密な条件
            #     show_progress_bar=True
            # )
            
            # result = matlantis_optimizer(MatlantisAtoms(atoms))
            # optimized_atoms = result.atoms.ase_atoms
            # final_energy = result.output.energy_log[-1]
            
            # print(f"    -> ✔️ Matlantis最適化完了。最終エネルギー: {final_energy:.3f} eV")
            optimized_atoms=atoms
            return optimized_atoms

        except Exception as e:
            print(f"  -> ❌ 最適化中にエラーが発生しました: {e}")
            # エラーが発生した場合は、元の構造をそのまま返す
            return atoms

    
    def create_structure_from_target(self, base_atoms, target_composition, transition_metals):
        """目標組成から新しい構造を作成"""
        if not math.isclose(sum(target_composition.values()), 1.0):
            raise ValueError("目標組成の合計が1.0になりません。")
    
        variant_atoms = base_atoms.copy()
        tm_indices = [i for i, atom in enumerate(variant_atoms) if atom.symbol in transition_metals]
        total_tm_sites = len(tm_indices)
    
        if total_tm_sites == 0:
            print(f"警告: 置換対象の元素 {transition_metals} が見つかりません")
            return None
    
        # 目標原子数の計算
        target_counts = {symbol: round(fraction * total_tm_sites) 
                        for symbol, fraction in target_composition.items()}
    
        # 四捨五入による過不足を調整
        current_total = sum(target_counts.values())
        diff = total_tm_sites - current_total
        if diff != 0:
            most_abundant_element = max(target_composition, key=target_composition.get)
            target_counts[most_abundant_element] += diff
    
        # 新しい元素リストを作成
        new_symbols = []
        for symbol, count in target_counts.items():
            new_symbols.extend([symbol] * count)
    
        # ランダムに原子を再配置
        random.seed(42)
        random.shuffle(tm_indices)
    
        for i, atom_index in enumerate(tm_indices):
            variant_atoms[atom_index].symbol = new_symbols[i]
    
        return variant_atoms

    def create_composition_variants(self, base_atoms, base_name="NMC"):
        """組成変動構造の作成"""
        print(f"=== {base_name}の組成変動構造作成 ===")
        
        variants = {f"{base_name}_pristine": base_atoms.copy()}
        
        # シミュレーション計画の定義
        simulation_plan = [
            # ニッケル(Ni)増減の影響
            {"name": "Ni_increase", "composition": {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}},
            {"name": "Ni_decrease", "composition": {"Ni": 0.2, "Mn": 0.4, "Co": 0.4}},
            
            # マンガン(Mn)増減の影響
            {"name": "Mn_increase", "composition": {"Ni": 0.2, "Mn": 0.6, "Co": 0.2}},
            {"name": "Mn_decrease", "composition": {"Ni": 0.4, "Mn": 0.2, "Co": 0.4}},
            
            # コバルト(Co)増減の影響
            {"name": "Co_increase", "composition": {"Ni": 0.2, "Mn": 0.2, "Co": 0.6}},
            {"name": "Co_decrease", "composition": {"Ni": 0.4, "Mn": 0.4, "Co": 0.2}},
            
            # ハイニッケル化トレンド
            {"name": "NMC523_approx", "composition": {"Ni": 0.50, "Mn": 0.25, "Co": 0.25}},
            {"name": "NMC622_approx", "composition": {"Ni": 0.60, "Mn": 0.20, "Co": 0.20}},
            {"name": "NMC811_approx", "composition": {"Ni": 0.80, "Mn": 0.10, "Co": 0.10}},
        ]
        
        for target in simulation_plan:
            name = target["name"]
            composition = target["composition"]
            
            try:
                variant_atoms = self.create_structure_from_target(
                    base_atoms, composition, ["Ni", "Mn", "Co"])
                
                if variant_atoms is not None:
                    variant_name = f"{base_name}_{name}"
                    variants[variant_name] = variant_atoms
                    print(f"✅ 作成完了: {variant_name}")
                    
            except Exception as e:
                print(f"❌ エラー: {name} - {e}")
        
        return variants
    
    def makesurface_advanced(self, atoms, miller_indices=(1,0,0), layers=4, 
                           rep=[4,4,1], vacuum=15.0, fix_ratio=0.3, name="surface"):
        """改良版表面作成関数"""
        print(f"表面作成: {name} {miller_indices}面, {layers}層")
        
        try:
            slab = surface(atoms, miller_indices, layers, vacuum=vacuum)
            slab = slab.repeat(rep)
            
            # 位置調整
            positions = slab.get_positions()
            z_min = positions[:, 2].min()
            slab.set_positions(positions - [0, 0, z_min])
            
            # 下層原子の固定
            z_coords = slab.positions[:, 2]
            z_max = z_coords.max()
            z_threshold = z_coords.min() + (z_max - z_coords.min()) * fix_ratio
            fixed_indices = [i for i, z in enumerate(z_coords) if z <= z_threshold]
            
            if fixed_indices:
                constraint = FixAtoms(indices=fixed_indices)
                slab.set_constraint(constraint)
                print(f"固定原子数: {len(fixed_indices)}/{len(slab)}")
            
            # 構造最適化
            slab = self.optimize_structure_systematic(slab, f"{name}")
            
            return slab
        except Exception as e:
            print(f"表面作成エラー: {e}")
            return None
    
    def prepare_al_slab(self, size=(6, 6, 10), vacuum=0.0):
        """Al(100)スラブモデルを作成"""
        slab_al = fcc100("Al", size=size, orthogonal=True, periodic=True, vacuum=vacuum)
        print(f"Al(100)スラブ作成: サイズ={size}")
        return slab_al
    
    def build_al_structures(self):
        """Al構造の構築"""
        print("=== Al構造構築 ===")
        
        al_bulk = self.prepare_al_slab(size=(5,5,10), vacuum=1.0)
        
        al_surfaces = {}
        for miller in self.miller_indices[:3]:  # Al主要面のみ
            surface_name = f"Al_{miller[0]}{miller[1]}{miller[2]}"
            al_slab = self.makesurface_advanced(al_bulk, miller_indices=miller, 
                                              name=surface_name)
            if al_slab is not None:
                al_surfaces[surface_name] = al_slab
                
        return al_bulk, al_surfaces
    
    # =================== PHASE 2: ファイル整理フェーズ ===================
    
    def cleanup_trajectory_files(self):
        """Trajectoryファイルの整理"""
        print("\n--- Trajectoryファイルの分析と整理を開始 ---")
        
        traj_files = list(self.output_dir.glob("*.traj"))
        print(f"🔍 {len(traj_files)} 個の .traj ファイルを発見")
        
        for traj_path in traj_files:
            try:
                file_size = traj_path.stat().st_size
                # if file_size == 0:
                #     print(f"  -> 🗑️  0KBファイルを削除: {traj_path.name}")
                #     traj_path.unlink()
                #     continue

                # 正常なファイルは変換
                base_name = traj_path.stem
                initial_xyz_path = self.output_dir / f"{base_name}_initial.xyz"
                final_xyz_path = self.output_dir / f"{base_name}_final.xyz"

                print(f"  -> 🔄 変換中: {traj_path.name}")

                atoms_list = read(str(traj_path), index=":")
                if not atoms_list:
                    print(f"  -> ❌ 空のTrajectory、削除: {traj_path.name}")
                    traj_path.unlink()
                    continue

                # .xyzファイルとして保存
                write(str(initial_xyz_path), atoms_list[0])
                cif_path = traj_path.with_suffix('.cif')
                
                print(f"  -> 変換中: {traj_path.name} -> {cif_path.name}")
                write(str(cif_path), final_structure)
                # 元の.trajファイルを削除
                traj_path.unlink()
                print(f"  -> ✔️ 変換成功: {traj_path.name}")

            except Exception as e:
                print(f"  -> ❌ エラー ({traj_path.name}): {e}")

        print("✨ Trajectory整理完了")
    
    # =================== PHASE 3: 界面構築・最適化フェーズ ===================
    
    def run_matlantis_optimization(self, atoms, trajectory_path, fmax=0.05, name="structure"):
        """Matlantis最適化"""
        print(f"  -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
        
        if not MATLANTIS_AVAILABLE:
            print(f"  -> ⚠️ Matlantisが利用できないため、最適化をスキップします")
            return atoms
        
        try:
            matlantis_atoms = MatlantisAtoms(atoms)
            estimator_function = pfp_estimator_fn(
                model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
            )
            position_optimizer = FireLBFGSASEOptFeature(
                estimator_fn=estimator_function, filter=False, 
                trajectory=str(trajectory_path),
                n_run=5000, fmax=fmax, show_progress_bar=True
            )
            
            result = position_optimizer(matlantis_atoms)
            optimized_atoms = result.atoms.ase_atoms
            final_energy = result.output.energy_log[-1]
            print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
            
            # .xyzファイルでの保存
            xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
            write(str(xyz_save_path), optimized_atoms)
            return optimized_atoms
            
        except Exception as e:
            print(f"  -> ❌ 最適化中にエラーが発生: {e}")
            return None

    def find_and_categorize_surfaces(self):
        """表面ファイルの分類"""
        print(f"🔍 表面ファイルを分類中...")
        
        all_surfaces = list(self.output_dir.glob("*_final.xyz"))
        al_surfaces = [p for p in all_surfaces if "Al" in p.stem]
        cathode_surfaces = [p for p in all_surfaces if "NMC" in p.stem]
        
        print(f"  - Al表面: {len(al_surfaces)}個")
        print(f"  - 正極材表面: {len(cathode_surfaces)}個")
        
        return al_surfaces, cathode_surfaces

    def build_interface(self, slab1, slab2, target_xy=(12.0, 12.0), separation=2.0):
        """界面構築"""
        print(f"    1. 界面を構築中...")
        
        # XY方向のトリミング
        position1 = slab1.get_positions()
        position2 = slab2.get_positions()
        indices_to_delete1 = np.where((position1[:, 0] >= target_xy[0]) | 
                                     (position1[:, 1] >= target_xy[1]))[0]
        indices_to_delete2 = np.where((position2[:, 0] >= target_xy[0]) | 
                                     (position2[:, 1] >= target_xy[1]))[0]
        
        cut_slab1, cut_slab2 = slab1.copy(), slab2.copy()
        if len(indices_to_delete1) > 0: 
            del cut_slab1[indices_to_delete1]
        if len(indices_to_delete2) > 0: 
            del cut_slab2[indices_to_delete2]
        
        # Z方向の位置調整
        z1_max = cut_slab1.positions[:, 2].max()
        z2_min = cut_slab2.positions[:, 2].min()
        cut_slab2.positions[:, 2] += z1_max - z2_min + separation
        
        # 界面の結合
        interface = cut_slab1 + cut_slab2
        interface.cell = [12, 12, 53]
        interface.pbc = [True, True, False]
        
        return interface

    def trim_interface(self, atoms, thickness_per_slab=25.0):
        """界面のトリミング"""
        print(f"    2. 界面をトリミング中 (各スラブ厚み: {thickness_per_slab} Å)...")
        
        # 界面のZ座標を推定
        al_indices = [a.index for a in atoms if a.symbol == 'Al']
        nmc_indices = [a.index for a in atoms if a.symbol not in ['Al']]
        
        if not al_indices or not nmc_indices: 
            return None
            
        al_pos = atoms.get_positions()[al_indices]
        nmc_pos = atoms.get_positions()[nmc_indices]
        
        if np.mean(al_pos[:, 2]) < np.mean(nmc_pos[:, 2]):
            interface_z = (al_pos[:, 2].max() + nmc_pos[:, 2].min()) / 2
            is_al_bottom = True
        else:
            interface_z = (nmc_pos[:, 2].max() + al_pos[:, 2].min()) / 2
            is_al_bottom = False

        # 削除する原子を特定
        indices_to_delete = []
        for atom in atoms:
            if is_al_bottom:
                if atom.symbol == 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol != 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)
            else:
                if atom.symbol != 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol == 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)

        print(f"       -> {len(indices_to_delete)} 個の原子を削除")
        if len(indices_to_delete) > 0:
            del atoms[indices_to_delete]
        return atoms

    def create_trim_and_optimize_interface(self, slab1_path, slab2_path):
        """界面の作成、トリミング、最適化"""
        slab1_name = slab1_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        slab2_name = slab2_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        interface_name = f"Interface_{slab1_name}_on_{slab2_name}_trimmed"
        output_path = self.interfaces_dir / f"{interface_name}.cif"

        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return

        print(f"\n--- 処理開始: {slab1_name} + {slab2_name} ---")
        try:
            # STEP 1: 構築
            slab1, slab2 = read(str(slab1_path)), read(str(slab2_path))
            interface_built = self.build_interface(slab1, slab2)

            # STEP 2: トリミング
            interface_trimmed = self.trim_interface(interface_built, thickness_per_slab=25.0)
            if interface_trimmed is None:
                print("   -> ❌ トリミングに失敗")
                return

            # STEP 3: 最適化
            print(f"    3. トリミング後の構造を最適化中...")
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            optimized_structure = self.run_matlantis_optimization(
                interface_trimmed, traj_path, fmax=0.05, name=interface_name)

            # STEP 4: 保存
            if optimized_structure:
                optimized_structure.center(vacuum=15.0, axis=2)
                optimized_structure.pbc = (True, True, False)
                print(f"    4. 最終構造をファイルに保存中 -> {output_path.name}")
                write(str(output_path), optimized_structure)
                print(f"✅ 界面構造完成: {interface_name}")
            else:
                print("   -> ❌ 最適化に失敗")

        except Exception as e:
            print(f"  -> ❌ この界面の作成中にエラーが発生: {e}")

    def process_all_interfaces(self):
        """全界面の処理"""
        print("\n--- STEP 3: 界面構築・トリミング・最適化プロセス開始 ---")
        
        al_slabs, cathode_slabs = self.find_and_categorize_surfaces()
        if not al_slabs or not cathode_slabs:
            print(" -> 表面ファイルが不足しているため、スキップします。")
            return

        print(f"\n--- Al/NMC界面の組み合わせ処理を開始 ---")
        total_combinations = len(al_slabs) * len(cathode_slabs)
        current = 0
        
        for al_path in al_slabs:
            for cathode_path in cathode_slabs:
                current += 1
                print(f"\n進行状況: {current}/{total_combinations}")
                self.create_trim_and_optimize_interface(al_path, cathode_path)

        print("\n✨ 全ての界面処理が完了")
    
    # =================== メイン実行フェーズ ===================
    
    def save_structures(self, structures, phase_name="structures"):
        """構造の保存"""
        print(f"\n=== {phase_name}の保存 ===")
        
        for name, atoms in structures.items():
            try:
                if atoms is not None:
                    filepath = self.output_dir / f"{name}.traj"
                    write(str(filepath), atoms)
                    print(f"保存: {name}")
            except Exception as e:
                print(f"保存失敗: {name} - {e}")
        
        print(f"保存完了: {len(structures)}個の構造")

    def generate_final_summary(self):
        """最終サマリーの生成"""
        print("\n" + "=" * 70)
        print("🎯 統合NMC-Al界面構造システム 実行結果サマリー")
        print("=" * 70)
        
        # ファイル数のカウント
        cif_files = list(self.interfaces_dir.glob("*.cif"))
        xyz_files = list(self.output_dir.glob("*.xyz"))
        traj_files = list(self.output_dir.glob("*.traj"))
        
        print(f"\n📊 生成されたファイル:")
        print(f"  - 最終界面構造 (CIF): {len(cif_files)}個")
        print(f"  - 中間構造 (XYZ): {len(xyz_files)}個")
        print(f"  - Trajectoryファイル: {len(traj_files)}個")
        
        if cif_files:
            print(f"\n📋 生成された界面構造リスト:")
            for i, cif_file in enumerate(cif_files[:10], 1):
                file_size = cif_file.stat().st_size / 1024  # KB
                print(f"  {i:2d}. {cif_file.name} ({file_size:.1f} KB)")
            
            if len(cif_files) > 10:
                print(f"     ... (他 {len(cif_files) - 10} 個)")
        
        print(f"\n🔬 解析されたNMC組成バリエーション:")
        print(f"  - 基本NMC111構造")
        print(f"  - Ni増減バリアント (高活性/低活性)")
        print(f"  - Mn増減バリアント (構造安定性)")
        print(f"  - Co増減バリアント (電気化学特性)")
        print(f"  - ハイニッケル化トレンド (NMC523/622/811)")
        
        print(f"\n⚙️ 処理されたMiller指数:")
        miller_descriptions = {
            (0, 0, 1): "最安定面、プレート形状",
            (1, 0, 4): "中活性、多面体形状",
            (0, 1, 2): "高活性、切頭八面体形状",
            (0, 1, 0): "Li拡散活性面",
            (1, 1, 1): "Al最安定面",
            (1, 0, 0): "基本面",
            (1, 1, 0): "副次面"
        }
        for miller in self.miller_indices:
            desc = miller_descriptions.get(miller, "")
            print(f"  - {miller} : {desc}")
        
        print(f"\n🎯 次のステップ推奨事項:")
        print(f"  1. 界面接着強度の定量評価")
        print(f"  2. カレンダリング圧力下でのシミュレーション")
        print(f"  3. 熱処理プロセス条件の最適化")
        print(f"  4. 機械的分離効率の評価")
        print(f"  5. リサイクル歩留まりの予測")
        
        # サマリーをファイルに保存
        summary_text = []
        summary_text.append("統合NMC-Al界面構造システム 実行結果")
        summary_text.append("=" * 50)
        summary_text.append(f"実行日時: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        summary_text.append(f"")
        summary_text.append(f"生成されたファイル:")
        summary_text.append(f"- 最終界面構造 (CIF): {len(cif_files)}個")
        summary_text.append(f"- 中間構造 (XYZ): {len(xyz_files)}個")
        summary_text.append(f"- Trajectoryファイル: {len(traj_files)}個")
        summary_text.append(f"")
        summary_text.append("処理フェーズ:")
        summary_text.append("1. NMC組成バリアント生成")
        summary_text.append("2. Al/NMC表面構造作成")
        summary_text.append("3. Trajectoryファイル整理")
        summary_text.append("4. 界面構築・トリミング")
        summary_text.append("5. Matlantis最適化")
        summary_text.append("6. 最終構造保存")
        
        summary_path = self.output_dir / "execution_summary.txt"
        with open(summary_path, "w", encoding="utf-8") as f:
            f.write("\n".join(summary_text))
        
        print(f"\n💾 実行サマリーを保存しました: {summary_path}")
        print("=" * 70)
    
    def run_complete_workflow(self):
        """完全統合ワークフローの実行"""
        print("🚀 統合NMC-Al界面構造システム 開始")
        print("=" * 70)
        
        start_time = time.time()
        
        try:
            # =================== PHASE 1: 構造生成 ===================
            print("\n🔬 PHASE 1: NMC構造生成フェーズ")
            print("-" * 50)
            
            # 基本構造の読み込みまたは作成
            base_nmc = self.load_or_create_nmc_base()
            
            # 組成変動構造の作成
            nmc_variants = self.create_composition_variants(base_nmc, "NMC111")
            
            # Al構造の構築
            al_bulk, al_surfaces = self.build_al_structures()
            
            # 全構造の統合
            all_structures = {"Al_bulk": al_bulk}
            all_structures.update(nmc_variants)
            all_surfaces = al_surfaces.copy()
            
            # NMC表面の作成
            print("\n=== NMC表面構造作成 ===")
            for variant_name, atoms in nmc_variants.items():
                for miller in self.miller_indices[:4]:  # NMC主要面
                    try:
                        surface_name = f"{variant_name}_{miller[0]}{miller[1]}{miller[2]}"
                        slab = self.makesurface_advanced(atoms, miller_indices=miller,
                                                       name=surface_name)
                        if slab is not None:
                            all_surfaces[surface_name] = slab
                    except Exception as e:
                        print(f"表面作成失敗: {surface_name} - {e}")
            
            # 構造の保存
            self.save_structures(all_structures, "バルク構造")
            self.save_structures(all_surfaces, "表面構造")
            
            print(f"✅ PHASE 1 完了: バルク{len(all_structures)}個, 表面{len(all_surfaces)}個")
            
            # =================== PHASE 2: ファイル整理 ===================
            print("\n📁 PHASE 2: Trajectoryファイル整理フェーズ")
            print("-" * 50)
            
            self.cleanup_trajectory_files()
            print("✅ PHASE 2 完了")
            
            # =================== PHASE 3: 界面最適化 ===================
            print("\n⚙️ PHASE 3: 界面構築・最適化フェーズ")
            print("-" * 50)
            
            self.process_all_interfaces()
            print("✅ PHASE 3 完了")
            
            # =================== 最終サマリー ===================
            execution_time = time.time() - start_time
            print(f"\n⏱️ 総実行時間: {execution_time:.1f} 秒")
            self.cleanup_trajectory_files()
            self.generate_final_summary()
            
            return True
            
        except Exception as e:
            print(f"\n❌ ワークフロー実行中に重大なエラーが発生しました: {e}")
            print("エラーが発生しても、一部の処理は完了している可能性があります。")
            print("出力ディレクトリを確認してください。")
            return False


def main():
    """メイン実行関数"""
    print("=" * 80)
    print("🧪 統合NMC-Al界面構造生成・最適化システム")
    print("   リチウムイオン電池リサイクル研究用")
    print("=" * 80)
    
    # システム初期化
    try:
        system = IntegratedNMCInterfaceSystem()
        
        # 完全ワークフローの実行
        success = system.run_complete_workflow()
        
        if success:
            print("\n🎉 全てのワークフローが正常に完了しました！")
            print(f"📂 結果ファイル: {system.interfaces_dir}")
            print(f"📊 実行ログ: {system.output_dir / 'execution_summary.txt'}")
        else:
            print("\n⚠️ 一部のプロセスでエラーが発生しましたが、")
            print("   完了した処理の結果は保存されています。")
            
    except Exception as e:
        print(f"\n💥 システム初期化またはメイン実行でエラーが発生: {e}")
        print("環境設定または依存関係を確認してください。")
        
        # 最小限のフォールバック処理
        print("\n🔄 フォールバック: 基本的なNMC構造のみ生成します...")
        try:
            fallback_system = IntegratedNMCInterfaceSystem()
            base_nmc = fallback_system.load_or_create_nmc_base()
            
            # 基本構造のみ保存
            write(str(fallback_system.output_dir / "NMC111_fallback.xyz"), base_nmc)
            print(f"✅ フォールバック完了: {fallback_system.output_dir / 'NMC111_fallback.xyz'}")
            
        except Exception as fallback_error:
            print(f"❌ フォールバック処理も失敗: {fallback_error}")


if __name__ == "__main__":
    main()


# =================== 追加ユーティリティ関数 ===================

def analyze_interface_files(interfaces_dir):
    """界面ファイルの詳細分析"""
    from pathlib import Path
    
    interfaces_path = Path(interfaces_dir)
    if not interfaces_path.exists():
        print(f"ディレクトリが見つかりません: {interfaces_dir}")
        return
    
    print(f"\n🔍 界面構造ファイルの詳細分析")
    print(f"対象ディレクトリ: {interfaces_path}")
    print("-" * 60)
    
    cif_files = list(interfaces_path.glob("*.cif"))
    
    if not cif_files:
        print("CIFファイルが見つかりません。")
        return
    
    composition_analysis = {}
    miller_analysis = {}
    
    for cif_file in cif_files:
        try:
            atoms = read(str(cif_file))
            
            # 組成分析
            symbols = [atom.symbol for atom in atoms]
            composition = Counter(symbols)
            
            # ファイル名からNMC組成とMiller指数を抽出
            filename = cif_file.stem
            
            # NMC組成の抽出
            if "NMC111_Ni_" in filename:
                nmc_type = "Ni_variant"
            elif "NMC111_Mn_" in filename:
                nmc_type = "Mn_variant"
            elif "NMC111_Co_" in filename:
                nmc_type = "Co_variant"
            elif "NMC111_NMC" in filename:
                nmc_type = "High_Ni"
            else:
                nmc_type = "Standard"
            
            if nmc_type not in composition_analysis:
                composition_analysis[nmc_type] = []
            composition_analysis[nmc_type].append({
                'file': cif_file.name,
                'atoms': len(atoms),
                'composition': dict(composition)
            })
            
            # Miller指数の抽出（ファイル名から）
            for miller_str in ["001", "104", "012", "010", "111", "100", "110"]:
                if miller_str in filename:
                    if miller_str not in miller_analysis:
                        miller_analysis[miller_str] = 0
                    miller_analysis[miller_str] += 1
                    break
            
        except Exception as e:
            print(f"ファイル分析エラー: {cif_file.name} - {e}")
    
    # 結果の表示
    print(f"📊 組成別界面構造数:")
    for comp_type, files in composition_analysis.items():
        print(f"  {comp_type}: {len(files)}個")
        for file_info in files[:3]:  # 最初の3個のみ表示
            print(f"    - {file_info['file']} ({file_info['atoms']} atoms)")
    
    print(f"\n📐 Miller指数別界面構造数:")
    for miller, count in miller_analysis.items():
        print(f"  {miller}: {count}個")
    
    return composition_analysis, miller_analysis


def create_batch_visualization_script(interfaces_dir):
    """界面構造の一括可視化スクリプトを生成"""
    script_content = f'''#!/usr/bin/env python3
"""
NMC-Al界面構造一括可視化スクリプト
自動生成日: {time.strftime('%Y-%m-%d %H:%M:%S')}
"""

import matplotlib.pyplot as plt
from ase.io import read
from ase.visualize.plot import plot_atoms
from pathlib import Path
import numpy as np

def visualize_interfaces():
    interfaces_dir = Path(r"{interfaces_dir}")
    cif_files = list(interfaces_dir.glob("*.cif"))
    
    if not cif_files:
        print("CIFファイルが見つかりません")
        return
    
    # 最初の6個の界面構造を可視化
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, cif_file in enumerate(cif_files[:6]):
        try:
            atoms = read(str(cif_file))
            
            # 側面図を描画
            plot_atoms(atoms, axes[i], radii=0.5, rotation=('90x,0y,0z'))
            axes[i].set_title(f"{{cif_file.stem[:30]}}")
            axes[i].set_aspect('equal')
            
        except Exception as e:
            print(f"可視化エラー: {{cif_file.name}} - {{e}}")
            axes[i].text(0.5, 0.5, f'エラー\\n{{cif_file.stem}}', 
                        ha='center', va='center', transform=axes[i].transAxes)
    
    plt.tight_layout()
    plt.savefig(interfaces_dir / "interface_overview.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"可視化完了: {{interfaces_dir / 'interface_overview.png'}}")

if __name__ == "__main__":
    visualize_interfaces()
'''
    
    script_path = Path(interfaces_dir) / "visualize_interfaces.py"
    with open(script_path, 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    print(f"📊 可視化スクリプトを生成しました: {script_path}")
    return script_path


# =================== 使用例とテスト関数 ===================

def run_quick_test():
    """システムの動作確認用クイックテスト"""
    print("🧪 統合システム動作確認テスト開始")
    print("-" * 50)
    
    try:
        # テスト用システム初期化
        test_system = IntegratedNMCInterfaceSystem()
        
        # 基本NMC構造の生成テスト
        print("1. NMC構造生成テスト...")
        base_nmc = test_system.create_nmc_structure()
        print(f"   ✅ NMC構造: {len(base_nmc)} atoms")
        
        # Al構造生成テスト
        print("2. Al構造生成テスト...")
        al_slab = test_system.prepare_al_slab(size=(3,3,5))
        print(f"   ✅ Al構造: {len(al_slab)} atoms")
        
        # 組成変動の小規模テスト
        print("3. 組成変動テスト...")
        test_composition = {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}
        variant = test_system.create_structure_from_target(
            base_nmc, test_composition, ["Ni", "Mn", "Co"])
        if variant:
            print(f"   ✅ 組成変動: {len(variant)} atoms")
        
        print("✅ 全テスト通過")
        return True
        
    except Exception as e:
        print(f"❌ テスト失敗: {e}")
        return False


if __name__ == "__main__":
    # 通常実行
    main()
    
    # オプション: テストモードでの実行
    # run_quick_test()

/home/jovyan/.py311/lib/python3.11/site-packages/pfp_api_client/__init__.py:42: UserWarning: New version of pfp-api-client is available. Please consider upgrading by `pip install -U pfp-api-client`.
  warnings.warn(
/home/jovyan/.py311/lib/python3.11/site-packages/pfp_api_client/__init__.py:42: UserWarning: New version of matlantis-features is available. Please consider upgrading by `pip install -U matlantis-features`.
  warnings.warn(


✅ PFP/Matlantis計算器を正常に初期化しました
🧪 統合NMC-Al界面構造生成・最適化システム
   リチウムイオン電池リサイクル研究用
✅ 統合システム初期化完了
  ベースディレクトリ: /home/jovyan/Kaori/MD/LiB_2
  出力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output
  界面保存先: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces
🚀 統合NMC-Al界面構造システム 開始

🔬 PHASE 1: NMC構造生成フェーズ
--------------------------------------------------
CIFファイルから読み込み: /home/jovyan/Kaori/MD/input/Li3MnCoNiO6.cif
✅ CIF読み込み成功: 24 atoms
=== NMC111の組成変動構造作成 ===
✅ 作成完了: NMC111_Ni_increase
✅ 作成完了: NMC111_Ni_decrease
✅ 作成完了: NMC111_Mn_increase
✅ 作成完了: NMC111_Mn_decrease
✅ 作成完了: NMC111_Co_increase
✅ 作成完了: NMC111_Co_decrease
✅ 作成完了: NMC111_NMC523_approx
✅ 作成完了: NMC111_NMC622_approx
✅ 作成完了: NMC111_NMC811_approx
=== Al構造構築 ===
Al(100)スラブ作成: サイズ=(5, 5, 10)
表面作成: Al_001 (0, 0, 1)面, 4層
固定原子数: 4800/16000
=== Al_001の構造最適化開始 (BFGS → Matlantis) ===
  -> Step 1: BFGSによる予備最適化 (fmax=0.05)


In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys, time
from pathlib import Path
import random
import math
from collections import Counter

# ASE
import ase
from ase import Atoms, units
from ase.io import read, write
from ase.build import surface, molecule, add_adsorbate, fcc100, stack
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, ExpCellFilter, UnitCellFilter
from ase.optimize import BFGS, LBFGS, FIRE
from ase.visualize import view

In [1]:
#!/usr/bin/env python3
"""
統合NMC-Al界面構造生成・最適化システム
リチウムイオン電池リサイクル研究用

完全統合版: 構造生成 → 表面作成 → 界面構築 → トリミング → 最適化

Features:
- NMC組成変動構造の自動生成
- 体系的な表面・界面構築
- Trajectoryファイルの自動整理
- 界面構造のトリミングと最適化
- 複数ミラー指数対応
- エラー耐性とロバスト処理
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys, time
from pathlib import Path
import random
import math
from collections import Counter

# ASE
import ase
from ase import Atoms, units
from ase.io import read, write
from ase.build import surface, molecule, add_adsorbate, fcc100, stack
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, ExpCellFilter, UnitCellFilter
from ase.optimize import BFGS, LBFGS, FIRE
from ase.visualize import view

# PFP (Matlantis) - 統合版
try:
    # 基本PFP
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    from pfp_api_client.pfp.estimator import Estimator
    from pfp_api_client.pfp.estimator import EstimatorCalcMode
    
    # Matlantis機能
    from matlantis_features.atoms import MatlantisAtoms
    from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
    from matlantis_features.utils.calculators import pfp_estimator_fn
    
    # 計算器初期化
    estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
    calculator = ASECalculator(estimator)
    MATLANTIS_AVAILABLE = True
    print("✅ PFP/Matlantis計算器を正常に初期化しました")
    
except ImportError as e:
    print(f"❌ PFP/Matlantisが利用できません: {e}")
    print("  基本ASE最適化のみを使用します")
    calculator = None
    MATLANTIS_AVAILABLE = False

class IntegratedNMCInterfaceSystem:
    """統合NMC-Al界面構造システム"""
    
    def __init__(self, base_dir="/home/jovyan/Kaori/MD/LiB_2"):
        self.base_dir = Path(base_dir)
        self.input_dir = "/home/jovyan/Kaori/MD/input"
        self.output_dir = self.base_dir / "structure" / "output"
        self.interfaces_dir = self.output_dir / "final_interfaces"
        
        # ディレクトリ作成
        for dir_path in [self.input_dir, self.output_dir, self.interfaces_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)
        
        # 計算器設定
        self.calculator = calculator
        
        # Miller指数リスト（重要度順）
        self.miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 0, 4),  # NMC(104) - 中活性、多面体形状
            (0, 1, 2),  # NMC(012) - 高活性、切頭八面体形状
            (0, 1, 0),  # NMC(010) - Li拡散活性面
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        
        print(f"✅ 統合システム初期化完了")
        print(f"  ベースディレクトリ: {self.base_dir}")
        print(f"  出力ディレクトリ: {self.output_dir}")
        print(f"  界面保存先: {self.interfaces_dir}")
    
    # =================== PHASE 1: 構造生成フェーズ ===================
    
    def create_nmc_structure(self):
        """NMC111基本構造の作成（CIFファイルがない場合の代替）"""
        print("NMC111基本構造を作成中...")
        
        # 層状岩塩構造のNMC111を近似的に作成
        # Li層、遷移金属層、酸素層の積層構造
        a = 2.87  # 格子定数 (Å)
        c = 14.2  # c軸方向格子定数
        
        # 基本セルの原子位置
        positions = [
            # Li層 (z=0)
            [0.0, 0.0, 0.0],      # Li
            [0.5, 0.5, 0.0],      # Li
            
            # 遷移金属層 (z=0.25c)
            [0.0, 0.5, 0.25],     # Ni
            [0.5, 0.0, 0.25],     # Mn
            
            # 酸素層 (z=0.125c, z=0.375c)
            [0.0, 0.0, 0.125],    # O
            [0.5, 0.5, 0.125],    # O
            [0.0, 0.5, 0.375],    # O
            [0.5, 0.0, 0.375],    # O
            
            # 遷移金属層 (z=0.5c)
            [0.0, 0.0, 0.5],      # Co
            [0.5, 0.5, 0.5],      # Ni
            
            # Li層 (z=0.75c)
            [0.0, 0.5, 0.75],     # Li
            [0.5, 0.0, 0.75],     # Li
        ]
        
        symbols = ['Li', 'Li', 'Ni', 'Mn', 'O', 'O', 'O', 'O', 'Co', 'Ni', 'Li', 'Li']
        
        cell = [[a, 0, 0], [0, a, 0], [0, 0, c]]
        
        nmc_atoms = Atoms(symbols=symbols, 
                         positions=np.array(positions) * [a, a, c],
                         cell=cell, 
                         pbc=True)
        
        print(f"✅ NMC111構造作成完了: {len(nmc_atoms)} atoms")
        return nmc_atoms
    
    def load_or_create_nmc_base(self):
        """NMC基本構造の読み込みまたは作成"""
        cif_path = self.input_dir / "Li3MnCoNiO6.cif"
        
        if cif_path.exists():
            print(f"CIFファイルから読み込み: {cif_path}")
            try:
                base_atoms = read(str(cif_path))
                print(f"✅ CIF読み込み成功: {len(base_atoms)} atoms")
                return base_atoms
            except Exception as e:
                print(f"❌ CIF読み込み失敗: {e}")
        
        print("CIFファイルがないため、基本構造を生成します")
        return self.create_nmc_structure()
    
    def optimize_structure_systematic(self, atoms, name="structure"):
        """FireLBFGS最適化（Matlantis使用）"""
        print(f"=== {name}のMatlantis最適化開始 ===")
        
        if not MATLANTIS_AVAILABLE:
            print("Matlantisが利用できないため、最適化をスキップします")
            return atoms
        
        # FireLBFGS最適化を実行
        trajectory_path = self.output_dir / f"{name}_optimization.traj"
        optimized_atoms = self.run_matlantis_optimization(
            atoms, trajectory_path, fmax=0.05, name=name
        )
        
        return optimized_atoms if optimized_atoms is not None else atoms
    
    def create_structure_from_target(self, base_atoms, target_composition, transition_metals):
        """目標組成から新しい構造を作成"""
        if not math.isclose(sum(target_composition.values()), 1.0):
            raise ValueError("目標組成の合計が1.0になりません。")
    
        variant_atoms = base_atoms.copy()
        tm_indices = [i for i, atom in enumerate(variant_atoms) if atom.symbol in transition_metals]
        total_tm_sites = len(tm_indices)
    
        if total_tm_sites == 0:
            print(f"警告: 置換対象の元素 {transition_metals} が見つかりません")
            return None
    
        # 目標原子数の計算
        target_counts = {symbol: round(fraction * total_tm_sites) 
                        for symbol, fraction in target_composition.items()}
    
        # 四捨五入による過不足を調整
        current_total = sum(target_counts.values())
        diff = total_tm_sites - current_total
        if diff != 0:
            most_abundant_element = max(target_composition, key=target_composition.get)
            target_counts[most_abundant_element] += diff
    
        # 新しい元素リストを作成
        new_symbols = []
        for symbol, count in target_counts.items():
            new_symbols.extend([symbol] * count)
    
        # ランダムに原子を再配置
        random.seed(42)
        random.shuffle(tm_indices)
    
        for i, atom_index in enumerate(tm_indices):
            variant_atoms[atom_index].symbol = new_symbols[i]
    
        return variant_atoms

    def create_composition_variants(self, base_atoms, base_name="NMC"):
        """組成変動構造の作成"""
        print(f"=== {base_name}の組成変動構造作成 ===")
        
        variants = {f"{base_name}_pristine": base_atoms.copy()}
        
        # シミュレーション計画の定義（簡略版）
        simulation_plan = [
            # ニッケル(Ni)増減の影響
            {"name": "Ni_increase", "composition": {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}},
            {"name": "Ni_decrease", "composition": {"Ni": 0.2, "Mn": 0.4, "Co": 0.4}},
            
            # マンガン(Mn)増減の影響
            {"name": "Mn_increase", "composition": {"Ni": 0.2, "Mn": 0.6, "Co": 0.2}},
            {"name": "Mn_decrease", "composition": {"Ni": 0.4, "Mn": 0.2, "Co": 0.4}},
            
            # コバルト(Co)増減の影響
            {"name": "Co_increase", "composition": {"Ni": 0.2, "Mn": 0.2, "Co": 0.6}},
            {"name": "Co_decrease", "composition": {"Ni": 0.4, "Mn": 0.4, "Co": 0.2}},
            
            # ハイニッケル化トレンドの評価
            {"name": "NMC523_approx", "composition": {"Ni": 0.50, "Mn": 0.25, "Co": 0.25}},
            {"name": "NMC622_approx", "composition": {"Ni": 0.60, "Mn": 0.20, "Co": 0.20}},
            {"name": "NMC811_approx", "composition": {"Ni": 0.80, "Mn": 0.10, "Co": 0.10}},
        ]
        
        for target in simulation_plan:
            name = target["name"]
            composition = target["composition"]
            
            try:
                variant_atoms = self.create_structure_from_target(
                    base_atoms, composition, ["Ni", "Mn", "Co"])
                
                if variant_atoms is not None:
                    variant_name = f"{base_name}_{name}"
                    variants[variant_name] = variant_atoms
                    print(f"✅ 作成完了: {variant_name}")
                    
            except Exception as e:
                print(f"❌ エラー: {name} - {e}")
        
        return variants
    
    def makesurface_advanced(self, atoms, miller_indices=(1,0,0), layers=4, 
                           rep=[4,4,1], vacuum=15.0, fix_ratio=0.3, name="surface"):
        """改良版表面作成関数"""
        print(f"表面作成: {name} {miller_indices}面, {layers}層")
        
        try:
            slab = surface(atoms, miller_indices, layers, vacuum=vacuum)
            slab = slab.repeat(rep)
            
            # 位置調整
            positions = slab.get_positions()
            z_min = positions[:, 2].min()
            slab.set_positions(positions - [0, 0, z_min])
            
            # 下層原子の固定
            z_coords = slab.positions[:, 2]
            z_max = z_coords.max()
            z_threshold = z_coords.min() + (z_max - z_coords.min()) * fix_ratio
            fixed_indices = [i for i, z in enumerate(z_coords) if z <= z_threshold]
            
            if fixed_indices:
                constraint = FixAtoms(indices=fixed_indices)
                slab.set_constraint(constraint)
                print(f"固定原子数: {len(fixed_indices)}/{len(slab)}")
            
            # 構造最適化
            slab = self.optimize_structure_systematic(slab, f"{name}")
            
            return slab
        except Exception as e:
            print(f"表面作成エラー: {e}")
            return None
    
    def prepare_al_slab(self, size=(6, 6, 10), vacuum=5.0):
        """Al(100)スラブモデルを作成"""
        slab_al = fcc100("Al", size=size, orthogonal=True, periodic=True, vacuum=vacuum)
        print(f"Al(100)スラブ作成: サイズ={size}")
        return slab_al
    
    def build_al_structures(self):
        """Al構造の構築"""
        print("=== Al構造構築 ===")
        
        al_bulk = self.prepare_al_slab(size=(5,5,10), vacuum=1.0)
        
        al_surfaces = {}
        for miller in self.miller_indices[:3]:  # Al主要面のみ
            surface_name = f"Al_{miller[0]}{miller[1]}{miller[2]}"
            al_slab = self.makesurface_advanced(al_bulk, miller_indices=miller, 
                                              name=surface_name)
            if al_slab is not None:
                al_surfaces[surface_name] = al_slab
                
        return al_bulk, al_surfaces
    
    # =================== PHASE 2: ファイル整理フェーズ ===================
    
    def cleanup_trajectory_files(self):
        """Trajectoryファイルの整理"""
        print("\n--- Trajectoryファイルの分析と整理を開始 ---")
        
        traj_files = list(self.output_dir.glob("*.traj"))
        print(f"🔍 {len(traj_files)} 個の .traj ファイルを発見")
        
        for traj_path in traj_files:
            try:
                file_size = traj_path.stat().st_size
                if file_size == 0:
                    print(f"  -> 🗑️  0KBファイルを削除: {traj_path.name}")
                    traj_path.unlink()
                    continue

                # 正常なファイルは変換
                base_name = traj_path.stem
                initial_xyz_path = self.output_dir / f"{base_name}_initial.xyz"
                final_xyz_path = self.output_dir / f"{base_name}_final.xyz"

                print(f"  -> 🔄 変換中: {traj_path.name}")

                atoms_list = read(str(traj_path), index=":")
                if not atoms_list:
                    print(f"  -> ❌ 空のTrajectory、削除: {traj_path.name}")
                    traj_path.unlink()
                    continue

                # .xyzファイルとして保存
                write(str(initial_xyz_path), atoms_list[0])
                write(str(final_xyz_path), atoms_list[-1])

                # 元の.trajファイルを削除
                traj_path.unlink()
                print(f"  -> ✔️ 変換成功: {traj_path.name}")

            except Exception as e:
                print(f"  -> ❌ エラー ({traj_path.name}): {e}")

        print("✨ Trajectory整理完了")
    
    # =================== PHASE 3: 界面構築・最適化フェーズ ===================
    
    def run_matlantis_optimization(self, atoms, trajectory_path, fmax=0.05, name="structure"):
        """Matlantis最適化"""
        print(f"  -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
        
        if not MATLANTIS_AVAILABLE:
            print(f"  -> ⚠️ Matlantisが利用できないため、最適化をスキップします")
            return atoms
        
        try:
            matlantis_atoms = MatlantisAtoms(atoms)
            estimator_function = pfp_estimator_fn(
                model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
            )
            position_optimizer = FireLBFGSASEOptFeature(
                estimator_fn=estimator_function, filter=False, 
                trajectory=str(trajectory_path),
                n_run=5000, fmax=fmax, show_progress_bar=True
            )
            
            result = position_optimizer(matlantis_atoms)
            optimized_atoms = result.atoms.ase_atoms
            final_energy = result.output.energy_log[-1]
            print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
            
            # .xyzファイルでの保存
            xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
            write(str(xyz_save_path), optimized_atoms)
            return optimized_atoms
            
        except Exception as e:
            print(f"  -> ❌ 最適化中にエラーが発生: {e}")
            return None

    def find_and_categorize_surfaces(self):
        """表面ファイルの分類"""
        print(f"🔍 表面ファイルを分類中...")
        
        all_surfaces = list(self.output_dir.glob("*_final.xyz"))
        al_surfaces = [p for p in all_surfaces if "Al" in p.stem]
        cathode_surfaces = [p for p in all_surfaces if "NMC" in p.stem]
        
        print(f"  - Al表面: {len(al_surfaces)}個")
        print(f"  - 正極材表面: {len(cathode_surfaces)}個")
        
        return al_surfaces, cathode_surfaces

    def build_interface(self, slab1, slab2, target_xy=(12.0, 12.0), separation=2.0):
        """界面構築"""
        print(f"    1. 界面を構築中...")
        
        # XY方向のトリミング
        position1 = slab1.get_positions()
        position2 = slab2.get_positions()
        indices_to_delete1 = np.where((position1[:, 0] >= target_xy[0]) | 
                                     (position1[:, 1] >= target_xy[1]))[0]
        indices_to_delete2 = np.where((position2[:, 0] >= target_xy[0]) | 
                                     (position2[:, 1] >= target_xy[1]))[0]
        
        cut_slab1, cut_slab2 = slab1.copy(), slab2.copy()
        if len(indices_to_delete1) > 0: 
            del cut_slab1[indices_to_delete1]
        if len(indices_to_delete2) > 0: 
            del cut_slab2[indices_to_delete2]
        
        # Z方向の位置調整
        z1_max = cut_slab1.positions[:, 2].max()
        z2_min = cut_slab2.positions[:, 2].min()
        cut_slab2.positions[:, 2] += z1_max - z2_min + separation
        
        # 界面の結合
        interface = cut_slab1 + cut_slab2
        interface.cell = [12, 12, 53]
        interface.pbc = [True, True, False]
        
        return interface

    def trim_interface(self, atoms, thickness_per_slab=25.0):
        """界面のトリミング"""
        print(f"    2. 界面をトリミング中 (各スラブ厚み: {thickness_per_slab} Å)...")
        
        # 界面のZ座標を推定
        al_indices = [a.index for a in atoms if a.symbol == 'Al']
        nmc_indices = [a.index for a in atoms if a.symbol not in ['Al']]
        
        if not al_indices or not nmc_indices: 
            return None
            
        al_pos = atoms.get_positions()[al_indices]
        nmc_pos = atoms.get_positions()[nmc_indices]
        
        if np.mean(al_pos[:, 2]) < np.mean(nmc_pos[:, 2]):
            interface_z = (al_pos[:, 2].max() + nmc_pos[:, 2].min()) / 2
            is_al_bottom = True
        else:
            interface_z = (nmc_pos[:, 2].max() + al_pos[:, 2].min()) / 2
            is_al_bottom = False

        # 削除する原子を特定
        indices_to_delete = []
        for atom in atoms:
            if is_al_bottom:
                if atom.symbol == 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol != 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)
            else:
                if atom.symbol != 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol == 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)

        print(f"       -> {len(indices_to_delete)} 個の原子を削除")
        if len(indices_to_delete) > 0:
            del atoms[indices_to_delete]
        return atoms

    def create_trim_and_optimize_interface(self, slab1_path, slab2_path):
        """界面の作成、トリミング、最適化"""
        slab1_name = slab1_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        slab2_name = slab2_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        interface_name = f"Interface_{slab1_name}_on_{slab2_name}_trimmed"
        output_path = self.interfaces_dir / f"{interface_name}.cif"

        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return

        print(f"\n--- 処理開始: {slab1_name} + {slab2_name} ---")
        try:
            # STEP 1: 構築
            slab1, slab2 = read(str(slab1_path)), read(str(slab2_path))
            interface_built = self.build_interface(slab1, slab2)

            # STEP 2: トリミング
            interface_trimmed = self.trim_interface(interface_built, thickness_per_slab=25.0)
            if interface_trimmed is None:
                print("   -> ❌ トリミングに失敗")
                return

            # STEP 3: 最適化
            print(f"    3. トリミング後の構造を最適化中...")
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            optimized_structure = self.run_matlantis_optimization(
                interface_trimmed, traj_path, fmax=0.05, name=interface_name)

            # STEP 4: 保存
            if optimized_structure:
                optimized_structure.center(vacuum=15.0, axis=2)
                optimized_structure.pbc = (True, True, False)
                print(f"    4. 最終構造をファイルに保存中 -> {output_path.name}")
                write(str(output_path), optimized_structure)
                print(f"✅ 界面構造完成: {interface_name}")
            else:
                print("   -> ❌ 最適化に失敗")

        except Exception as e:
            print(f"  -> ❌ この界面の作成中にエラーが発生: {e}")

    def process_all_interfaces(self):
        """全界面の処理"""
        print("\n--- STEP 3: 界面構築・トリミング・最適化プロセス開始 ---")
        
        al_slabs, cathode_slabs = self.find_and_categorize_surfaces()
        if not al_slabs or not cathode_slabs:
            print(" -> 表面ファイルが不足しているため、スキップします。")
            return

        print(f"\n--- Al/NMC界面の組み合わせ処理を開始 ---")
        total_combinations = len(al_slabs) * len(cathode_slabs)
        current = 0
        
        for al_path in al_slabs:
            for cathode_path in cathode_slabs:
                current += 1
                print(f"\n進行状況: {current}/{total_combinations}")
                self.create_trim_and_optimize_interface(al_path, cathode_path)

        print("\n✨ 全ての界面処理が完了")
    
    # =================== メイン実行フェーズ ===================
    
    def save_structures(self, structures, phase_name="structures"):
        """構造の保存"""
        print(f"\n=== {phase_name}の保存 ===")
        
        for name, atoms in structures.items():
            try:
                if atoms is not None:
                    filepath = self.output_dir / f"{name}.traj"
                    write(str(filepath), atoms)
                    print(f"保存: {name}")
            except Exception as e:
                print(f"保存失敗: {name} - {e}")
        
        print(f"保存完了: {len(structures)}個の構造")

    def generate_final_summary(self):
        """最終サマリーの生成"""
        print("\n" + "=" * 70)
        print("🎯 統合NMC-Al界面構造システム 実行結果サマリー")
        print("=" * 70)
        
        # ファイル数のカウント
        cif_files = list(self.interfaces_dir.glob("*.cif"))
        xyz_files = list(self.output_dir.glob("*.xyz"))
        traj_files = list(self.output_dir.glob("*.traj"))
        
        print(f"\n📊 生成されたファイル:")
        print(f"  - 最終界面構造 (CIF): {len(cif_files)}個")
        print(f"  - 中間構造 (XYZ): {len(xyz_files)}個")
        print(f"  - Trajectoryファイル: {len(traj_files)}個")
        
        if cif_files:
            print(f"\n📋 生成された界面構造リスト:")
            for i, cif_file in enumerate(cif_files[:10], 1):
                file_size = cif_file.stat().st_size / 1024  # KB
                print(f"  {i:2d}. {cif_file.name} ({file_size:.1f} KB)")
            
            if len(cif_files) > 10:
                print(f"     ... (他 {len(cif_files) - 10} 個)")
        
        print(f"\n🔬 解析されたNMC組成バリエーション:")
        print(f"  - 基本NMC111構造")
        print(f"  - Ni増減バリアント (高活性/低活性)")
        print(f"  - Mn増減バリアント (構造安定性)")
        print(f"  - Co増減バリアント (電気化学特性)")
        print(f"  - ハイニッケル化トレンド (NMC523/622/811)")
        
        print(f"\n⚙️ 処理されたMiller指数:")
        miller_descriptions = {
            (0, 0, 1): "最安定面、プレート形状",
            (1, 0, 4): "中活性、多面体形状",
            (0, 1, 2): "高活性、切頭八面体形状",
            (0, 1, 0): "Li拡散活性面",
            (1, 1, 1): "Al最安定面",
            (1, 0, 0): "基本面",
            (1, 1, 0): "副次面"
        }
        for miller in self.miller_indices:
            desc = miller_descriptions.get(miller, "")
            print(f"  - {miller} : {desc}")
        
        print(f"\n🎯 次のステップ推奨事項:")
        print(f"  1. 界面接着強度の定量評価")
        print(f"  2. カレンダリング圧力下でのシミュレーション")
        print(f"  3. 熱処理プロセス条件の最適化")
        print(f"  4. 機械的分離効率の評価")
        print(f"  5. リサイクル歩留まりの予測")
        
        # サマリーをファイルに保存
        summary_text = []
        summary_text.append("統合NMC-Al界面構造システム 実行結果")
        summary_text.append("=" * 50)
        summary_text.append(f"実行日時: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        summary_text.append(f"")
        summary_text.append(f"生成されたファイル:")
        summary_text.append(f"- 最終界面構造 (CIF): {len(cif_files)}個")
        summary_text.append(f"- 中間構造 (XYZ): {len(xyz_files)}個")
        summary_text.append(f"- Trajectoryファイル: {len(traj_files)}個")
        summary_text.append(f"")
        summary_text.append("処理フェーズ:")
        summary_text.append("1. NMC組成バリアント生成")
        summary_text.append("2. Al/NMC表面構造作成")
        summary_text.append("3. Trajectoryファイル整理")
        summary_text.append("4. 界面構築・トリミング")
        summary_text.append("5. Matlantis最適化")
        summary_text.append("6. 最終構造保存")
        
        summary_path = self.output_dir / "execution_summary.txt"
        with open(summary_path, "w", encoding="utf-8") as f:
            f.write("\n".join(summary_text))
        
        print(f"\n💾 実行サマリーを保存しました: {summary_path}")
        print("=" * 70)
    
    def run_complete_workflow(self):
        """完全統合ワークフローの実行"""
        print("🚀 統合NMC-Al界面構造システム 開始")
        print("=" * 70)
        
        start_time = time.time()
        
        try:
            # =================== PHASE 1: 構造生成 ===================
            print("\n🔬 PHASE 1: NMC構造生成フェーズ")
            print("-" * 50)
            
            # 基本構造の読み込みまたは作成
            base_nmc = self.load_or_create_nmc_base()
            
            # 組成変動構造の作成
            nmc_variants = self.create_composition_variants(base_nmc, "NMC111")
            
            # Al構造の構築
            al_bulk, al_surfaces = self.build_al_structures()
            
            # 全構造の統合
            all_structures = {"Al_bulk": al_bulk}
            all_structures.update(nmc_variants)
            all_surfaces = al_surfaces.copy()
            
            # NMC表面の作成
            print("\n=== NMC表面構造作成 ===")
            for variant_name, atoms in nmc_variants.items():
                for miller in self.miller_indices[:4]:  # NMC主要面
                    try:
                        surface_name = f"{variant_name}_{miller[0]}{miller[1]}{miller[2]}"
                        slab = self.makesurface_advanced(atoms, miller_indices=miller,
                                                       name=surface_name)
                        if slab is not None:
                            all_surfaces[surface_name] = slab
                    except Exception as e:
                        print(f"表面作成失敗: {surface_name} - {e}")
            
            # 構造の保存
            self.save_structures(all_structures, "バルク構造")
            self.save_structures(all_surfaces, "表面構造")
            
            print(f"✅ PHASE 1 完了: バルク{len(all_structures)}個, 表面{len(all_surfaces)}個")
            
            # =================== PHASE 2: ファイル整理 ===================
            print("\n📁 PHASE 2: Trajectoryファイル整理フェーズ")
            print("-" * 50)
            
            self.cleanup_trajectory_files()
            print("✅ PHASE 2 完了")
            
            # =================== PHASE 3: 界面最適化 ===================
            print("\n⚙️ PHASE 3: 界面構築・最適化フェーズ")
            print("-" * 50)
            
            self.process_all_interfaces()
            print("✅ PHASE 3 完了")
            
            # =================== 最終サマリー ===================
            execution_time = time.time() - start_time
            print(f"\n⏱️ 総実行時間: {execution_time:.1f} 秒")
            
            self.generate_final_summary()
            
            return True
            
        except Exception as e:
            print(f"\n❌ ワークフロー実行中に重大なエラーが発生しました: {e}")
            print("エラーが発生しても、一部の処理は完了している可能性があります。")
            print("出力ディレクトリを確認してください。")
            return False


def main():
    """メイン実行関数"""
    print("=" * 80)
    print("🧪 統合NMC-Al界面構造生成・最適化システム")
    print("   リチウムイオン電池リサイクル研究用")
    print("=" * 80)
    
    # システム初期化
    try:
        system = IntegratedNMCInterfaceSystem()
        
        # 完全ワークフローの実行
        success = system.run_complete_workflow()
        
        if success:
            print("\n🎉 全てのワークフローが正常に完了しました！")
            print(f"📂 結果ファイル: {system.interfaces_dir}")
            print(f"📊 実行ログ: {system.output_dir / 'execution_summary.txt'}")
        else:
            print("\n⚠️ 一部のプロセスでエラーが発生しましたが、")
            print("   完了した処理の結果は保存されています。")
            
    except Exception as e:
        print(f"\n💥 システム初期化またはメイン実行でエラーが発生: {e}")
        print("環境設定または依存関係を確認してください。")
        
        # 最小限のフォールバック処理
        print("\n🔄 フォールバック: 基本的なNMC構造のみ生成します...")
        try:
            fallback_system = IntegratedNMCInterfaceSystem()
            base_nmc = fallback_system.load_or_create_nmc_base()
            
            # 基本構造のみ保存
            write(str(fallback_system.output_dir / "NMC111_fallback.xyz"), base_nmc)
            print(f"✅ フォールバック完了: {fallback_system.output_dir / 'NMC111_fallback.xyz'}")
            
        except Exception as fallback_error:
            print(f"❌ フォールバック処理も失敗: {fallback_error}")


if __name__ == "__main__":
    main()


# =================== 追加ユーティリティ関数 ===================

def analyze_interface_files(interfaces_dir):
    """界面ファイルの詳細分析"""
    from pathlib import Path
    
    interfaces_path = Path(interfaces_dir)
    if not interfaces_path.exists():
        print(f"ディレクトリが見つかりません: {interfaces_dir}")
        return
    
    print(f"\n🔍 界面構造ファイルの詳細分析")
    print(f"対象ディレクトリ: {interfaces_path}")
    print("-" * 60)
    
    cif_files = list(interfaces_path.glob("*.cif"))
    
    if not cif_files:
        print("CIFファイルが見つかりません。")
        return
    
    composition_analysis = {}
    miller_analysis = {}
    
    for cif_file in cif_files:
        try:
            atoms = read(str(cif_file))
            
            # 組成分析
            symbols = [atom.symbol for atom in atoms]
            composition = Counter(symbols)
            
            # ファイル名からNMC組成とMiller指数を抽出
            filename = cif_file.stem
            
            # NMC組成の抽出
            if "NMC111_Ni_" in filename:
                nmc_type = "Ni_variant"
            elif "NMC111_Mn_" in filename:
                nmc_type = "Mn_variant"
            elif "NMC111_Co_" in filename:
                nmc_type = "Co_variant"
            elif "NMC111_NMC" in filename:
                nmc_type = "High_Ni"
            else:
                nmc_type = "Standard"
            
            if nmc_type not in composition_analysis:
                composition_analysis[nmc_type] = []
            composition_analysis[nmc_type].append({
                'file': cif_file.name,
                'atoms': len(atoms),
                'composition': dict(composition)
            })
            
            # Miller指数の抽出（ファイル名から）
            for miller_str in ["001", "104", "012", "010", "111", "100", "110"]:
                if miller_str in filename:
                    if miller_str not in miller_analysis:
                        miller_analysis[miller_str] = 0
                    miller_analysis[miller_str] += 1
                    break
            
        except Exception as e:
            print(f"ファイル分析エラー: {cif_file.name} - {e}")
    
    # 結果の表示
    print(f"📊 組成別界面構造数:")
    for comp_type, files in composition_analysis.items():
        print(f"  {comp_type}: {len(files)}個")
        for file_info in files[:3]:  # 最初の3個のみ表示
            print(f"    - {file_info['file']} ({file_info['atoms']} atoms)")
    
    print(f"\n📐 Miller指数別界面構造数:")
    for miller, count in miller_analysis.items():
        print(f"  {miller}: {count}個")
    
    return composition_analysis, miller_analysis


def create_batch_visualization_script(interfaces_dir):
    """界面構造の一括可視化スクリプトを生成"""
    script_content = f'''#!/usr/bin/env python3
"""
NMC-Al界面構造一括可視化スクリプト
自動生成日: {time.strftime('%Y-%m-%d %H:%M:%S')}
"""

import matplotlib.pyplot as plt
from ase.io import read
from ase.visualize.plot import plot_atoms
from pathlib import Path
import numpy as np

def visualize_interfaces():
    interfaces_dir = Path(r"{interfaces_dir}")
    cif_files = list(interfaces_dir.glob("*.cif"))
    
    if not cif_files:
        print("CIFファイルが見つかりません")
        return
    
    # 最初の6個の界面構造を可視化
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, cif_file in enumerate(cif_files[:6]):
        try:
            atoms = read(str(cif_file))
            
            # 側面図を描画
            plot_atoms(atoms, axes[i], radii=0.5, rotation=('90x,0y,0z'))
            axes[i].set_title(f"{{cif_file.stem[:30]}}")
            axes[i].set_aspect('equal')
            
        except Exception as e:
            print(f"可視化エラー: {{cif_file.name}} - {{e}}")
            axes[i].text(0.5, 0.5, f'エラー\\n{{cif_file.stem}}', 
                        ha='center', va='center', transform=axes[i].transAxes)
    
    plt.tight_layout()
    plt.savefig(interfaces_dir / "interface_overview.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"可視化完了: {{interfaces_dir / 'interface_overview.png'}}")

if __name__ == "__main__":
    visualize_interfaces()
'''
    
    script_path = Path(interfaces_dir) / "visualize_interfaces.py"
    with open(script_path, 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    print(f"📊 可視化スクリプトを生成しました: {script_path}")
    return script_path


# =================== 使用例とテスト関数 ===================

def run_quick_test():
    """システムの動作確認用クイックテスト"""
    print("🧪 統合システム動作確認テスト開始")
    print("-" * 50)
    
    try:
        # テスト用システム初期化
        test_system = IntegratedNMCInterfaceSystem()
        
        # 基本NMC構造の生成テスト
        print("1. NMC構造生成テスト...")
        base_nmc = test_system.create_nmc_structure()
        print(f"   ✅ NMC構造: {len(base_nmc)} atoms")
        
        # Al構造生成テスト
        print("2. Al構造生成テスト...")
        al_slab = test_system.prepare_al_slab(size=(3,3,5))
        print(f"   ✅ Al構造: {len(al_slab)} atoms")
        
        # 組成変動の小規模テスト
        print("3. 組成変動テスト...")
        test_composition = {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}
        variant = test_system.create_structure_from_target(
            base_nmc, test_composition, ["Ni", "Mn", "Co"])
        if variant:
            print(f"   ✅ 組成変動: {len(variant)} atoms")
        
        print("✅ 全テスト通過")
        return True
        
    except Exception as e:
        print(f"❌ テスト失敗: {e}")
        return False


if __name__ == "__main__":
    # 通常実行
    main()
    
    # オプション: テストモードでの実行
    # run_quick_test()

/home/jovyan/.py311/lib/python3.11/site-packages/pfp_api_client/__init__.py:42: UserWarning: New version of pfp-api-client is available. Please consider upgrading by `pip install -U pfp-api-client`.
  warnings.warn(
/home/jovyan/.py311/lib/python3.11/site-packages/pfp_api_client/__init__.py:42: UserWarning: New version of matlantis-features is available. Please consider upgrading by `pip install -U matlantis-features`.
  warnings.warn(


✅ PFP/Matlantis計算器を正常に初期化しました
🧪 統合NMC-Al界面構造生成・最適化システム
   リチウムイオン電池リサイクル研究用

💥 システム初期化またはメイン実行でエラーが発生: 'str' object has no attribute 'mkdir'
環境設定または依存関係を確認してください。

🔄 フォールバック: 基本的なNMC構造のみ生成します...
❌ フォールバック処理も失敗: 'str' object has no attribute 'mkdir'
🧪 統合NMC-Al界面構造生成・最適化システム
   リチウムイオン電池リサイクル研究用

💥 システム初期化またはメイン実行でエラーが発生: 'str' object has no attribute 'mkdir'
環境設定または依存関係を確認してください。

🔄 フォールバック: 基本的なNMC構造のみ生成します...
❌ フォールバック処理も失敗: 'str' object has no attribute 'mkdir'


In [ ]:
#!/usr/bin/env python3
"""
NMC系材料とAl界面の体系的構造構築・最適化プログラム
リチウムイオン電池リサイクル研究用

Features:
- 段階的構造最適化
- 組成変動NMC構造の自動生成
- 体系的な表面・界面構築
- 複数ミラー指数対応
- カレンダリング圧力シミュレーション対応
"""


# PFP (Matlantis)
try:
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    from pfp_api_client.pfp.estimator import Estimator
    from pfp_api_client.pfp.estimator import EstimatorCalcMode
    estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
    calculator = ASECalculator(estimator)
    print("✅ PFP/Matlantis計算器を正常に初期化しました")
except ImportError:
    print("❌ PFP/Matlantisが利用できません。代替計算器を設定してください。")
    calculator = None

class NMCInterfaceBuilder:
    """NMC-Al界面構造構築クラス"""
    
    def __init__(self, base_dir="/home/jovyan/Kaori/MD/LiB_2"):
        self.base_dir = base_dir
        self.input_dir = os.path.join(base_dir, "input")
        self.output_dir = os.path.join(base_dir, "structure", "output")
        os.makedirs(self.output_dir, exist_ok=True)
        
        # 計算器設定
        self.calculator = calculator
        
        # Miller指数リスト（重要度順、NMC111最頻面を反映）
        self.miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 0, 4),  # NMC(104) - 中活性、多面体形状
            (0, 1, 2),  # NMC(012) - 高活性、切頭八面体形状
            (0, 1, 0),  # NMC(010) - Li拡散活性面
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        
        # カレンダリング圧力設定（MPa）
        self.calendering_pressures = [10, 30, 60, 120, 200]  # 実用範囲
        
    def optimize_structure_systematic(self, atoms, name="structure"):
        """段階的構造最適化"""
        print(f"=== {name}の構造最適化開始 ===")
        
        if self.calculator is None:
            print("計算器が利用できないため、最適化をスキップします")
            return atoms
            
        atoms.calc = self.calculator
        
        # Step 1: 格子定数最適化
        print("Step 1: 格子定数最適化")
        try:
            cell_opt = BFGS(UnitCellFilter(atoms), 
                           trajectory=f"{self.output_dir}/{name}_cell_opt.traj")
            cell_opt.run(fmax=0.01, steps=100)
            print(f"格子最適化完了: {atoms.cell.cellpar()}")
        except Exception as e:
            print(f"格子最適化警告: {e}")
        
        # Step 2: 原子位置最適化
        print("Step 2: 原子位置最適化")
        try:
            pos_opt = BFGS(atoms, 
                          trajectory=f"{self.output_dir}/{name}_pos_opt.traj")
            pos_opt.run(fmax=0.005, steps=200)
            print(f"位置最適化完了: エネルギー = {atoms.get_potential_energy():.3f} eV")
        except Exception as e:
            print(f"位置最適化警告: {e}")
        
        return atoms
    
    def makesurface_advanced(self, atoms, miller_indices=(1,0,0), layers=4, 
                           rep=[4,4,1], vacuum=15.0, fix_ratio=0.3, name="surface"):
        """改良版表面作成関数"""
        print(f"表面作成: {name} {miller_indices}面, {layers}層")
        
        try:
            # 表面作成
            slab = surface(atoms, miller_indices, layers, vacuum=vacuum)
            slab = slab.repeat(rep)
            
            # 位置調整
            positions = slab.get_positions()
            z_min = positions[:, 2].min()
            slab.set_positions(positions - [0, 0, z_min])
            
            # 下層原子の固定（fix_ratio分を固定）
            z_coords = slab.positions[:, 2]
            z_max = z_coords.max()
            z_threshold = z_coords.min() + (z_max - z_coords.min()) * fix_ratio
            fixed_indices = [i for i, z in enumerate(z_coords) if z <= z_threshold]
            
            if fixed_indices:
                constraint = FixAtoms(indices=fixed_indices)
                slab.set_constraint(constraint)
                print(f"固定原子数: {len(fixed_indices)}/{len(slab)}")
            
            # 構造最適化
            slab = self.optimize_structure_systematic(slab, f"{name}_{miller_indices}")
            
            return slab
        except Exception as e:
            print(f"表面作成エラー: {e}")
            return None
    
    def create_structure_from_target(self, base_atoms, target_composition, transition_metals):
        """
        ベース構造と目標組成から、新しい組成の構造を作成する関数
        
        Args:
            base_atoms (Atoms): 元となる構造オブジェクト
            target_composition (dict): 目標組成 (例: {"Ni": 0.5, "Mn": 0.2, "Co": 0.3})
            transition_metals (list): 置換対象となる遷移金属元素のリスト
        
        Returns:
            Atoms: 新しい組成を持つ構造オブジェクト
        """
        # 目標組成の合計が1.0になっているかチェック
        if not math.isclose(sum(target_composition.values()), 1.0):
            raise ValueError("目標組成の合計が1.0になりません。")
    
        variant_atoms = base_atoms.copy()
    
        # 1. 置換対象となる原子のインデックスをすべて取得
        tm_indices = [i for i, atom in enumerate(variant_atoms) if atom.symbol in transition_metals]
        total_tm_sites = len(tm_indices)
    
        if total_tm_sites == 0:
            print(f"警告: 置換対象の元素 {transition_metals} が見つかりません")
            return None
    
        # 2. 目標組成に基づいて、各元素の目標原子数を計算
        target_counts = {symbol: round(fraction * total_tm_sites) for symbol, fraction in target_composition.items()}
    
        # 四捨五入による過不足を調整
        current_total = sum(target_counts.values())
        diff = total_tm_sites - current_total
        if diff != 0:
            # 最も割合が大きい元素で数を調整する
            most_abundant_element = max(target_composition, key=target_composition.get)
            target_counts[most_abundant_element] += diff
    
        # 3. 新しい元素リストを作成
        new_symbols = []
        for symbol, count in target_counts.items():
            new_symbols.extend([symbol] * count)
    
        # 4. ランダムに原子を再配置
        random.seed(42)  # 再現性のためにseedを固定
        random.shuffle(tm_indices)  # インデックスをシャッフル
    
        for i, atom_index in enumerate(tm_indices):
            variant_atoms[atom_index].symbol = new_symbols[i]
    
        return variant_atoms

    def create_variants_from_targets(self, base_atoms, target_list, base_name="NMC"):
        """
        目標組成のリストを受け取り、ループで複数の構造バリアントを作成する
        """
        print(f"=== {base_name}の目標組成リストに基づく構造作成 ===")
        variants = {}

        for target in target_list:
            name = target["name"]
            composition = target["composition"]
            
            print(f"\n--- {name} の作成開始 ---")
            print(f"目標組成: {composition}")

            try:
                variant_atoms = self.create_structure_from_target(
                    base_atoms,
                    composition,
                    ["Ni", "Mn", "Co"]
                )

                if variant_atoms is not None:
                    variant_name = f"{base_name}_{name}"
                    variants[variant_name] = variant_atoms
                    print(f"完了: {variant_name}")
                else:
                    print(f"失敗: {name}")
            except Exception as e:
                print(f"エラー: {name} - {e}")
        
        return variants
    
    def _analyze_composition(self, atoms):
        """組成分析"""
        symbols = [atom.symbol for atom in atoms]
        composition = Counter(symbols)
        
        # 遷移金属のみの比率計算
        tm_elements = ['Ni', 'Mn', 'Co']
        tm_counts = {elem: composition.get(elem, 0) for elem in tm_elements}
        total_tm = sum(tm_counts.values())
        
        if total_tm > 0:
            tm_ratios = {elem: count/total_tm for elem, count in tm_counts.items()}
            return f"Li{composition.get('Li', 0)}[Ni{tm_ratios['Ni']:.2f}Mn{tm_ratios['Mn']:.2f}Co{tm_ratios['Co']:.2f}]O{composition.get('O', 0)}"
        else:
            return str(dict(composition))

    def prepare_al_slab(self, size=(6, 6, 10), vacuum=5.0):
        """標準Al(100)スラブモデルを作成"""
        from ase.build import fcc100
        slab_al = fcc100("Al", size=size, orthogonal=True, periodic=True, vacuum=vacuum)
        print(f"Al(100)スラブ作成: サイズ={size}, セルサイズ={slab_al.cell[0,0]:.3f}×{slab_al.cell[1,1]:.3f}×{slab_al.cell[2,2]:.3f}Å")
        return slab_al
    
    def build_al_structures(self):
        """Al構造の構築"""
        print("=== Al構造構築 ===")
        
        # Al bulk構造作成
        al_bulk = self.prepare_al_slab(size=(5,5,10), vacuum=1.0)
        
        # Al表面構築
        al_surfaces = {}
        for miller in self.miller_indices[:3]:  # Al主要面のみ
            surface_name = f"Al_{miller[0]}{miller[1]}{miller[2]}"
            al_slab = self.makesurface_advanced(al_bulk, miller_indices=miller, 
                                              name=surface_name)
            if al_slab is not None:
                al_surfaces[surface_name] = al_slab
            
        return al_bulk, al_surfaces
    
    def adjust_slab_size(self, slab, target_area=100.0):
        """スラブサイズの調整（格子不整合最小化）"""
        current_area = np.linalg.norm(np.cross(slab.cell[0], slab.cell[1]))
        scale_factor = np.sqrt(target_area / current_area)
        
        rep_x = max(1, int(scale_factor))
        rep_y = max(1, int(scale_factor))
        
        return slab.repeat([rep_x, rep_y, 1])
    
    def create_interface(self, slab1, slab2, separation=2.0, name="interface"):
        """界面構造作成"""
        print(f"界面作成: {name}, 分離距離: {separation} Å")
        
        try:
            # サイズ調整（格子不整合の最小化）
            area1 = np.linalg.norm(np.cross(slab1.cell[0], slab1.cell[1]))
            area2 = np.linalg.norm(np.cross(slab2.cell[0], slab2.cell[1]))
            target_area = max(area1, area2)
            
            slab1_adj = self.adjust_slab_size(slab1, target_area)
            slab2_adj = self.adjust_slab_size(slab2, target_area)
            
            # 界面作成
            interface = stack(slab1_adj, slab2_adj, axis=2, distance=separation)
            
            # 制約設定（両端を固定）
            z_coords = interface.positions[:, 2]
            z_min, z_max = z_coords.min(), z_coords.max()
            z_range = z_max - z_min
            
            # 下部20%と上部20%を固定
            bottom_threshold = z_min + z_range * 0.2
            top_threshold = z_max - z_range * 0.2
            
            fixed_indices = [i for i, z in enumerate(z_coords) 
                            if z <= bottom_threshold or z >= top_threshold]
            
            if fixed_indices:
                constraint = FixAtoms(indices=fixed_indices)
                interface.set_constraint(constraint)
            
            # 構造最適化
            interface = self.optimize_structure_systematic(interface, name)
            
            return interface
        except Exception as e:
            print(f"界面作成エラー: {e}")
            return None
    
    def build_all_structures(self, cif_files):
        """全構造の体系的構築"""
        print("=" * 50)
        print("NMC-Al界面構造体系的構築開始")
        print("=" * 50)
        
        all_structures = {}
        all_surfaces = {}
        all_interfaces = {}
        
        # 1. Al構造構築
        al_bulk, al_surfaces = self.build_al_structures()
        all_structures["Al_bulk"] = al_bulk
        all_surfaces.update(al_surfaces)
        
        # 2. NMC系構造構築
        for material_name, cif_path in cif_files.items():
            if not os.path.exists(cif_path):
                print(f"警告: {cif_path}が見つかりません")
                continue
                
            print(f"\n=== {material_name}構造構築 ===")
            
            try:
                # 基本構造読み込み・最適化
                base_atoms = read(cif_path)
                base_atoms = self.optimize_structure_systematic(base_atoms, material_name)
                all_structures[material_name] = base_atoms
                
                # 組成変動構造作成（NMC系のみ）
                variants = {}
                if "NMC" in material_name or "Li" in material_name:
                    # シミュレーション計画の定義
                    simulation_plan = [
                        # ニッケル(Ni)増減の影響
                        {"name": "Ni_increase", "composition": {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}},
                        {"name": "Ni_decrease", "composition": {"Ni": 0.2, "Mn": 0.4, "Co": 0.4}},
                        
                        # マンガン(Mn)増減の影響
                        {"name": "Mn_increase", "composition": {"Ni": 0.2, "Mn": 0.6, "Co": 0.2}},
                        {"name": "Mn_decrease", "composition": {"Ni": 0.4, "Mn": 0.2, "Co": 0.4}},
                        
                        # コバルト(Co)増減の影響
                        {"name": "Co_increase", "composition": {"Ni": 0.2, "Mn": 0.2, "Co": 0.6}},
                        {"name": "Co_decrease", "composition": {"Ni": 0.4, "Mn": 0.4, "Co": 0.2}},
                        
                        # ハイニッケル化トレンド
                        {"name": "NMC523_approx", "composition": {"Ni": 0.50, "Mn": 0.25, "Co": 0.25}},
                        {"name": "NMC622_approx", "composition": {"Ni": 0.60, "Mn": 0.20, "Co": 0.20}},
                        {"name": "NMC811_approx", "composition": {"Ni": 0.80, "Mn": 0.10, "Co": 0.10}},
                    ]
                    
                    # 組成変動構造の生成
                    generated_variants = self.create_variants_from_targets(
                        base_atoms=base_atoms,
                        target_list=simulation_plan,
                        base_name=material_name
                    )
                    variants.update(generated_variants)
                else:
                    variants = {material_name: base_atoms}
                
                all_structures.update(variants)
                
                # 各材料・組成の表面構築
                for variant_name, atoms in variants.items():
                    for miller in self.miller_indices:
                        try:
                            surface_name = f"{variant_name}_{miller[0]}{miller[1]}{miller[2]}"
                            slab = self.makesurface_advanced(atoms, miller_indices=miller,
                                                           name=surface_name)
                            if slab is not None:
                                all_surfaces[surface_name] = slab
                        except Exception as e:
                            print(f"表面作成失敗: {surface_name}, エラー: {e}")
                            
            except Exception as e:
                print(f"材料構築失敗: {material_name}, エラー: {e}")
        
        print(f"\n構造構築完了:")
        print(f"  - バルク構造: {len(all_structures)}")
        print(f"  - 表面構造: {len(all_surfaces)}")
        
        return all_structures, all_surfaces, all_interfaces
    
    def save_all_structures(self, structures, surfaces, interfaces):
        """全構造の保存"""
        print("\n=== 構造ファイル保存 ===")
        
        # ディレクトリ作成
        dirs = ["bulk", "surfaces", "interfaces"]
        for d in dirs:
            os.makedirs(os.path.join(self.output_dir, d), exist_ok=True)
        
        # バルク構造保存
        for name, atoms in structures.items():
            try:
                filepath = os.path.join(self.output_dir, "bulk", f"{name}.traj")
                write(filepath, atoms)
                print(f"保存: {filepath}")
            except Exception as e:
                print(f"保存失敗: {name} - {e}")
        
        # 表面構造保存
        for name, atoms in surfaces.items():
            try:
                filepath = os.path.join(self.output_dir, "surfaces", f"{name}.traj")
                write(filepath, atoms)
            except Exception as e:
                print(f"表面保存失敗: {name} - {e}")
        
        # 界面構造保存
        for name, atoms in interfaces.items():
            try:
                filepath = os.path.join(self.output_dir, "interfaces", f"{name}.traj")
                write(filepath, atoms)
            except Exception as e:
                print(f"界面保存失敗: {name} - {e}")
        
        print(f"総保存数: バルク{len(structures)}, 表面{len(surfaces)}, 界面{len(interfaces)}")
    
    def generate_analysis_summary(self, structures, surfaces, interfaces):
        """解析サマリー生成"""
        summary = []
        summary.append("=" * 60)
        summary.append("NMC-Al界面構造構築サマリー")
        summary.append("=" * 60)
        
        summary.append(f"\n構築した構造数:")
        summary.append(f"  - バルク構造: {len(structures)}")
        summary.append(f"  - 表面構造: {len(surfaces)}")
        summary.append(f"  - 界面構造: {len(interfaces)}")
        
        summary.append(f"\n主要材料:")
        for name in structures.keys():
            if "NMC" in name:
                composition = self._analyze_composition(structures[name])
                summary.append(f"  - {name}: {composition}")
            else:
                summary.append(f"  - {name}")
        
        summary.append(f"\n検討ミラー指数:")
        miller_descriptions = {
            (0, 0, 1): "最安定面、プレート形状",
            (1, 0, 4): "中活性、多面体形状", 
            (0, 1, 2): "高活性、切頭八面体形状",
            (0, 1, 0): "Li拡散活性面",
            (1, 1, 1): "Al最安定面",
            (1, 0, 0): "基本面",
            (1, 1, 0): "副次面"
        }
        for miller in self.miller_indices:
            desc = miller_descriptions.get(miller, "")
            summary.append(f"  - {miller} : {desc}")
        
        summary_text = "\n".join(summary)
        
        # ファイル保存
        with open(os.path.join(self.output_dir, "analysis_summary.txt"), "w") as f:
            f.write(summary_text)
        
        print(summary_text)
        return summary_text

def main():
    """メイン実行関数"""
    
    # ビルダー初期化
    builder = NMCInterfaceBuilder()
    
    # CIFファイル設定
    cif_files = {
        "NMC111": "/home/jovyan/Kaori/MD/input/Li3MnCoNiO6.cif",
        # 以下は利用可能な場合に追加
        # "Li2O": "/home/jovyan/Kaori/MD/input/Li2O.cif",
        # "CoO": "/home/jovyan/Kaori/MD/input/CoO.cif", 
        # "MnO": "/home/jovyan/Kaori/MD/input/MnO.cif",
        # "NiO": "/home/jovyan/Kaori/MD/input/NiO.cif",
    }
    
    # 利用可能なCIFファイルのみフィルター
    available_cifs = {name: path for name, path in cif_files.items() 
                     if os.path.exists(path)}
    
    if not available_cifs:
        print("エラー: 利用可能なCIFファイルがありません")
        print("必要なCIFファイルを /home/jovyan/Kaori/MD/input/ に配置してください")
        return
    
    print(f"利用可能なCIFファイル: {list(available_cifs.keys())}")
    
    # 全構造構築
    structures, surfaces, interfaces = builder.build_all_structures(available_cifs)
    
    # 結果保存
    builder.save_all_structures(structures, surfaces, interfaces)
    
    # サマリー生成
    builder.generate_analysis_summary(structures, surfaces, interfaces)
    
    print("\n構築完了！次はカレンダリングシミュレーションに進んでください。")

if __name__ == "__main__":
    main()

/home/jovyan/.py311/lib/python3.11/site-packages/pfp_api_client/__init__.py:42: UserWarning: New version of pfp-api-client is available. Please consider upgrading by `pip install -U pfp-api-client`.
  warnings.warn(
/tmp/ipykernel_9373/2154063940.py:83: FutureWarning: Import UnitCellFilter from ase.filters
  cell_opt = BFGS(UnitCellFilter(atoms),


✅ PFP/Matlantis計算器を正常に初期化しました
利用可能なCIFファイル: ['NMC111']
NMC-Al界面構造体系的構築開始
=== Al構造構築 ===
Al(100)スラブ作成: サイズ=(5, 5, 10), セルサイズ=14.319×14.319×20.225Å
表面作成: Al_001 (0, 0, 1)面, 4層
固定原子数: 4800/16000
=== Al_001_(0, 0, 1)の構造最適化開始 ===
Step 1: 格子定数最適化
